# Create, modify and work with a general network.

In [3]:

import numpy as np
import random
from pymnet import *
import tensorflow as tf
import pandas as pd
import math
import matplotlib.pyplot as plt
import geopandas as gpd

import heapq
import copy
from collections import defaultdict



## Intro

In [4]:
# IMPORTANT: No aspects can be added to an empty network, 
# a 0 aspect network is just a monoplex
monoplex = net.MultilayerNetwork(
    aspects=0,
    noEdge=True,
    directed=False,
    fullyInterconnected=False,
)

monoplex.add_node('a')
monoplex.add_node('b')
monoplex.add_node('c')

monoplex['a','b'] = 1
monoplex['b','c'] = 2
monoplex['c','c'] = 3


In [5]:
multinetwork = net.MultilayerNetwork(
    aspects=1,
    noEdge=True,
    directed=False,
    fullyInterconnected=False,
)

multinetwork.add_layer('l1', 1)
multinetwork.add_layer('l2', 1)

multinetwork.add_node('a', 'l1')
multinetwork.add_node('c', 'l1')
multinetwork.add_node('a', 'l2')
multinetwork.add_node('b', 'l2')

multinetwork['a','b','l1','l2'] = 1
multinetwork['b','c','l2','l1'] = 2
multinetwork['a','a','l1','l2'] = 3
multinetwork['c','a','l1','l1'] = 4


## 1.1. ---Tensorflow functionality---

In [54]:
def __init_from_sparse_tensor__(sparse_tensor,  nodes_names=None, aspects_names=None,directed=False):
    """Initialize a MultilayerNetwork from a sparse tensor.
    Parameters
    ----------
    sparse_tensor : tf.SparseTensor
        The sparse tensor to initialize the network from.  
            Given a sparse tensor of shape (a1,a2,b1,b2,...,an1,an2,n1,n2),
            the first n dimensions are the aspects, and the last two dimensions
            are the nodes. The values of the sparse tensor are the weights of 
    nodes_names : list of str
        The names of the nodes in the network.
    aspects_names : list of list of str
        For each aspect, the names of the layers in the aspect.
    directed : bool
        Whether the network is directed or not. If True, the network is directed.
        If False, the network is undirected. Default is False.
    """

    if sparse_tensor.shape.rank < 2:
        raise ValueError("The sparse tensor must have at least 2 dimensions.")
    
    if len(sparse_tensor.shape[:-2]) % 2 != 0:
        raise ValueError("The sparse tensor must have an even number of dimensions.")
        
    if any([sparse_tensor.shape[i] != sparse_tensor.shape[i+1] for i in range(0, sparse_tensor.shape.rank-2, 2)]):
        raise ValueError("The sparse tensor must have the same size in the first and second dimensions for each aspect.")
    
    if nodes_names and len(nodes_names) != sparse_tensor.shape[-2]:
        raise ValueError("The number of nodes must be equal to the size of the last two dimensions in the sparse tensor.")
    
    if nodes_names is None:
        nodes_names = [f"n{i}" for i in range(sparse_tensor.shape[-2])]

    if aspects_names:
        if len(aspects_names) != sparse_tensor.shape[:-2].rank/2:
            raise ValueError("The number of aspects must be equal to the number of non-nodes dimensions in the sparse tensor: ", sparse_tensor.shape[:-2])
    
        for i in range(len(aspects_names)):
            if len(aspects_names[i]) != sparse_tensor.shape[2*i] or len(aspects_names[i]) != sparse_tensor.shape[2*i+1]:
                raise ValueError(f"The number of layers in aspect {i} must be equal to the size of dimension {i} in the sparse tensor.")
    else:
        aspects_names = [ [f'l{i}_{j}' for j in range(sparse_tensor.shape[2*i])] for i in range(len(sparse_tensor.shape[:-2])//2) ]
    

    # Initialize the network
    n = MultilayerNetwork(
        aspects=len(aspects_names),
        noEdge=False,
        directed=directed,
        fullyInterconnected=True,
    )

    for node in nodes_names:
        n.add_node(node)

    for i in range(len(aspects_names)):
        for j in range(len(aspects_names[i])):
            n.add_layer(aspects_names[i][j], i+1)

    for (index, value) in zip(sparse_tensor.indices, sparse_tensor.values):

        aspects_index = index[:-2]
        if aspects_index is not None:
            edge = [nodes_names[index[-2]],  nodes_names[index[-1]]] + [aspects_names[i//2][aspects_index[i]] for i in range(len(aspects_index))]
        else:
            edge = [nodes_names[index[-2]],  nodes_names[index[-1]]]
        print(edge)
        n[tuple(edge)] = value

    return n
    

In [55]:
def __get_sparse_tensor__(self, return_mappings=False):
    """Get the sparse tensor representation of the network.
    Returns
    -------
    sparse_tensor : tf.SparseTensor
        The sparse tensor representation of the network.
    """
    
    if 'sparse_tensor' in self.__dict__ and self.sparse_tensor is not None:
        return self.sparse_tensor
    # definde dictionary with name and position of each node
    nodes = {}
    for i in range(len(self.slices[0])):
        nodes[list(self.slices[0])[i]] = i

        
    layers = [ {list(self.slices[i])[j] : j for j in range(len(self.slices[i]))} for i in range(1, self.aspects+1)]
    
    indices = []
    values = []
    for edge in self.edges:
        edge_nodes = edge[:2]
        edge_aspects = edge[2:-1]
        
        edge_aspects = [layers[i//2][edge_aspects[i]] for i in range(len(edge_aspects))]
        edge_nodes = [nodes[edge_nodes[i]] for i in range(len(edge_nodes))]

        indices.append(edge_nodes + edge_aspects)
        values.append(edge[-1])

        if not self.directed:
            inverted_edge = edge_nodes[::-1] + edge_aspects[::-1]
            indices.append(inverted_edge)
            values.append(edge[-1])
    
        

    indices = tf.constant(indices, dtype=tf.int64)
    values = tf.constant(values, dtype=tf.float32)
    
    # shape = [len(self.slices[i]) for i in range(self.aspects+1)]
    shape = [len(self.slices[i//2]) for i in range(0,(self.aspects+1)*2)]

    
    sparse_tensor = tf.SparseTensor(indices=indices, values=values, dense_shape=shape)
    sparse_tensor = tf.sparse.reorder(sparse_tensor)  # Ensure the indices are sorted
     
    if return_mappings:
        inv_nodes = {v: k for k, v in nodes.items()}
        inv_layers = [{v: k for k, v in l.items()} for l in layers]
        return sparse_tensor, nodes, layers, inv_nodes, inv_layers
    
    return sparse_tensor


def __get_sparse_tensor_between_layers__(self,l1,l2):
    """
    Get the sparse tensor representation of the network between two layers.
    
    Args:
        l1 (list of str/int) : The names of the first layer.
        l2 (list of str/int) : The names of the second layer. 
        
    Returns:
        tf.sparse.SparseTensor: The sparse tensor representation of the network between the two layers.
    """
    #generate a list fix_index = [l1[0], l2[0], l1[1], l2[1], ..., l1[n-1], l2[n-1]]

    
    sparse, nodes, layers, inv_nodes, inv_layers = __get_sparse_tensor__(self, return_mappings=True)
    for i in range(len(l1)):
        if isinstance(l1[i], str):
            l1[i] = layers[0][l1[i]]
        if isinstance(l2[i], str):
            l2[i] = layers[0][l2[i]]

    fixed_indices = [item for pair in zip(l1, l2) for item in pair]

    # fixed_indices = []
    # for i in range(len(l1)):
    #     fixed_indices.append(l1[i])
    #     fixed_indices.append(l2[i])
    

    return __slice_sparse_tensor__(sparse, fixed_indices)

def __slice_sparse_tensor__(sparse_tensor, fixed_indices):
    """
    Extracts a slice from a 2k-dimensional SparseTensor by fixing the last k dimensions.
    
    Args:
        sparse_tensor (tf.sparse.SparseTensor): The original sparse tensor of shape [d1, d2, ..., d_{2k}]
        fixed_indices (List[int]): A list of length k containing fixed indices for the last k dimensions.
        
    Returns:
        tf.sparse.SparseTensor: A sparse tensor of shape [d1, ..., d_k] corresponding to the slice.
    """
    
    fixed_indices = tf.convert_to_tensor(fixed_indices, dtype=tf.int64)
    num_dims = tf.shape(sparse_tensor.dense_shape)[0]
    k = tf.size(fixed_indices)

    
    # Split indices into first k and last k parts
    first_k = sparse_tensor.indices[:, :num_dims - k]
    last_k = sparse_tensor.indices[:, num_dims - k:]
    
    # Create mask for matching fixed_indices (login and between last_k and fixed_indices)
    mask = tf.reduce_all(tf.equal(last_k, fixed_indices), axis=1)

    
    # Filter indices and values
    new_indices = tf.boolean_mask(first_k, mask)
    new_values = tf.boolean_mask(sparse_tensor.values, mask)
    
    # New shape is the first k dimensions
    new_shape = sparse_tensor.dense_shape[:num_dims - k]
    
    return tf.SparseTensor(indices=new_indices, values=new_values, dense_shape=new_shape)


## 1.2. ---Tensroflow generation ---

In [39]:
def er_general_multilayer(
    n_layers_per_aspect : list,
    n_nodes : int,
    p=0.5,
    randomWeights=False,
    directed=False
):
    """
    Generate a random multilayer network with the given parameters.
    
    Parameters
    ----------
    n_layers_per_aspect : list of int
        The number of layers per aspect. Default is [2,2].
    n_nodes : int
        The number of nodes in the network. Default is 8.
    p : float
        The probability of creating an edge between two nodes. Default is 0.5.
    randomWeights : bool
        Whether to use random weights for the edges or not. Default is False.
    directed : bool
        Whether the network is directed or not. Default is False.
    fullyInterconnected : bool
        Determines if the network is fully interconnected, i.e. all nodes
       are shared between all layers.

    Returns
    -------
    net : MultilayerNetwork
        The generated multilayer network.
    
    """
    mnet = MultilayerNetwork(
        aspects=len(n_layers_per_aspect),
        directed=directed
    )

    for node in range(n_nodes):
        mnet.add_node(node)
    if not n_layers_per_aspect:
        for aspect in range(len(n_layers_per_aspect)):
            for layer in range(n_layers_per_aspect[aspect]):
                mnet.add_layer(layer, aspect+1)
    
    for edge in __generate_random_edges_er(n_nodes, n_layers_per_aspect, p):
        if randomWeights:
            weight = random.random()
        else:
            weight = 1
        mnet[tuple(edge)] = weight

    
    return mnet

def __generate_random_edges_er(n_nodes, aspect_sizes, p, directed=False):
    """
    Efficient generator of random edges in a multilayer graph.

    Each node-layer is a tuple of integers: (node, layer1, ..., layerN)

    Parameters:
        n_nodes (int): Number of nodes.
        aspect_sizes (list of int): Number of layers for each aspect.
        p (float): Probability of including an edge between two node-layers.

    Yields:
        tuple: An edge as a tuple of two node-layer tuples.
    """
    if len(aspect_sizes) > 0:
        total_layers = 1
        for size in aspect_sizes:
            total_layers *= size
        total_node_layers = n_nodes * total_layers

        # Helper: convert flat index to node-layer tuple
        def index_to_node_layer(idx):
            node = idx // total_layers
            rest = idx % total_layers
            layers = []
            for size in reversed(aspect_sizes):
                layers.append(rest % size)
                rest //= size
            return (node, *reversed(layers))

        for i in range(total_node_layers):
            for j in range(i + 1, total_node_layers):
                if random.random() < p:
                    nl1 = index_to_node_layer(i)
                    nl2 = index_to_node_layer(j)
                    yield (nl1[0], nl2[0], *nl1[1:], *nl2[1:])  # Flatten the tuple
    else:
        for i in range(n_nodes):
            for j in range(i + 1, n_nodes):
                if random.random() < p:
                    yield (i, j)

In [40]:
def ws_multiplex(
        n_nodes,
        n_layers,
        degrees = [],
        rewiring_probs = [],
) :
    """Generate multilayer Watts-Strogatz network.

    The produced multilayer network has a single aspect.

    Parameters
    ----------
    n_nodes : int
        Number of nodes
    n_layers : int
        Number of layers
    degrees : list
        List of degrees for each layer
    p_rewiring : list
        List of rewiring probabilities for each layer
    """


    multplex = MultiplexNetwork(couplings='categorical')
    rewires = [] 
    for i in range(n_nodes):
        multplex.add_node(i)

    for i in range(n_layers):
        multplex.add_layer(i)

    for i in range(n_layers):
        deg = degrees[i]//2
        for j in range(n_nodes):
            for edges in range(deg):
                multplex.A[i][j,(j+edges+1)%n_nodes] = 1
                
    for i in range(n_layers):
        for j in range(n_nodes):
                # iterate on neighbors of node j in layer i
                neighbors = list(multplex._iter_neighbors_out((j,i),(None,i)))
                not_neighbors = _get_not_neighbors_layer(multplex, j, i)

                for neighbor in neighbors:
                    if random.random() < rewiring_probs[i] and len(not_neighbors) > 0:
                        rewired = random.choice(not_neighbors)
                        print(((j,i), neighbor), " to ", ((j,i), (rewired,i)))
                        multplex.A[i][j,neighbor[0]] = 0
                        multplex.A[i][j,rewired] = 1
                        rewire = ((j,i),(neighbor,i))
                        rewires.append(rewire)
                        
    
    return multplex, rewires

def _get_not_neighbors_layer(net, node, layer, loops = False):
    nodelayers = [(x,layer)  for x in list(net.iter_nodes(layer))]
    neighbors = list(net._iter_neighbors((node, layer),(None,layer)))
    not_neighbors = [node for node in nodelayers if node not in neighbors]
    if not loops:
        not_neighbors = [n[0] for n in not_neighbors if n != (node,layer)]
    return not_neighbors


In [41]:
def generate_random_multilayer_edges_streaming(n_nodes, aspects, p):
    """
    Generate random edges for a multilayer graph with minimal memory usage.
    Each edge connects two different node-layers with probability p.

    Parameters:
        n_nodes (list): List of node identifiers.
        aspects (list of lists): List of aspects, each a list of possible values.
        p (float): Probability of including an edge.

    Yields:
        tuple: An edge as a tuple of two node-layer tuples.
    """


    total_layers = 1
    for a in aspects:
        total_layers *= len(a)
    total_nodes = len(n_nodes) * total_layers

    # Use indexed access to avoid building the list
    def node_layer_at(index):
        node_index = index // total_layers
        layer_index = index % total_layers
        layer_values = []
        for a in reversed(aspects):
            layer_values.append(a[layer_index % len(a)])
            layer_index //= len(a)
        return (n_nodes[node_index], *reversed(layer_values))

    # Iterate through pairs by index (i < j) to avoid storing all pairs
    for i in range(total_nodes):
        for j in range(i + 1, total_nodes):
            if random.random() < p:
                yield (node_layer_at(i), node_layer_at(j)) 

## 1.3. ---Centrality measures--

####        PCI functions

In [16]:
def dijkstra(network, start, max_distance = 100000):
    """
    Given a multilayer network and a node of the network,
    this function returns a dictionary with the distances from the
    node to all the other nodes in the network.

    Parameters
    ----------
    n : MultilayerNetwork
        The multilayer network to analyze
    start : tuple
        The node to analyze
    max_distance : int
        The maximum distance to calculate

    Returns
    -------
    distances : dict
        A dictionary with the distances from the node to all the other nodes
    """
    prio_queue = [(0,start)]  
    distances = { start : 0}
    visited = set()

    while prio_queue:
        distance, node = heapq.heappop(prio_queue)
        if node in visited:
            continue
        visited.add(node)
        for neighbor in network._iter_neighbors(node):
            if neighbor not in visited and distance+1 <= max_distance and neighbor not in distances:
                heapq.heappush(prio_queue, (distance+1,neighbor))
                distances[neighbor] = distance+ network[network._nodes_to_link(node, neighbor)]
    
    return distances


In [17]:
def layer_connections(net, node):
    """
    Given a multilayer network and a node of the network,
    this function returns a dictionary with the number of connections
    of the node to each layer

    Parameters
    ----------
    n : MultilayerNetwork
        The multilayer network to analyze
    node : tuple
        The node to analyze

    Returns
    -------
    connections : dict
        A dictionary with the number of connections of the node to each layer
    """
    connections = {}
    for neighbor in net._iter_neighbors(node):
        layer = neighbor[1:]
        if layer in connections:
            connections[layer] += 1
        else:
            connections[layer] = 1
    return connections

In [18]:
import heapq
import numpy as np
import pymnet as net


def dijkstra(self, start, max_distance=100000):
    """
    Compute shortest-path distances from a given node in a multilayer network using Dijkstra's algorithm.

    Args:
        self (MultilayerNetwork): The multilayer network to analyze.
        start (tuple): The starting node (as a multilayer node tuple).
        max_distance (int): Maximum path length to explore.

    Returns:
        dict: A dictionary mapping nodes to their shortest-path distance from the `start` node.
    """
    prio_queue = [(0,start)]  
    distances = { start : 0}
    visited = set()

    while prio_queue:
        distance, node = heapq.heappop(prio_queue)
        if node in visited:
            continue
        visited.add(node)
        for neighbor in self._iter_neighbors(node):
            if type(self) is net.MultiplexNetwork and self._get_degree(neighbor) == len(self.slices):
                continue  # skip if the neighbor is a multiplex node that is not connected to any node 
            if neighbor not in visited and distance+1 <= max_distance and neighbor not in distances:
                heapq.heappush(prio_queue, (distance+1,neighbor))
                distances[neighbor] = distance+ self[self._nodes_to_link(node, neighbor)]
    
    return distances


def layer_connections(self, node):
    """
    Count how many neighbors a node has in each layer.

    Args:
        self (MultilayerNetwork): The multilayer network to analyze.
        node (tuple): The node to inspect.

    Returns:
        dict: A dictionary mapping each layer (tuple of layer identifiers) to the number of neighbors in that layer.
    """

    connections = {}
    for neighbor in self._iter_neighbors(node):
        if type(self) is net.MultiplexNetwork and self._get_degree(neighbor) == len(self.slices):
            continue  # skip if the neighbor is a multiplex node that is not connected to any node 
        layer = neighbor[1:]
        if layer in connections:
            connections[layer] += 1
        else:
            connections[layer] = 1
    return connections


def mu_PCI(self, node, mu=1):
    """
    Compute the μ-PCI of a node: the maximum number k of nodes within μ hops
    that each have a degree ≥ k.

    Args:
        self (MultilayerNetwork): The multilayer network to analyze.
        node (tuple): The node to evaluate.
        mu (int): Maximum path length to consider.

    Returns:
        int: The μ-PCI value for the given node.
    """

    distances = dijkstra(self, node, mu)
    # remove node from distances
    distances.pop(node, None)
    degrees = np.array([self._get_degree(node) for node in distances])
    # sort the degrees descending
    degrees = np.sort(degrees)[::-1]

    # indices = np.arange(1,len(degrees)+1)
    seq = (k for k, d in enumerate(degrees, start=1) if d >= k)
    if not seq:
        mu_PCI = 0
    else:
        mu_PCI = max(seq, default=0)
    return mu_PCI
        
def mlPCI_n(self, node, n=1):
    """
    Compute the ml-PCI_n of a node: the maximum number k of neighbors
    that are connected to at least n different layers with at least k edges.

    Args:
        self (MultilayerNetwork): The multilayer network to analyze.
        node (tuple): The node to evaluate.
        n (int): Number of layers a neighbor must be connected to.

    Returns:
        int: The ml-PCI_n value for the given node.
    """
    #layer_degrees is a numpy array with k rows and n columns, being k the number of neighbors of the node and n the parameter n
    layer_degrees = np.zeros(len([1 for _ in self._iter_neighbors(node)]))
    # for each neighbor of the node, get the number of connections to each layer and store it in layer_degrees
    for i, neighbor in enumerate(self._iter_neighbors(node)):
        if type(self) is net.MultiplexNetwork and self._get_degree(neighbor) == len(self.slices):
            continue  # skip if the neighbor is a multiplex node that is not connected to any node 
        if type(self) is net.MultiplexNetwork and neighbor[0] == node[0]:
            continue  # skip if the neighbor is the same node in a different layer
        layer_conn = layer_connections(self, neighbor)
        layer_conn = sorted(list(layer_conn.values()),reverse=True)
        if len(layer_conn) < n:
            layer_degrees[i] = 0
        else:
            ey = layer_conn[n-1]
            layer_degrees[i] = ey
        
    layer_degrees = sorted(layer_degrees,reverse=True)
    
    # mlPCI_n = np.max(np.where(layer_degrees >= indices, indices, 0))
    seq = (k for k, d in enumerate(layer_degrees, start=1) if d >= k)
    # if seq is empty, return 0
    if not seq:
        mlPCI_n = 0
    else:
    # get the maximum k such that d >= k
        mlPCI_n = max(seq, default=0)


    return mlPCI_n
                    
   
def allPCI(self, node):
    """
    Compute the all-PCI of a node: the maximum number k of neighbors
    that are connected to all layers in the network.

    Args:
        self (MultilayerNetwork): The multilayer network to analyze.
        node (tuple): The node to evaluate.

    Returns:
        int: The all-PCI value for the given node.
    """

    aspects_lens = np.array([len(self.slices[i]) for i in range(1,self.aspects+1)])
    n_layers = aspects_lens.prod()
    all_PCI = 0
    for neighbor in self._iter_neighbors(node):
        if type(self) is net.MultiplexNetwork and self._get_degree(neighbor) == len(self.slices):
            continue
        if type(self) is net.MultiplexNetwork and neighbor[0] == node[0]:
            continue
        layer_conn = layer_connections(self, neighbor)
        if len(layer_conn) < n_layers:
            continue
        all_PCI += 1
        
        
    return all_PCI 


def lsPCI(self, node):
    """
    Compute the ls-PCI of a node: the maximum number k such that the node
    has at least k neighbors, each connected to at least k different nodes
    in at least k different layers.

    Args:
        self (MultilayerNetwork): The multilayer network to analyze.
        node (tuple): The node to evaluate.

    Returns:
        int: The ls-PCI value for the given node.
    """

    neighbors_layer_degrees = []
    for neighbor in self._iter_neighbors(node):
        if type(self) is net.MultiplexNetwork and self._get_degree(neighbor) == len(self.slices):
            continue  # skip if the neighbor is a multiplex node that is not connected to any node 
        if type(self) is net.MultiplexNetwork and neighbor[0] == node[0]:
            continue
        layer_conn = layer_connections(self, neighbor)
        neighbors_layer_degrees.append(sorted(list(layer_conn.values()),reverse=True))
    dim2_seq = [len(layer_conn) for layer_conn in neighbors_layer_degrees]
    if len(dim2_seq) == 0:
        return 0
    

    nl_matrix = np.zeros((len(neighbors_layer_degrees),max([len(layer_conn) for layer_conn in neighbors_layer_degrees])))
    for i, layer_conn in enumerate(neighbors_layer_degrees):
        nl_matrix[i,:len(layer_conn)] = layer_conn

    lsPCI = 0
    for i in range(0,nl_matrix.shape[1]):
        discarded = np.where(nl_matrix[:,i] < i)[0]
        if len(discarded) > i:
            lsPCI = i
            break
    return lsPCI
    

#### Eigencentrality

In [19]:
def sparse_power_iteration(sparse_matrix, num_iters=1000):
    n = sparse_matrix.dense_shape[1]
    sparse_matrix = tf.sparse.reorder(sparse_matrix)  # ensure canonical order
    b_k = tf.random.normal([n, 1])  # column vector

    for _ in range(num_iters):

        b_k1 = tf.sparse.sparse_dense_matmul(sparse_matrix, b_k)
        norm = tf.norm(b_k1)
        b_k = b_k1 / norm

    return tf.squeeze(b_k)


In [20]:
def independent_layer_eigenvector_centrality(self):
    # if tyupe of layers is not int, map them to int
    layers_to_int = {layer: i for i, layer in enumerate(self.get_layers())}
    int_to_layers = {i: layer for i, layer in enumerate(self.get_layers())}
    layers = [layers_to_int[layer] for layer in self.get_layers()]
    layers = tf.convert_to_tensor(layers, dtype=tf.int32)
    # layers = tf.convert_to_tensor(list(self.get_layers()), dtype=tf.int32)

    def map_fn_body(li):
        # li is a scalar tensor (layer index)
        layer_matrix = __get_sparse_tensor_between_layers__(self, tf.expand_dims(li, 0), tf.expand_dims(li, 0))
        layer_matrix = tf.sparse.reorder(layer_matrix)
        eigvec = sparse_power_iteration(layer_matrix)  # returns [n,] or [n,1]
        return eigvec

    eigenvectors = tf.map_fn(map_fn_body, layers, fn_output_signature=tf.TensorSpec([None], tf.float32))
    return eigenvectors, int_to_layers


In [21]:
def uniform_eigenvector_centrality(self):
    """
    Compute the eigenvector centrality for each layer in a uniform manner.
    
    This function computes the eigenvector centrality for the aggregated adjacency matrix of all layers in the network.
    It sums the adjacency matrices of all layers and then applies the power iteration method to compute the eigenvector centrality.

    Returns
    -------
    tf.Tensor
        A tensor containing the eigenvector centrality for each layer.
    """
    layers = self.get_layers()

    sum_tensor = None

    for li in layers:
        # Get the sparse tensor for the current layer
        layer_matrix = __get_sparse_tensor_between_layers__(self, [li], [li])
        layer_matrix = tf.sparse.reorder(layer_matrix)  # ensure canonical order
        
        if sum_tensor is None:
            sum_tensor = layer_matrix
        else:
            sum_tensor = tf.sparse.add(sum_tensor, layer_matrix)


    # Compute the eigenvector centrality for the aggregated layer matrix
    eigvec = sparse_power_iteration(sum_tensor)
    return eigvec


In [22]:
def local_heterogeneus_eigenvector_centrality(self:MultilayerNetwork, weights = None ):
    #weights must be a [n_layers,n_layers] dense tensor, where n_layers is the number of layers in the network
    n_layers = len(self.get_layers())
    tensor, nodes_to_int, layers_to_int, int_to_nodes, int_to_layers = __get_sparse_tensor__(self, return_mappings=True)
    
    if (weights is not None) and (weights.shape[0] != n_layers or weights.shape[1] != n_layers):
        raise ValueError(f"Weights must be a square tensor of shape [{n_layers}, {n_layers}]. Got {weights.shape}.")
    
    if weights is None:
        weights = tf.constant(np.identity(n_layers), dtype=tf.float32)

    A_star = __sparse_masked_tensordot(tensor, weights)
    
    A_star = tf.sparse.reorder(A_star)  # Ensure the sparse tensor is in canonical order
    
    layers_tensor = tf.convert_to_tensor(list(range(n_layers)), dtype=tf.int32)
    # layers = tf.convert_to_tensor(list(self.get_layers()), dtype=tf.int32)

    def compute_eigvec(li):
        layer_matrix = __slice_sparse_tensor__(A_star, [li,li])
        layer_matrix = tf.sparse.reorder(layer_matrix)
        eigvec = sparse_power_iteration(layer_matrix)
        return eigvec
    
    eigenvectors = tf.map_fn(compute_eigvec, layers_tensor, fn_output_signature=tf.TensorSpec([None], tf.float32))
    return eigenvectors, nodes_to_int, layers_to_int, int_to_nodes, int_to_layers




def __sparse_masked_tensordot(A_sparse: tf.SparseTensor, W: tf.Tensor) -> tf.SparseTensor:
    """
    Args:
        A_sparse: tf.SparseTensor of shape [n, n, d, d]
        W: Tensor of shape [d, d], weights for combining diagonals
    
    Returns:
        A_sparse_star: tf.SparseTensor of shape [n, n, d, d], where only entries at [:,:,k,k]
                       are non-zero, and:
            A_star[i,j,k,k] = sum_{t=k}^{d-1} W[k, t] * A[i,j,t,t]
    """
    A_sparse = tf.sparse.reorder(A_sparse)
    indices = A_sparse.indices     # [nnz, 4]
    values = A_sparse.values       # [nnz]
    dense_shape = A_sparse.dense_shape
    n, _, d1, d2 = dense_shape.numpy()
    assert d1 == d2, "Expected square last dimensions for diagonals."

    # Extract only diagonal entries: where indices[:,2] == indices[:,3]
    is_diag = tf.equal(indices[:, 2], indices[:, 3])
    diag_indices = tf.boolean_mask(indices, is_diag)
    diag_values = tf.boolean_mask(values, is_diag)
    diag_t = diag_indices[:, 2]  # the shared diagonal index t

    output_entries = {}  # (i, j, k) -> value

    for k in range(d1):
        # Select entries with t >= k
        mask_k = tf.greater_equal(diag_t, k)
        selected_indices = tf.boolean_mask(diag_indices, mask_k)
        selected_values = tf.boolean_mask(diag_values, mask_k)
        selected_t = tf.boolean_mask(diag_t, mask_k)

        # Apply weights: W[k, t]
        weights = tf.gather(W[k], selected_t)
        weighted_vals = selected_values * weights

        for idx, val in zip(selected_indices.numpy(), weighted_vals.numpy()):
            i, j, t, _ = idx
            key = (i, j, k, k)
            output_entries[key] = output_entries.get(key, 0.0) + val

    final_indices = tf.constant(list(output_entries.keys()), dtype=tf.int64)
    final_values = tf.constant(list(output_entries.values()), dtype=tf.float32)

    return tf.SparseTensor(indices=final_indices, values=final_values, dense_shape=dense_shape)





In [23]:
def khatri_rao_weighted_block_sparse(W: tf.Tensor, A_sparse: tf.SparseTensor) -> tf.SparseTensor:
    """
    Compute the Khatri-Rao block product of a sparse tensor:
        A^⊗[i,j] = W[i,j] * A_j, where A_j = A_sparse[:,:,j,j]

    Args:
        W: Tensor of shape [m, m]
        A_sparse: SparseTensor of shape [n, n, m, m] (assumes A_j = A[:,:,j,j])

    Returns:
        A dense tensor of shape [n*m, n*m]
    """
    n = tf.cast(A_sparse.dense_shape[0], tf.int32)
    m = tf.shape(W)[0]
    
    indices = A_sparse.indices
    values = A_sparse.values

    # Only keep diagonal slices: i == j in the last two dims
    diag_mask = tf.equal(indices[:, 2], indices[:, 3])
    diag_indices = tf.boolean_mask(indices, diag_mask)
    diag_values = tf.boolean_mask(values, diag_mask)

    output_indices = []
    output_values = []

    for i in range(m):
        for j in range(m):
            # Extract entries of A_j = A[:, :, j, j]
            mask_j = tf.equal(diag_indices[:, 2], j)    
            indices_j = tf.boolean_mask(diag_indices, mask_j)[:, :2]  # [row, col]
            values_j = tf.boolean_mask(diag_values, mask_j)

            # Compute block offset
            row_offset = i * n
            col_offset = j * n
            
            # Shift indices into block [i,j]
            shifted_indices = indices_j + tf.cast(tf.stack([row_offset, col_offset]), tf.int64)

            # Scale values by W[i, j]
            scaled_values = tf.cast(W[i, j], tf.float32) * values_j

            output_indices.append(shifted_indices)
            output_values.append(scaled_values)

    all_indices = tf.concat(output_indices, axis=0)
    all_values = tf.concat(output_values, axis=0)
    dense_shape = tf.constant(np.array([n*m, n*m]), dtype=tf.int64)

    return tf.SparseTensor(indices=all_indices, values=all_values, dense_shape=dense_shape)


In [24]:
def global_heterogeneus_eigenvector_centrality(self:MultilayerNetwork, weights = None):
    """
    Compute the global eigenvector centrality for a multilayer network with heterogeneous weights.
    
    Parameters
    ----------
    self : MultilayerNetwork
        The multilayer network to compute the eigenvector centrality for.
    weights : tf.Tensor, optional
        A square tensor of shape [n_layers, n_layers] representing the weights for each layer pair.
        If None, an identity matrix is used.

    Returns
    -------
    tf.Tensor
        A tensor containing the global eigenvector centrality for each layer.
    """
    
    n_layers = len(self.get_layers())
    
    if (weights is not None) and (weights.shape[0] != n_layers or weights.shape[1] != n_layers):
        raise ValueError(f"Weights must be a square tensor of shape [{n_layers}, {n_layers}]. Got {weights.shape}.")
    
    if weights is None:
        weights = tf.identity(np.identity(n_layers))

    sparse_tensor, nodes_to_int, layers_to_int, int_to_nodes, int_to_layers = __get_sparse_tensor__(self, return_mappings=True)
    
    A_block = khatri_rao_weighted_block_sparse(weights, sparse_tensor)

    A_block = tf.sparse.reorder(A_block)  # Ensure the sparse tensor is in canonical order
    
    eigvec = sparse_power_iteration(A_block)
    #divide eigvec in n_layers parts of size n_nodes
    n_nodes = len(self.slices[0]) 
    eigvec = tf.reshape(eigvec, (n_layers,n_nodes))
    
    return eigvec, nodes_to_int, layers_to_int, int_to_nodes, int_to_layers

## 1.4. ---Deteccion Comunidades---


In [4]:
multinet_coms = net.MultiplexNetwork(
    couplings='categorical',
)
multinet_coms.add_layer(1)
multinet_coms.add_layer(2)
multinet_coms.add_layer(3)

multinet_coms.add_node(1)
multinet_coms.add_node(2)
multinet_coms.add_node(3)
multinet_coms.add_node(4)
multinet_coms.add_node(5)
multinet_coms.add_node(6)

multinet_coms[(1,1)][(3,1)] = 1
multinet_coms[(1,3)][(3,3)] = 1
multinet_coms[(1,1)][(4,1)] = 1
multinet_coms[(1,2)][(4,2)] = 1
multinet_coms[(1,3)][(4,3)] = 1
multinet_coms[(1,1)][(5,1)] = 1


multinet_coms[(2,2)][(3,2)] = 1
multinet_coms[(2,3)][(3,3)] = 1
multinet_coms[(2,1)][(4,1)] = 1
multinet_coms[(2,2)][(4,2)] = 1
multinet_coms[(2,3)][(4,3)] = 1

multinet_coms[(3,1)][(4,1)] = 1
multinet_coms[(3,2)][(5,2)] = 1
multinet_coms[(3,1)][(6,1)] = 1

multinet_coms[(5,1)][(6,1)] = 1
multinet_coms[(5,2)][(6,2)] = 1
multinet_coms[(5,3)][(6,3)] = 1



#### loudain

In [5]:
# Mucha et al. (2010)	for new modularity in multiplex networks
def modularity(communities):
    mod = 0
    graph_size = communities.get('graph_size', 0)
    for community in communities['com2nodes'].keys():
        comm_inner = communities['com_inner_weight'][community]
        comm_total = communities['com_total_weight'][community]
        mod += comm_inner/(2*graph_size) - (comm_total/(2*graph_size))**2
    return mod

#### TODO Dendrogram

# def move_nodes_step(self, subcoms):
#     sub_coms = list(subcoms['com2nodes'].keys())
#     new_coms = {
#                     'com2nodes': {k: v[:] for k, v in subcoms['com2nodes'].items()},
#                     'node2com': subcoms['node2com'].copy(),
#                     'com_inner_weight': subcoms['com_inner_weight'].copy(),
#                     'com_total_weight': subcoms['com_total_weight'].copy(),
#                     'neigh_coms': {k: set(v) for k, v in subcoms['neigh_coms'].items()},
#                     'graph_size': subcoms['graph_size'],
#                     'objective_function_value': subcoms['objective_function_value'],
#                 }
#     improvement = True
#     while improvement:
#         improvement = False
#         # the aggregated nodes of any given point are the list of subcoms at the start of the function
#         random.shuffle(sub_coms)

#         for subcom in sub_coms:
#             node_in_subcom = subcoms['com2nodes'][subcom][0]
#             current_com = new_coms['node2com'][node_in_subcom]

#             neigbor_coms = new_coms['neigh_coms'][current_com]

#             best_increase = -0
#             best_coms = new_coms

#             for target_com in neigbor_coms:
#                 # print(' checking target_com', target_com, 'for subcom', subcom, 'in current_com', current_com)
#                 temp_coms ={
#                     'com2nodes': {k: v[:] for k, v in new_coms['com2nodes'].items()},
#                     'node2com': new_coms['node2com'].copy(),
#                     'com_inner_weight': new_coms['com_inner_weight'].copy(),
#                     'com_total_weight': new_coms['com_total_weight'].copy(),
#                     'neigh_coms': {k: set(v) for k, v in new_coms['neigh_coms'].items()},
#                     'graph_size': new_coms['graph_size'],
#                     'objective_function_value': new_coms['objective_function_value'],
#                 }

#                 temp_coms = move_subcom(self, subcoms, temp_coms, subcom, current_com, target_com)
#                 gain = temp_coms['objective_function_value'] -  new_coms['objective_function_value']

#                 if gain > best_increase:
#                     best_increase = gain
#                     best_coms = temp_coms
#             if best_increase > 0:
#                 improvement = True
#                 new_coms = best_coms
#             # elif fully_merge:
#             #     # always merge, even if there is no gain, just minimizing the number of communities
#             #     improvement = True
#             #     new_coms = best_coms
                

#     return new_coms
            

def move_nodes_step(self, subcoms, force_merge=False):
    """
    Perform a step of the Louvain algorithm by moving nodes between communities.

    Parameters
    ----------
    self : MultilayerNetwork
        The multilayer network to analyze.
    subcoms : dict
        Current communities structure.
    force_merge : bool, optional
        If True, forces the merging of two communities with the smallest modularity loss
        when no improvement is found. Default is False.

    Returns
    -------
    dict
        Updated communities structure.
    """
    sub_coms = list(subcoms['com2nodes'].keys())
    new_coms = {
        'com2nodes': {k: v[:] for k, v in subcoms['com2nodes'].items()},
        'node2com': subcoms['node2com'].copy(),
        'com_inner_weight': subcoms['com_inner_weight'].copy(),
        'com_total_weight': subcoms['com_total_weight'].copy(),
        'neigh_coms': {k: set(v) for k, v in subcoms['neigh_coms'].items()},
        'graph_size': subcoms['graph_size'],
        'objective_function_value': subcoms['objective_function_value'],
    }
    improvement = True
    already_forced_merge = False 
    iteration = 0

    while improvement and not (already_forced_merge and force_merge):
        improvement = False
        iteration += 1
        random.shuffle(sub_coms)

        best_merge_coms = None
        best_merge_gain = float('-inf')

        for subcom in sub_coms:
            node_in_subcom = subcoms['com2nodes'][subcom][0]
            current_com = new_coms['node2com'][node_in_subcom]
            neighbor_coms = new_coms['neigh_coms'][current_com]

            best_increase = 0
            best_coms = new_coms

            for target_com in neighbor_coms:
                temp_coms = {
                    'com2nodes': {k: v[:] for k, v in new_coms['com2nodes'].items()},
                    'node2com': new_coms['node2com'].copy(),
                    'com_inner_weight': new_coms['com_inner_weight'].copy(),
                    'com_total_weight': new_coms['com_total_weight'].copy(),
                    'neigh_coms': {k: set(v) for k, v in new_coms['neigh_coms'].items()},
                    'graph_size': new_coms['graph_size'],
                    'objective_function_value': new_coms['objective_function_value'],
                }

                temp_coms = move_subcom(self, subcoms, temp_coms, subcom, current_com, target_com)
                gain = temp_coms['objective_function_value'] - new_coms['objective_function_value']

                if gain > best_increase:
                    best_increase = gain
                    best_coms = temp_coms

                # Track the best merge with the smallest gain
                if gain > best_merge_gain:
                    best_merge_gain = gain
                    best_merge_coms = temp_coms

            if best_increase > 0:
                improvement = True
                new_coms = best_coms

        # Force merge if no improvement and force_merge is enabled
        if not improvement and force_merge and best_merge_coms:
            already_forced_merge = True
            new_coms = best_merge_coms

    return new_coms

def move_subcom(self, subcoms, coms, subcom, current_com, target_com):
    """
    Move a subcommunity to a target community and update the communities structure.
    
    Parameters
    ----------
    self : MultilayerNetwork
        The multilayer network to analyze.
    subcoms : list
        List of subcommunities.
    coms : dict
        Current communities structure.
    subcom : str
        The subcommunity to move.
    target_com : str
        The target community to move the subcommunity to.
    
    Returns
    -------
    dict
        Updated communities structure after moving the subcommunity.
    """

    prev_modularity_target = coms['com_inner_weight'][target_com] / (2*coms['graph_size']) - (coms['com_total_weight'][target_com] / (2*coms['graph_size']))**2
    prev_modularity_current = coms['com_inner_weight'][current_com] / (2*coms['graph_size']) - (coms['com_total_weight'][current_com] / (2*coms['graph_size']))**2

    new_coms = coms.copy()
    #move nodes of the subcommunity to the target community
    new_coms['com2nodes'][target_com].extend(subcoms['com2nodes'][subcom])
    for node in subcoms['com2nodes'][subcom]:
        new_coms['com2nodes'][current_com].remove(node)


    # update node2com mapping
    for node in subcoms['com2nodes'][subcom]:
        new_coms['node2com'][node] = target_com

    # update total weights
    new_coms['com_total_weight'][target_com] = 0
    new_coms['com_total_weight'][current_com] = 0

    new_coms['com_inner_weight'][target_com] = 0
    new_coms['com_inner_weight'][current_com] = 0

    new_coms['neigh_coms'][target_com] = set()
    new_coms['neigh_coms'][current_com] = set()

    for node in new_coms['com2nodes'][target_com]:
        # update the neighbors of the target community
        for neighbor in self._iter_neighbors(node):
            new_coms['com_total_weight'][target_com] += self[node][neighbor]
            if new_coms['node2com'][neighbor] != target_com:
                new_coms['neigh_coms'][target_com].add(new_coms['node2com'][neighbor])
            else:
                new_coms['com_inner_weight'][target_com] += self[node][neighbor]
    for node in new_coms['com2nodes'][current_com]:
        # update the neighbors of the current community
        for neighbor in self._iter_neighbors(node):
            new_coms['com_total_weight'][current_com] += self[node][neighbor]
            if new_coms['node2com'][neighbor] != current_com:
                new_coms['neigh_coms'][current_com].add(new_coms['node2com'][neighbor])
            else:
                new_coms['com_inner_weight'][current_com] += self[node][neighbor]

    
    new_modularity_target = new_coms['com_inner_weight'][target_com] / (2*new_coms['graph_size']) - (new_coms['com_total_weight'][target_com] / (2*new_coms['graph_size']))**2
    new_modularity_current = new_coms['com_inner_weight'][current_com] / (2*new_coms['graph_size']) - (new_coms['com_total_weight'][current_com] / (2*new_coms['graph_size']))**2


    if len(new_coms['com2nodes'][current_com]) == 0:
        # remove the current community if it has no nodes left
        new_coms['com2nodes'].pop(current_com)
        new_coms['com_inner_weight'].pop(current_com)
        new_coms['com_total_weight'].pop(current_com)
        for k in new_coms['neigh_coms'].keys():
            if current_com in new_coms['neigh_coms'][k] and k != current_com:
                new_coms['neigh_coms'][k].remove(current_com)
                new_coms['neigh_coms'][k].add(target_com)
        new_coms['neigh_coms'].pop(current_com)
                                                                  
    # update modularity
    new_coms['objective_function_value'] =  new_coms['objective_function_value'] - prev_modularity_current - prev_modularity_target + new_modularity_target + new_modularity_current


    return new_coms


def louvain_algorithm(net, max_iter=1000, force_merge=False):
    """
    Apply the Louvain algorithm to find communities in a multiplex network.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    max_iter : int, optional
        The maximum number of iterations to run the algorithm (default is 1000).
    force_merge : bool, optional
        If True, forces the merging of communities with the smallest modularity loss when no improvement is found.
    
    Returns
    -------
    dict
        A dictionary containing the communities and their properties.
    """
    
    communities = init_multiplex_communities_louvain(net)
    states = [communities]
    for _ in range(max_iter):
        print(f"Iteration {_+1}/{max_iter}")
        new_communities = move_nodes_step(net, communities, force_merge=force_merge)
        if len(new_communities['com2nodes']) == len(communities['com2nodes']):
            # if there is no improvement, we stop
            print("No movements found, stopping.")
            states.append(new_communities)
            break
        if len(new_communities['com2nodes']) == 1:
            # if there is only one community, we stop
            states.append(new_communities)
            break

        states.append(new_communities)
        communities = new_communities
    
    return states     

def init_multiplex_communities_louvain(net):
    # all representation of a node is within the same community
    coms = list(net.slices[0])

    communities = {
        'com2nodes' : {com : [nl for nl in net.iter_node_layers() if nl[0] == com] for com in coms},
        'node2com' : {},
        'com_inner_weight' : {com : 0 for com in coms},
        'com_total_weight' : {com : 0 for com in coms},
        'neigh_coms' : {com : set() for com in coms},
        'graph_size' : len(net.edges),
        'objective_function_value' : 0,
    }

    for com, nodes in communities['com2nodes'].items():
        inner_w = 0
        total_w = 0
        for node1 in nodes:
            for node2 in nodes:
                inner_w += net[node1][node2]
            for neighbor in net._iter_neighbors(node1):
                total_w += net[node1][neighbor]
        communities['com_inner_weight'][com] = inner_w
        communities['com_total_weight'][com] = total_w
        communities['neigh_coms'][com].update([x[0] for node in nodes for x in net._iter_neighbors(node) if x[0] != node[0] ])
    
    
    communities['node2com'] = {node: com for com, nodes in communities['com2nodes'].items() for node in nodes}
    communities['objective_function_value'] = modularity(communities)
    return communities    

In [212]:
##### new net generation functions #####

def communities_are_neighbors(net, com1, com2, communities):
    """
    Check if two communities are neighbors in the network.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    com1 : str
        The first community.
    com2 : str
        The second community.
    communities : dict
        A dictionary containing the communities of the network.

    Returns
    -------
    bool
        True if the communities are neighbors, False otherwise.
    """
    
    for node1 in communities['com2nodes'][com1]:
        for node2 in net._iter_neighbors(node1):
            if node2 in communities['com2nodes'][com2]:
                return True
    return False

def get_highest_degree_node(net, communities, com):
    """
    Get the node with the highest degree in a given community.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    communities : dict
        A dictionary containing the communities of the network.
    com : str
        The community to analyze.

    Returns
    -------
    tuple
        The node with the highest degree in the community.
    """

    nodes_no_layers_degree = { n_l[0] : 0 for n_l in communities['com2nodes'][com]}

    for node1 in communities['com2nodes'][com]:
        nodes_no_layers_degree[node1[0]] += net._get_degree(node1)
    # get the node with the highest degree
    highest_degree_node = max(nodes_no_layers_degree, key=nodes_no_layers_degree.get)
    return highest_degree_node
        
    
def generate_louvain_communities_net(net,communities):
    """
    Generate a new network from the communities of a multilayer network.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    communities : dict
        A dictionary containing the communities of the network.

    Returns
    -------
    MultilayerNetwork
        A new multilayer network with the communities as nodes and edges between them.
    """
    
    com_net = MultilayerNetwork(aspects=1, directed=False)
    com_net.add_layer(1)  # Add a single layer for the communities
    
    com_2_name = {com:"" for com in communities['com2nodes'].keys()}
    for com, nodes in communities['com2nodes'].items():
        com_2_name[com] = get_highest_degree_node(net, communities, com)


    for com in communities['com2nodes'].keys():
        com_net.add_node(com_2_name[com])

    neighbors = defaultdict(set)
    for com in communities['com2nodes'].keys():
        for com2 in communities['com2nodes'].keys():
            if com != com2 and communities_are_neighbors(net, com, com2, communities):
                com_name = com_2_name[com]
                com2_name = com_2_name[com2]
                neighbors[com_name].add(com2_name)
            if com2 != com and communities_are_neighbors(net, com2, com, communities):
                com_name = com_2_name[com]
                com2_name = com_2_name[com2]
                neighbors[com2_name].add(com_name)

    for com, neighbors_set in neighbors.items():
        for neighbor in neighbors_set:
            com_net[(com,1)][(neighbor,1)] = 1  # or any weight you want, here we use 1

    com_sizes = {com_2_name[com]: len(nodes) for com, nodes in communities['com2nodes'].items()}

    

    return com_net, com_sizes, com_2_name


#### Leiden

In [51]:
import random
import copy
import queue
import math
from pymnet import MultilayerNetwork
from collections import defaultdict
from copy import deepcopy


def CPM(communities):
    """
    Compute the Constant Potts Model (CPM) for the given communities.
    
    Parameters
    ----------
    communities : dict
        A dictionary containing the communities of the network.

    Returns
    -------
    float
        The CPM value for the communities.
    """
    
    cpm = 0
    gamma = communities.get('gamma', 1)
    
    for com in communities['com2nodes'].keys():
        inner_weight = communities['com_inner_weight'][com]
        size = len(communities['com2nodes'][com])
        
        cpm += inner_weight - gamma * (size* (size - 1) / 2)
    
    return cpm


def leiden_algorithm(net, gamma=1.0, max_iterations=100):
    """
    Perform the Leiden algorithm for community detection in a multilayer network.

    This algorithm identifies communities through iterative improvement based on the Constant Potts Model (CPM).

    Args:
        net (MultilayerNetwork): The multilayer network to analyze.
        gamma (float, optional): Resolution parameter. Higher values lead to smaller communities.
        max_iterations (int, optional): Maximum number of iterations to run. Default is 100.

    Returns:
        list of dict: List of community states for each iteration until convergence.
    """

    
    communities = init_multiplex_communities_leiden(net,gamma)
    states = [deepcopy(communities)]
    for _ in range(max_iterations):
        print(f"Iteration {_+1}/{max_iterations}")
        old_communities, communities, merge_history = leiden_FastNodeMove(net, deepcopy(communities))

        communities = refine_partition(net, deepcopy(communities),deepcopy(old_communities), merge_history)

        # check if the com2nodes of old_communities and communities are the same
        if old_communities['com2nodes'] == communities['com2nodes']:
            print("Convergence reached.")
            break
        else:
            states.append(communities)


    return states


def init_multiplex_communities_leiden(net,gamma):
    """
    Initialize each node-layer pair in its own community for the Leiden algorithm.

    Args:
        net (MultilayerNetwork): The network to initialize communities on.

    Returns:
        dict: A dictionary with initial community structures including:
            - com2nodes: Community to node-layer mapping.
            - node2com: Node-layer to community mapping.
            - com_inner_weight: Intra-community weights.
            - com_total_weight: Total weights per community.
            - neigh_coms: Neighboring community relationships.
            - graph_size: Total number of edges.
    """

    # all representation of a node is within the same community
    coms = list(net.slices[0])

    communities = {
        'com2nodes' : {com : [nl for nl in net.iter_node_layers() if nl[0] == com] for com in coms},
        'node2com' : {},
        'com_inner_weight' : {com : 0 for com in coms},
        'com_total_weight' : {com : 0 for com in coms},
        'graph_size' : len(net.edges),
        'neigh_coms' : {com : set() for com in coms},
        'gamma': gamma,
        'objective_function_value': 0.0

    }


    for com, nodes in communities['com2nodes'].items():
        inner_w = 0
        total_w = 0
        for node1 in nodes:
            for node2 in nodes:
                inner_w += net[node1][node2]
            for neighbor in net._iter_neighbors(node1):
                total_w += net[node1][neighbor]
            communities['neigh_coms'][com].update([x[0] for x in net._iter_neighbors(node1) if x[0] != node1[0]])

        communities['com_inner_weight'][com] = inner_w
        communities['com_total_weight'][com] = total_w
    
    communities['node2com'] = {node: com for com, nodes in communities['com2nodes'].items() for node in nodes}
    communities['objective_function_value'] = CPM(communities)

    return communities

def leiden_FastNodeMove(net, communities):
    """
    Perform the node movement and merging phase of the Leiden algorithm.

    Args:
        net (MultilayerNetwork): The multilayer network.
        communities (dict): Current community structure.
        gamma (float): Resolution parameter.

    Returns:
        tuple: (old_communities, new_communities, merge_history)
            - old_communities: Previous state before moves.
            - new_communities: Updated community structure.
            - merge_history: History of merges performed.
    """
    q = queue.Queue()
    
    coms = list(communities['com2nodes'].keys())
    old_communities = copy.deepcopy(communities)

    random.shuffle(coms)
    for com in coms:
        q.put(com)
    # print(q.qsize())
    elems_in_q = coms

    merge_history = {com: [com] for com in coms}
    
    while not q.empty():
        com1 = q.get()
        elems_in_q.remove(com1)
        # print("step: ", com1)
        if com1 not in communities['com2nodes']:            
            continue
        max_cpm = 0
        max_com = com1
        for com2 in communities['neigh_coms'][com1]:
            weights_inter_coms = weights_inter_com1_com2(net,communities,com1,com2)
            new_cpm=compute_inc_CPM(communities,com1,com2,weights_inter_coms)
            # print("  com1: ",com1," com2: ",com2," new_cpm: ",new_cpm)
            if new_cpm > max_cpm:
                max_com = com2
                max_cpm = new_cpm
        if max_cpm > 0: 
            # print("  merging with",max_com)
            # print("  max_cpm ",max_cpm)
            weights_inter_coms = weights_inter_com1_com2(net,communities,com1,max_com)
            old_neigh_com1 = communities['neigh_coms'][com1]
            communities = merge_communities(net,communities,com1,max_com,weights_inter_coms)
            
            #append the merge history of max_com to com1
            merge_history[com1] = merge_history[com1] + merge_history[max_com]
            merge_history.pop(max_com, None)  # remove max_com from history

            

            for com in old_neigh_com1:
                if com not in elems_in_q:
                    q.put(com)   
                    elems_in_q.append(com)

    return old_communities, communities, merge_history

def refine_partition(net, parent_coms, subcoms, merge_history):
    """
    Refine community partitions by evaluating interconnections among subcommunities.

    Args:
        net (MultilayerNetwork): The multilayer network.
        parent_coms (dict): Original community structure before refinement.
        subcoms (dict): Community structure after merges.
        merge_history (dict): Tracks how subcommunities were merged.
        gamma (float): Resolution parameter.

    Returns:
        dict: Refined community structure.
    """
    for com_merged, subcoms_merged in merge_history.items():
        # print("Refining partition for com_merged:", com_merged, "with subcoms_merged:", subcoms_merged)
        R = check_interconected_subcoms(net, parent_coms, subcoms, com_merged, subcoms_merged)
        singleton = R.copy()
        # print("R: ", R)
        if len(R) < 2:
            # print("Skipping refinement for com_merged:", com_merged, "as R has less than 2 elements.")
            continue
        for subcom1 in R:
            # check if subcom1 is a singleton using singleton list
            if subcom1 not in singleton:
                continue


            T = check_interconected_subcoms(net, parent_coms, subcoms, com_merged, subcoms_merged)
            # print("     T: ", T)
            if not T:
                continue
            

            # Compute ΔH for each target subcommunity
            inc_CPMs = []
            for subcom2 in T:
                if subcom1 == subcom2:
                    continue
                weights_com1_com2 = weights_inter_com1_com2(net, subcoms, subcom1, subcom2)
                inc_CPM = compute_inc_CPM(subcoms, subcom1, subcom2,weights_com1_com2)
                inc_CPMs.append((subcom2, inc_CPM))

                        # Softmax-based random selection
            probs = []
            for (subcom2, inc_CPM) in inc_CPMs:
                # print("         subcom1:", subcom1, "subcom2:", subcom2, "inc_CPM:", inc_CPM)
                if inc_CPM > 0:
                    # print(math.exp(0.5 * inc_CPM))
                    probs.append(math.exp(0.5 * inc_CPM))
                else:
                    probs.append(0.0)

            if sum(probs) == 0:
                continue

            chosen = random.choices([x[0] for x in inc_CPMs], weights=probs, k=1)[0]

            if subcom1 in singleton:
                singleton.remove(subcom1)
            if chosen in singleton:
                singleton.remove(chosen)
            # print("         singleton: ", singleton)

            # print('         merging subcom1:', subcom1, 'with subcom2:', chosen, 'with inc_CPM:', inc_CPMs)
            subcoms = merge_communities(net, subcoms, chosen, subcom1, 
                                             weights_inter_com1_com2(net, subcoms, chosen, subcom1))

            subcoms_merged.remove(subcom1)


    return subcoms




def get_highest_degree_node(net, communities, com):
    """
    Identify the node (ignoring layers) with the highest degree in a community.

    Args:
        net (MultilayerNetwork): The multilayer network.
        communities (dict): Current community structure.
        com (str): Community ID.

    Returns:
        str: Node name with the highest aggregated degree.
    """


    nodes_no_layers_degree = { n_l[0] : 0 for n_l in communities['com2nodes'][com]}

    for node1 in communities['com2nodes'][com]:
        nodes_no_layers_degree[node1[0]] += net._get_degree(node1)
    # get the node with the highest degree
    highest_degree_node = max(nodes_no_layers_degree, key=nodes_no_layers_degree.get)
    return highest_degree_node
        
def weights_inter_com1_com2(net, coms, com1, com2):
    """
    Compute total weight of edges between two communities.

    Args:
        net (MultilayerNetwork): The network.
        coms (dict): Community dictionary with com2nodes and node2com.
        com1 (str): ID of the first community.
        com2 (str): ID of the second community.

    Returns:
        float: Sum of weights between `com1` and `com2`.
    """
    weight_inter_coms = 0
    for node1 in coms['com2nodes'][com1]:
        for neighbor in net._iter_neighbors(node1):
            if coms['node2com'][neighbor] == com2:
                weight_inter_coms += net[node1][neighbor]
    return weight_inter_coms

def weights_inter_subcom(net, subcoms, subcoms_merged):
    """
    Compute pairwise edge weights between all subcommunities in a set.

    Args:
        net (MultilayerNetwork): The network.
        subcoms (dict): Community dictionary.
        subcoms_merged (list): List of subcommunity IDs.

    Returns:
        dict: Mapping from subcommunity ID to total inter-subcommunity weight.
    """
    edge_weights = { subcom: 0 for subcom in subcoms_merged }
    for i in range(len(subcoms_merged)):
        subcom = subcoms_merged[i]
        for j in range(i + 1, len(subcoms_merged)):
            other_subcom = subcoms_merged[j]
            if subcom == other_subcom:
                continue
            weight = weights_inter_com1_com2(net, subcoms, subcom, other_subcom)
            edge_weights[subcom] += weight
            edge_weights[other_subcom] += weight
    return edge_weights
                        
def compute_inc_CPM(communities, com1, com2, weights_inter_com1_com2):
    """
    Compute the change in CPM objective function if two communities are merged.

    Args:
        communities (dict): Current community structure.
        com1 (str): First community ID.
        com2 (str): Second community ID.
        weights_inter_com1_com2 (float): Weight between the two communities.
        gamma (float): Resolution parameter.

    Returns:
        float: Change in CPM score after merging.
    """
    gamma = communities['gamma']
    com1_inner = communities['com_inner_weight'][com1]
    com1_size = len(communities['com2nodes'][com1])
    com2_inner = communities['com_inner_weight'][com2]
    com2_size = len(communities['com2nodes'][com2])
    dec = (com1_inner - gamma*(com1_size*(com1_size-1)/2)) + (com2_inner - gamma*(com2_size*(com2_size-1)/2))
    inc = (com1_inner + com2_inner + 2*weights_inter_com1_com2) - gamma*(com1_size + com2_size)*(com1_size + com2_size - 1)/2
    return inc - dec

def merge_communities(net, communities, com1, com2, weight_inter_coms):
    """
    Merge two communities into one, updating structure and metadata.

    Args:
        net (MultilayerNetwork): The network.
        communities (dict): Current community state.
        com1 (str): Community to merge into.
        com2 (str): Community to merge from.
        weight_inter_coms (float): Total weight between the two communities.

    Returns:
        dict: Updated community dictionary.
    """

    inc_CPM = compute_inc_CPM(communities, com1, com2, weight_inter_coms)

    communities['objective_function_value'] += inc_CPM

    communities = communities.copy()
    # merge both communities into the new one
    communities['com2nodes'][com1] = communities['com2nodes'][com1] + communities['com2nodes'][com2]
    # change all nodes to the new community
    for node in communities['com2nodes'][com2]:
        communities['node2com'][node] = com1
    

    # the new internal weight is the sum of both plus twice the sum between the communities
    communities['com_inner_weight'][com1] += communities['com_inner_weight'][com2] + 2*weight_inter_coms

    # new neighboors
    communities['neigh_coms'][com1] = communities['neigh_coms'][com1].union(communities['neigh_coms'][com2])
    communities['neigh_coms'][com1].discard(com1)  # remove self-loop if exists
    communities['neigh_coms'][com1].discard(com2)  # remove self-loop if exists

    communities['com_total_weight'][com1] = communities['com_total_weight'][com1] + communities['com_total_weight'][com2] + weight_inter_coms

    #remove the old community
    communities['com2nodes'].pop(com2)
    communities['com_inner_weight'].pop(com2)
    communities['com_total_weight'].pop(com2)
    communities['neigh_coms'].pop(com2)

    # update the neigh_coms of the neighbors, change com2 for com1 in the neigh_coms for other communities
    for k,v in communities['neigh_coms'].items():
        if com2 in v:
            v.remove(com2)
            v.add(com1)
    # if com1 in communities['neigh_coms'][com1]:
    #     communities['neigh_coms'][com1].remove(com1)


    return communities
        
def check_interconected_subcoms(net, parent_coms, subcoms, parent_com, subcoms_in_parent):
    """
    Determine which subcommunities in a parent are densely connected enough to merge.

    Args:
        net (MultilayerNetwork): The network.
        parent_coms (dict): Pre-refinement community structure.
        subcoms (dict): Post-merge structure.
        parent_com (str): Parent community being refined.
        subcoms_in_parent (list): Subcommunities within the parent.
        gamma (float): Resolution parameter.

    Returns:
        list: Subcommunity IDs eligible for merging.
    """
    gamma = parent_coms['gamma']
    # print(parent_com, subcoms_in_parent)
    weights_inter_subcoms = weights_inter_subcom(net, subcoms, subcoms_in_parent)
    R = []
    for subcom1 in subcoms_in_parent:

        e_v_S = weights_inter_subcoms[subcom1]
        # norm_v = deg of all nodes in subcom1
        norm_v = len(subcoms['com2nodes'][subcom1])
        norm_S = len(parent_coms['com2nodes'][parent_com])
        # edges(subcom1, com - subcom1) >= gamma * ||subcom1|| * (||com|| - ||subcom1||) 
        if e_v_S >= gamma * norm_v * (norm_S - norm_v):
            R.append(subcom1)
            continue
    random.shuffle(R)
    return R


def generate_leiden_communities_net(net, communities):
    """
    Generate a new simplified network from detected communities.

    Each node in the output represents a community, and edges reflect inter-community links.

    Args:
        net (MultilayerNetwork): Original network.
        communities (dict): Detected communities from Leiden.

    Returns:
        tuple:
            - MultilayerNetwork: Community-level network.
            - dict: Sizes of each community.
            - dict: Mapping from community ID to representative node.
    """
    
    com_net = MultilayerNetwork(aspects=1, directed=False)


    
    com_2_name = {com:"" for com in communities['com2nodes'].keys()}
    for com, nodes in communities['com2nodes'].items():
        com_2_name[com] = get_highest_degree_node(net, communities, com)


    for com in communities['com2nodes'].keys():
        com_net.add_node(com_2_name[com])

    neighbors = defaultdict(set)
    for com in communities['com2nodes'].keys():
        for com2 in communities['neigh_coms'][com]:
            if com != com2:
                com_name = com_2_name[com]
                com2_name = com_2_name[com2]
                neighbors[com_name].add(com2_name)
    for com, neighbors_set in neighbors.items():
        for neighbor in neighbors_set:
            com_net[(com,1)][(neighbor,1)] = 1  # or any weight you want, here we use 1

    com_sizes = {com_2_name[com]: len(nodes) for com, nodes in communities['com2nodes'].items()}

    return com_net, com_sizes, com_2_name

In [11]:
def generate_leiden_communities_net(net, communities):
    """
    Generate a new network from the communities of a multiplex network using the Leiden algorithm.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    communities : dict
        A dictionary containing the communities of the network.

    Returns
    -------
    MultilayerNetwork
        A new multilayer network with the communities as nodes and edges between them.
    """
    
    com_net = MultilayerNetwork(aspects=1, directed=False)
    com_net.add_layer(1)  # Add a single layer for the communities


    
    com_2_name = {com:"" for com in communities['com2nodes'].keys()}
    for com, nodes in communities['com2nodes'].items():
        com_2_name[com] = get_highest_degree_node(net, communities, com)


    for com in communities['com2nodes'].keys():
        com_net.add_node(com_2_name[com])

    neighbors = defaultdict(set)
    for com in communities['com2nodes'].keys():
        for com2 in communities['neigh_coms'][com]:
            if com != com2:
                com_name = com_2_name[com]
                com2_name = com_2_name[com2]
                neighbors[com_name].add(com2_name)
    for com, neighbors_set in neighbors.items():
        for neighbor in neighbors_set:
            com_net[(com,1)][(neighbor,1)] = 1  # or any weight you want, here we use 1

    com_sizes = {com_2_name[com]: len(nodes) for com, nodes in communities['com2nodes'].items()}

    return com_net, com_sizes, com_2_name

In [12]:
coms_states = leiden_algorithm(multinet_coms, gamma=1.0, max_iterations=100)
for coms in coms_states:
    print("Objective function value:", coms['objective_function_value'])
    print("Communities:", coms['com2nodes'])

Iteration 1/100
Convergence reached.
Objective function value: 18.0
Communities: {1: [(1, 1), (1, 2), (1, 3)], 2: [(2, 1), (2, 2), (2, 3)], 3: [(3, 1), (3, 2), (3, 3)], 4: [(4, 1), (4, 2), (4, 3)], 5: [(5, 1), (5, 2), (5, 3)], 6: [(6, 1), (6, 2), (6, 3)]}


## 1.5 ---Difusion---

In [194]:
def update_state(nl1_state, nl2_state,gamma_nl1,gamma_nl2,beta_nl1_nl2,beta_nl2_nl1):
    #begin cases
    match (nl1_state,nl2_state):

        case (1,1):
            return (np.random.choice([1,2], p=[1-gamma_nl1,gamma_nl1]),
                    np.random.choice([1,2], p=[1-gamma_nl2,gamma_nl2])) 
        case (0,1):
            return (np.random.choice([0,1], p=[1-beta_nl2_nl1,beta_nl2_nl1]),
                    np.random.choice([1,2], p=[1-gamma_nl2,gamma_nl2]))
        case (1,0):
            return (np.random.choice([1,2], p=[1-gamma_nl1,gamma_nl1]),
                    np.random.choice([0,1], p=[1-beta_nl1_nl2,beta_nl1_nl2]))
        
        case (1,2):
            return (np.random.choice([1,2], p=[1-gamma_nl1,gamma_nl1]),
                    2)
        case (2,1):
            return (2,
                    np.random.choice([1,2], p=[1-gamma_nl2,gamma_nl2]))
        case default:
            return (nl1_state,nl2_state)

def SIR_net_diffusion(self, interlayers_beta , intra_gamma, iterations, initial_state=None):
    """Given a multilayer network and a dictionary with
        the labels of the nodes in the form of a tuple (node, layer),
        this function returns the vector of the labels of the nodes
        after the number of iterations specified, using the SIR model.
    
        Parameters
        ----------
        net : MultilayerNetwork
            The multilayer network to analyze
        intra_beta : float
            The beta parameter of the SIR model for the intralayer edges
        inter_beta : float
            The beta parameter of the SIR model for the interlayer edges
        gamma : float
            The gamma parameter of the SIR model for each layer
        iterations : int
            The number of iterations to perform the diffusion

        Returns
        -------
    """
    is_multiplex = type(self) is MultiplexNetwork
    print("is_multiplex: ", is_multiplex)

    if not initial_state:
        initial_state = {edge_nodes: np.random.choice([0,1],p=[0.9,0.1]) for edge in self.edges for edge_nodes in self._link_to_nodes(edge[:-1])}
    
    if is_multiplex:
        for node in self.slices[0]:
            node_states = [initial_state[(node, l)] for l in self.get_layers()]

            highest_state = max(node_states)
            for l in self.get_layers():
                initial_state[(node, l)] = highest_state
            


    

    
    state_list = [initial_state.copy()]
    state = initial_state

    for i in range(iterations):
        # print(f"Iteration {i+1}/{iterations}")
        # print("Current state: ", state)
        state_changed = False  # Track if any node state changes
        new_state = state.copy()
        for edge in self.edges:
            ((nl1_0,nl1_1, weight),nl2) = self._link_to_nodes(edge)
            nl1 = (nl1_0, nl1_1)
            nl1_state = state[nl1]
            nl2_state = state[nl2]

            gamma_nl1 = intra_gamma[tuple(nl1[1:])]
            gamma_nl2 = intra_gamma[tuple(nl2[1:])]
            beta_nl1_nl2 = interlayers_beta[(nl1[1:], nl2[1:])]
            beta_nl2_nl1 = interlayers_beta[(nl2[1:], nl1[1:])]


            # Scale beta by weight
            beta_nl1_nl2 = min(beta_nl1_nl2 * weight, 1.0)
            beta_nl2_nl1 = min(beta_nl2_nl1 * weight, 1.0)

            updated_nl1, updated_nl2  = update_state(
                nl1_state, nl2_state,
                gamma_nl1, gamma_nl2,
                beta_nl1_nl2, beta_nl2_nl1
            )


            if new_state[nl1] != updated_nl1 and updated_nl1!= nl1_state:
                new_state[nl1] = updated_nl1
                state_changed = True

            if new_state[nl2] != updated_nl2 and updated_nl2 != nl2_state:
                new_state[nl2] = updated_nl2
                state_changed = True
            #print sum of new_state.values()
            
        # If the network is multiplex, ensure all layers of a node have the same state
        if is_multiplex and state_changed:
            for node in self.slices[0]:
                node_states = [new_state[(node, l)] for l in self.get_layers()]
                # print("node_states: ", node_states, ' for node: ', node)
                highest_state = max(node_states)
                for l in self.get_layers():
                    new_state[(node, l)] = highest_state
        
        if all(s in (0, 2) for s in new_state.values()):
            print("Epidemic has ended, all nodes are either susceptible or recovered.")
            state =  new_state
            state_list.append(state.copy())
            break
        #if all states are 0 or 2, then the epidemic is over
        # if not state_changed:
        #     # print("No state changes, continue.")
        #     continue
        state =  new_state
        state_list.append(state.copy())

            
    return state_list
            


## Small test to get visuals


In [2]:
multinet_test = net.MultiplexNetwork(
    couplings='categorical',
)
multinet_test.add_layer('Flatmates')
multinet_test.add_layer('Colleagues')
multinet_test.add_layer('Gym Buddies')

multinet_test.add_node('Alberto')
multinet_test.add_node('Marta')
multinet_test.add_node('Pablo')
multinet_test.add_node('Paula')
multinet_test.add_node('Juan')
multinet_test.add_node('Ana')
multinet_test.add_node('Andres')
multinet_test.add_node('Marcos')
multinet_test.add_node('Tomas')

multinet_test[('Alberto','Flatmates')][('Marta','Flatmates')] = 1
multinet_test[('Alberto','Flatmates')][('Pablo','Flatmates')] = 1
multinet_test[('Marta','Flatmates')][('Pablo','Flatmates')] = 1

multinet_test[('Paula','Flatmates')][('Ana','Flatmates')] = 1
multinet_test[('Paula','Flatmates')][('Andres','Flatmates')] = 1
multinet_test[('Ana','Flatmates')][('Andres','Flatmates')] = 1

multinet_test[('Alberto','Colleagues')][('Pablo','Colleagues')] = 1
multinet_test[('Alberto','Colleagues')][('Ana','Colleagues')] = 1
multinet_test[('Alberto','Colleagues')][('Juan','Colleagues')] = 1
multinet_test[('Pablo','Colleagues')][('Ana','Colleagues')] = 1
multinet_test[('Pablo','Colleagues')][('Juan','Colleagues')] = 1
multinet_test[('Ana','Colleagues')][('Juan','Colleagues')] = 1
multinet_test[('Tomas','Colleagues')][('Marcos','Colleagues')] = 1

multinet_test[('Paula','Colleagues')][('Andres','Colleagues')] = 1

multinet_test[('Alberto','Gym Buddies')][('Juan','Gym Buddies')] = 1
multinet_test[('Alberto','Gym Buddies')][('Pablo','Gym Buddies')] = 1
# multinet_test[('Juan','Gym Buddies')][('Pablo','Gym Buddies')] = 1

multinet_test[('Paula','Gym Buddies')][('Ana','Gym Buddies')] = 1
multinet_test[('Paula','Gym Buddies')][('Andres','Gym Buddies')] = 1
multinet_test[('Paula','Gym Buddies')][('Marta','Gym Buddies')] = 1
multinet_test[('Ana','Gym Buddies')][('Andres','Gym Buddies')] = 1
multinet_test[('Marta','Gym Buddies')][('Andres','Gym Buddies')] = 1
multinet_test[('Marta','Gym Buddies')][('Ana','Gym Buddies')] = 1



node_coords = {
    'Alberto': (1.8, 0.5),
    'Marta': (1.1, 0.6),
    'Marcos': (0.6, 0.6),
    'Tomas': (0.5, 1.1),
    'Pablo': (1.4, 1.2),
    'Paula': (0,2),
    'Ana': (0.7, 1.6),
    'Juan': (1.6, 1.8),
    'Andres': (0, 1),
}


In [3]:
#base_draw 

fig = plt.figure(figsize=(34, 38))
ax = fig.add_subplot(111, projection='3d')

nodeColorDict = {
    (node, layer): 'gray'
    for (node, layer) in multinet_test.iter_node_layers()
}
#set coordinates for the nodes


# nodeLabelDict = {
#     (node, layer): node for (node, layer) in multinet_test.iter_node_layers()
#     if state[(node, layer)] != 0
# }

draw(
    multinet_test,
    nodeCoords=node_coords,
    nodeColorDict=nodeColorDict,
    # nodeLabelRule={},                  # No automatic labels
    # nodeLabelDict=nodeLabelDict,  # Only labels for infected nodes
        defaultNodeLabelSize=40,
        defaultLayerLabelSize=30,
    defaultLayerLabelLoc=(-0.01,0),
    defaultNodeLabel="",               # Hide labels
    defaultLayerAlpha=0.3,
    defaultEdgeAlpha=0.5,
    layergap=1.2,
    ax=ax
)

plt.savefig(f"imgs/tests/base_net.png", bbox_inches='tight')
plt.close(fig)



#### louvain

In [70]:
louv_states = louvain_algorithm(multinet_test, max_iter=1000, force_merge=True)

print(len(louv_states))

for i, state in enumerate(louv_states):
    print(f"    Iteration {i}: Objective function value: {state['objective_function_value']}")
    louv_com_net, louv_com_sizes, louv_com_2_name = generate_louvain_communities_net(multinet_test, louv_states[i])




    nodeSizeDict_louv = {
        (louv_com_2_name[com], 1) : louv_com_sizes[louv_com_2_name[com]] /100
        for com, nodes in state['com2nodes'].items()
    }


    # map each community to a color
    nodeColorDict_louv = {
        node: colors(i) if node in louv_com_sizes else 'black'
        for i, node in enumerate(louv_com_net.slices[0])
        if node in louv_com_sizes
    }

    

    nodeColorDict_louv = {
        (louv_com_2_name[com], 1) : nodeColorDict_louv[louv_com_2_name[com]]
        for com, nodes in state['com2nodes'].items()
    }

    custom_node_labels = {
        (louv_com_2_name[com], 1): ','.join(set(n[0] for n in nodes))  # Custom labels for communities
        for com, nodes in state['com2nodes'].items()
    }


    
    fig2 = plt.figure(figsize=(36, 24))
    ax2 = fig2.add_subplot(111, projection='3d')

    draw(
        louv_com_net,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict=nodeSizeDict_louv,  # Use global node sizes
        nodeColorDict=nodeColorDict_louv,
        nodeLabelRule={},                  # No automatic labels
        nodeLabelDict=custom_node_labels,  # Use custom labels for communities
        defaultNodeLabelSize=40,
        defaultLayerLabelSize=30,
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.3,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )

    fig2.savefig(f'imgs/tests/louvain_step_{i}.png', bbox_inches='tight')
    plt.close(fig2)
    break

Iteration 1/1000
Iteration 2/1000
Iteration 3/1000
Iteration 4/1000
No movements found, stopping.
5
    Iteration 0: Objective function value: 0.43481882548937945


In [68]:
best_state = louv_states[1]
colors = plt.get_cmap('tab20', len(best_state['com2nodes']))
# map each community to a color


nodeColorDict_louv_base = {
    (node,layer) :nodeColorDict_louv[louv_com_2_name[best_state['node2com'][(node,layer)]]]
    for node, layer in multinet_test.iter_node_layers()
    
}

fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')

draw(
        multinet_test,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict=nodeSizeDict_louv,  # Use global node sizes
        nodeColorDict=nodeColorDict_louv_base,
        nodeLabelRule={},                  # No automatic labels
        nodeLabelDict=custom_node_labels,  # Use custom labels for communities
        defaultNodeLabelSize=40,
        defaultLayerLabelSize=30,
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.2,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )
fig2.savefig(f'imgs/tests/base_louv.png', bbox_inches='tight')
plt.close(fig2)

In [67]:
louv_com_net, louv_com_sizes, louv_com_2_name = generate_louvain_communities_net(multinet_test, louv_states[1])
colors = plt.get_cmap('tab20', len(louv_com_net.slices[0]))

nodeColorDict_louv = {
    node: colors(i) if node in louv_com_sizes else 'black'
    for i, node in enumerate(louv_com_net.slices[0])
    if node in louv_com_sizes
}

nodeColorDict_louv


{'Alberto': (0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0),
 'Tomas': (0.8392156862745098, 0.15294117647058825, 0.1568627450980392, 1.0),
 'Ana': (0.9686274509803922, 0.7137254901960784, 0.8235294117647058, 1.0),
 'Paula': (0.6196078431372549, 0.8549019607843137, 0.8980392156862745, 1.0)}

In [30]:
louv_com_2_name

{'Tomas': 'Tomas', 'Marta': 'Ana', 'Andres': 'Andres', 'Pablo': 'Alberto'}

In [62]:
nodeColorDict_louv[louv_com_2_name[best_state['node2com'][('Ana', 'Colleagues')]]]

(0.9686274509803922, 0.7137254901960784, 0.8235294117647058, 1.0)

#### leiden

In [262]:
GAMMA = 0.27
leid_states = leiden_algorithm(multinet_test, gamma=GAMMA, max_iterations=100)
print(len(leid_states))

Iteration 1/100
Iteration 2/100
Convergence reached.
2


In [15]:
GAMMA = 0.01
leid_states = leiden_algorithm(multinet_test, gamma=GAMMA, max_iterations=100)

len(leid_states)

for i, state in enumerate(leid_states):
    print(f"    Iteration {i}: Objective function value: {state['objective_function_value']}")
    leid_com_net, leid_com_sizes, leid_com_2_name = generate_leiden_communities_net(multinet_test, leid_states[i])


    nodeSizeDict_leid = {
        (leid_com_2_name[com], 1) : leid_com_sizes[leid_com_2_name[com]] /100
        for com, nodes in state['com2nodes'].items()
    }


    colors = plt.get_cmap('tab20', len(leid_com_net.slices[0])+2)
    # map each community to a color
    nodeColorDict_leid = {
        node: colors(i) if node in leid_com_sizes else 'black'
        for i, node in enumerate(leid_com_net.slices[0])
        if node in leid_com_sizes
    }

    nodeColorDict_leid = {
        (leid_com_2_name[com], 1) : nodeColorDict_leid[leid_com_2_name[com]]
        for com, nodes in state['com2nodes'].items()
    }

    custom_node_labels = {
        (leid_com_2_name[com], 1): ','.join(set(n[0] for n in nodes))  # Custom labels for communities
        for com, nodes in state['com2nodes'].items()
    }


    fig2 = plt.figure(figsize=(36, 28))
    ax2 = fig2.add_subplot(111, projection='3d')
    # if i == 0:
    #     draw(
    #         leid_com_net,
    #         nodeCoords=node_coords,  # Use the same coordinates as the original network
    #         nodeSizeDict=nodeSizeDict_leid,  # Use global node sizes
    #         nodeColorDict=nodeColorDict_leid,
    #         nodeLabelRule={},                  # No automatic labels
    #         nodeLabelDict=custom_node_labels,  # Use custom labels for communities
    #         defaultNodeLabelSize=40,
    #         defaultNodeLabel="",               # Hide labels
    #         defaultLayerAlpha=0.3,
    #         defaultEdgeAlpha=0.5,
    #         ax=ax2
    #     )
    #     fig2.savefig(f'imgs/tests/leiden_step_{i}_gamma_{GAMMA}-min.png', bbox_inches='tight')
    #     plt.close(fig2)
    if i ==2:
        draw(
            leid_com_net,
            nodeSizeDict=nodeSizeDict_leid,  # Use global node sizes
            nodeColorDict=nodeColorDict_leid,
            nodeLabelRule={},                  # No automatic labels
            nodeLabelDict=custom_node_labels,  # Use custom labels for communities
            defaultNodeLabelSize=30,
            defaultNodeLabel="",               # Hide labels
            defaultLayerAlpha=0.3,
            defaultEdgeAlpha=0.5,
            ax=ax2
        )
        fig2.savefig(f'imgs/tests/leiden_step_{i}_gamma_{GAMMA}-min.png', bbox_inches='tight')
        plt.close(fig2)
    # else:   
    #     draw(
    #         leid_com_net,
    #                     nodeCoords=node_coords,  # Use the same coordinates as the original network
    #         nodeSizeDict=nodeSizeDict_leid,  # Use global node sizes
    #         nodeColorDict=nodeColorDict_leid,
    #         nodeLabelRule={},                  # No automatic labels
    #         nodeLabelDict=custom_node_labels,  # Use custom labels for communities
    #         defaultNodeLabelSize=30,
    #         defaultNodeLabel="",               # Hide labels
    #         defaultLayerAlpha=0.3,
    #         defaultEdgeAlpha=0.5,
    #         ax=ax2
    #     )

        fig2.savefig(f'imgs/tests/leiden_step_{i}_gamma_{GAMMA}-min.png', bbox_inches='tight')
        plt.close(fig2)

best_state = leid_states[-1]
colors = plt.get_cmap('tab20', len(best_state['com2nodes'])+2)
# map each community to a color


nodeColorDict_leid_base = {
    (node,layer) :nodeColorDict_leid[(leid_com_2_name[best_state['node2com'][(node,layer)]], 1)]
    for node, layer in multinet_test.iter_node_layers()
    
}

fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')

draw(
        multinet_test,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict=nodeSizeDict_leid,  # Use global node sizes
        nodeColorDict=nodeColorDict_leid_base,
        nodeLabelRule={},                  # No automatic labels
        # nodeLabelDict=custom_node_labels,  # Use custom labels for communities
        defaultNodeLabelSize=30,
        defaultLayerLabelSize=0,
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.2,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )
fig2.savefig(f'imgs/tests/base_leid_gamma_{GAMMA}-min.png', bbox_inches='tight')
plt.close(fig2)


Iteration 1/100
Iteration 2/100
Iteration 3/100
Convergence reached.
    Iteration 0: Objective function value: 53.73
    Iteration 1: Objective function value: 86.83000000000001
    Iteration 2: Objective function value: 95.75000000000001


In [110]:
best_state = leid_states[-1]
leid_com_net, leid_com_sizes, leid_com_2_name = generate_leiden_communities_net(multinet_test, best_state)

colors = plt.get_cmap('tab20', len(best_state['com2nodes'])+2)
# map each community to a color

nodeColorDict_leid_base = {
    (node,layer) :nodeColorDict_leid[(leid_com_2_name[best_state['node2com'][(node,layer)]], 1)]
    for node, layer in multinet_test.iter_node_layers()
    
}


fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')

draw(
        multinet_test,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict=nodeSizeDict_leid,  # Use global node sizes
        nodeColorDict=nodeColorDict_leid_base,
        nodeLabelRule={},                  # No automatic labels
        # nodeLabelDict=custom_node_labels,  # Use custom labels for communities
        defaultNodeLabelSize=30,
        defaultLayerLabelSize=0,
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.2,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )
fig2.savefig(f'imgs/tests/base_leid_gamma_{GAMMA}-min.png', bbox_inches='tight')
plt.close(fig2)


In [ ]:
for (node,layer) in 

{('Tomas', 1): (0.596078431372549,
  0.8745098039215686,
  0.5411764705882353,
  1.0),
 ('Ana', 1): (0.5490196078431373,
  0.33725490196078434,
  0.29411764705882354,
  1.0),
 ('Alberto', 1): (0.12156862745098039,
  0.4666666666666667,
  0.7058823529411765,
  1.0)}

In [109]:
for node, layer in multinet_test.iter_node_layers():
    print(f"Node: {node}, Layer: {layer}, Color: {nodeColorDict_leid[(leid_com_2_name[best_state['node2com'][(node,layer)]], 1)]}")
    

Node: Marcos, Layer: Colleagues, Color: (0.596078431372549, 0.8745098039215686, 0.5411764705882353, 1.0)
Node: Marcos, Layer: Flatmates, Color: (0.596078431372549, 0.8745098039215686, 0.5411764705882353, 1.0)
Node: Marcos, Layer: Gym Buddies, Color: (0.596078431372549, 0.8745098039215686, 0.5411764705882353, 1.0)
Node: Tomas, Layer: Colleagues, Color: (0.596078431372549, 0.8745098039215686, 0.5411764705882353, 1.0)
Node: Tomas, Layer: Flatmates, Color: (0.596078431372549, 0.8745098039215686, 0.5411764705882353, 1.0)
Node: Tomas, Layer: Gym Buddies, Color: (0.596078431372549, 0.8745098039215686, 0.5411764705882353, 1.0)
Node: Juan, Layer: Colleagues, Color: (0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0)
Node: Juan, Layer: Flatmates, Color: (0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0)
Node: Juan, Layer: Gym Buddies, Color: (0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0)
Node: Paula, Layer: Colleagues, Color: (0.5490196078431373

#### PCI

In [27]:
color_map = {
    0: 'lightgray',
    1: 'grey',
    2: 'blue',
    3: 'darkblue',
    4: 'black'
}

mlPCI_1_color_map = {
    nl: color_map[mlPCI_n(multinet_test, nl,1)] for nl in multinet_test.iter_node_layers()
}

mlPCI_2_color_map = {
    nl: color_map[mlPCI_n(multinet_test, nl,2)] for nl
    in multinet_test.iter_node_layers()
}

allPCI_color_map = {
    nl: color_map[allPCI(multinet_test, nl)] for nl in multinet_test.iter_node_layers()
}

lsPCI_color_map = {
    nl: color_map[lsPCI(multinet_test, nl)] for nl in multinet_test.iter_node_layers()
}


colors_draw = {
    'mlPCI_1': mlPCI_1_color_map,
    'mlPCI_2': mlPCI_2_color_map,
    'allPCI': allPCI_color_map,
    'lsPCI': lsPCI_color_map
}

for name, color_map in colors_draw.items():
    fig = plt.figure(figsize=(34, 38))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title(name, fontsize=30)

    nodeColorDict = {
        (node, layer): color_map[(node, layer)]
        for (node, layer) in multinet_test.iter_node_layers()
    }
    
    draw(
        multinet_test,
        nodeCoords=node_coords,
        nodeColorDict=nodeColorDict,
        # nodeLabelRule={},                  # No automatic labels
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.1,
        defaultLayerLabelLoc=(-0.01,0), 
        defaultEdgeAlpha=0.5,
        defaultNodeLabelSize=40,
        defaultLayerLabelSize=30,
        layergap=1.2,
        ax=ax
    )

    plt.savefig(f"imgs/tests/{name}-min.png", dpi=50, bbox_inches='tight')
    plt.close(fig)



#### Eigenvector

In [460]:
# create a tensorflow tensor with dimensions (3, 3) and values from 0 to 8
import tensorflow as tf
weights = tf.constant([[1,0.25, 0.25], [0.5, 1, 0.25], [0.25, 0.25, 1]], dtype=tf.float32)


# IL_eig , IL_n_2_layer = independent_layer_eigenvector_centrality(multinet_test) 
# U_eig = uniform_eigenvector_centrality(multinet_test)
LH_eig, LH_nodes_to_int, LH_layers_to_int, LH_int_to_nodes, LH_int_to_layers = local_heterogeneus_eigenvector_centrality(multinet_test, weights=weights)
GH_eig, GH_nodes_to_int, GH_layers_to_int, GH_int_to_nodes, GH_int_to_layers = global_heterogeneus_eigenvector_centrality(multinet_test, weights=weights)
LH_eig

<tf.Tensor: shape=(3, 9), dtype=float32, numpy=
array([[-0.5198905 , -0.1249039 ,  0.        , -0.13252707,  0.        ,
        -0.49685326, -0.46105844, -0.13252707, -0.46855956],
       [-0.22850078, -0.31715968,  0.        , -0.5148298 ,  0.        ,
        -0.22712246, -0.02152409, -0.5148298 , -0.5148298 ],
       [ 0.        , -0.5       ,  0.        , -0.5       ,  0.        ,
         0.        ,  0.        , -0.5       , -0.5       ]],
      dtype=float32)>

In [463]:
# create a tensorflow tensor with dimensions (3, 3) and values from 0 to 8
import tensorflow as tf
weights = tf.constant([[1,0.25, 10], [0.5, 1, 0.25], [0.25, 0.25, 1]], dtype=tf.float32)


# IL_eig , IL_n_2_layer = independent_layer_eigenvector_centrality(multinet_test) 
# U_eig = uniform_eigenvector_centrality(multinet_test)
LH_eig, LH_nodes_to_int, LH_layers_to_int, LH_int_to_nodes, LH_int_to_layers = local_heterogeneus_eigenvector_centrality(multinet_test, weights=weights)
GH_eig, GH_nodes_to_int, GH_layers_to_int, GH_int_to_nodes, GH_int_to_layers = global_heterogeneus_eigenvector_centrality(multinet_test, weights=weights)
LH_eig

<tf.Tensor: shape=(3, 9), dtype=float32, numpy=
array([[ 0.04550587,  0.4876871 ,  0.        ,  0.50564235,  0.        ,
         0.03758796,  0.03341632,  0.50564235,  0.4961982 ],
       [-0.22850078, -0.31715968,  0.        , -0.5148298 ,  0.        ,
        -0.22712246, -0.02152409, -0.5148298 , -0.5148298 ],
       [ 0.        ,  0.5       ,  0.        ,  0.5       ,  0.        ,
         0.        ,  0.        ,  0.5       ,  0.5       ]],
      dtype=float32)>

In [ ]:
LH_eig_np = LH_eig.numpy()
#transform to absolute values and normalize
LH_eig_np = np.abs(LH_eig_np)
min_val = LH_eig_np.min()
max_val = LH_eig_np.max()
LH_eig_grad = (LH_eig_np - min_val) / (max_val - min_val + 1e-8)  # Avoid divide-by-zero

# Use a matplotlib colormap
colormap = plt.get_cmap('viridis')  # or 'plasma', 'inferno', etc.

# Convert to RGB (exclude alpha channel)
LH_eig_colors_rgb = np.array([
    [colormap(LH_eig_grad[i, j])[:3] for j in range(LH_eig_grad.shape[1])]
    for i in range(LH_eig_grad.shape[0])
])

# Construct the dictionary {(node, layer): normalized_value}
color_map = {}
for layer_idx in range(LH_eig_grad.shape[0]):
    for node_idx in range(LH_eig_grad.shape[1]):
        layer_name = LH_int_to_layers[0][layer_idx]
        node_name = LH_int_to_nodes[node_idx]
        color_map[(node_name, layer_name)] = LH_eig_colors_rgb[layer_idx, node_idx]

fig = plt.figure(figsize=(34, 38))
ax = fig.add_subplot(111, projection='3d')
#add color gradient legend


draw(
    multinet_test,
    nodeCoords=node_coords,
    nodeColorDict=color_map,
    # nodeLabelRule={},                  # No automatic labels
    defaultNodeLabel="",               # Hide labels
    defaultLayerAlpha=0.1,
    defaultLayerLabelLoc=(-0.01,0), 
    defaultEdgeAlpha=0.5,
    defaultNodeLabelSize=40,
    defaultLayerLabelSize=30,
    layergap=1.2,
    ax=ax
)

# Create a mappable object for the colorbar
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

norm = Normalize(vmin=min_val, vmax=max_val)
sm = ScalarMappable(norm=norm, cmap=colormap)
sm.set_array([])

# Add colorbar to the figure (adjust location as needed)
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.1, aspect=30)
cbar.set_label("LH_eig Gradient", fontsize=30)
cbar.ax.tick_params(labelsize=24)                       # Tick label font size


plt.savefig(f"imgs/tests/LH_eig_hichColleages.png", bbox_inches='tight')
plt.close(fig)


In [ ]:
GH_eig_np = GH_eig.numpy()
#transform to absolute values and normalize
GH_eig_np = np.abs(GH_eig_np)
min_val = GH_eig_np.min()
max_val = GH_eig_np.max()
GH_eig_grad = (GH_eig_np - min_val) / (max_val - min_val + 1e-8)  # Avoid divide-by-zero

# Use a matplotlib colormap
colormap = plt.get_cmap('viridis')  # or 'plasma', 'inferno', etc.

# Convert to RGB (exclude alpha channel)
GH_eig_colors_rgb = np.array([
    [colormap(GH_eig_grad[i, j])[:3] for j in range(GH_eig_grad.shape[1])]
    for i in range(GH_eig_grad.shape[0])
])

# Construct the dictionary {(node, layer): normalized_value}
color_map = {}
for layer_idx in range(GH_eig_grad.shape[0]):
    for node_idx in range(GH_eig_grad.shape[1]):
        layer_name = GH_int_to_layers[0][layer_idx]
        node_name = GH_int_to_nodes[node_idx]
        color_map[(node_name, layer_name)] = GH_eig_colors_rgb[layer_idx, node_idx]

fig = plt.figure(figsize=(34, 38))
ax = fig.add_subplot(111, projection='3d')
#add color gradient legend


draw(
    multinet_test,
    nodeCoords=node_coords,
    nodeColorDict=color_map,
    # nodeLabelRule={},                  # No automatic labels
    defaultNodeLabel="",               # Hide labels
    defaultLayerAlpha=0.1,
    defaultLayerLabelLoc=(-0.01,0), 
    defaultEdgeAlpha=0.5,
    defaultNodeLabelSize=40,
    defaultLayerLabelSize=30,
    layergap=1.2,
    ax=ax
)

# Create a mappable object for the colorbar
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

norm = Normalize(vmin=min_val, vmax=max_val)
sm = ScalarMappable(norm=norm, cmap=colormap)
sm.set_array([])

# Add colorbar to the figure (adjust location as needed)
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.1, aspect=30)
cbar.set_label("GH_eig Gradient", fontsize=30)
cbar.ax.tick_params(labelsize=24)                       # Tick label font size


plt.savefig(f"imgs/tests/GH_eig_highColleages.png", bbox_inches='tight')
plt.close(fig)


#### SIR

In [60]:

interlayers_beta = {
    (('Flatmates',), ('Colleagues',)): 1,
    (('Colleagues',), ('Flatmates',)): 1,
    (('Flatmates',), ('Gym Buddies',)): 1,
    (('Gym Buddies',), ('Flatmates',)): 1,
    (('Colleagues',), ('Gym Buddies',)): 1,
    (('Gym Buddies',), ('Colleagues',)): 1,
    
    (('Flatmates',), ('Flatmates',)): 0.8,
    (('Colleagues',), ('Colleagues',)): 0.3,
    (('Gym Buddies',), ('Gym Buddies',)): 0.5,
}

intra_gamma = {
    ('Flatmates',): 0.07,
    ('Colleagues',): 0.07,
    ('Gym Buddies',): 0.07,
}
initial_state = {
    (node, layer): 0
    for (node, layer) in multinet_test.iter_node_layers()
}

initial_state[('Alberto', 'Flatmates')] = 1
initial_state[('Alberto', 'Colleagues')] = 1
initial_state[('Alberto', 'Gym Buddies')] = 1

initial_state[('Ana', 'Flatmates')] = 1
initial_state[('Ana', 'Colleagues')] = 1
initial_state[('Ana', 'Gym Buddies')] = 1


dif = SIR_net_diffusion(multinet_test, interlayers_beta, intra_gamma, iterations=10, initial_state=initial_state)
print("Number of steps in the diffusion: ", len(dif))

for i, state in enumerate(dif):
# plot the network labeling only the starting infected nodes, all infected nodes are in red
    fig = plt.figure(figsize=(30, 24))
    ax = fig.add_subplot(111, projection='3d')

    nodeColorDict = {
        (node, layer): 'red' if state[(node, layer)] == 1 else 'blue' if state[(node, layer)] == 2 else 'gray'
        for (node, layer) in multinet_test.iter_node_layers()
    }
    #set coordinates for the nodes


    nodeLabelDict = {
        (node, layer): node for (node, layer) in multinet_test.iter_node_layers()
        if state[(node, layer)] != 0
    }

    draw(
        multinet_test,
        nodeCoords=node_coords,
        # nodeLabelRule={},                  # No automatic labels
        defaultNodeLabel="",               # Hide labels
        nodeLabelDict=nodeLabelDict,  # Only labels for infected nodes
        nodeColorDict=nodeColorDict,

        defaultLayerAlpha=0.1,
        defaultLayerLabelLoc=(-0.01,0), 
        defaultEdgeAlpha=0.5,
        defaultNodeLabelSize=40,
        defaultLayerLabelSize=30,
        layergap=1.2,
        ax=ax
    )
    # prin
    plt.savefig(f"imgs/tests/multinet_diffusion_step_{i}-min.png",dpi=50, bbox_inches='tight')
    plt.close(fig)

is_multiplex:  True
Epidemic has ended, all nodes are either susceptible or recovered.
Number of steps in the diffusion:  5


# Apply to Madrid Transport Data

## 2.1. ---read from csv---

In [13]:
def _nodes_to_link(node1, node2):
    """Return a link when tuple of nodes is given in the graph representing
    the multislice structure. I.e. when given (i,s_1,...,s_d),(j,r_1,...,r_d)
    (i,j,s_1,r_1, ... ,s_d,r_d) is returned.
    """
    assert len(node1) == len(node2)
    l = []
    for i, n1 in enumerate(node1):
        l.append(n1)
        l.append(node2[i])
    return tuple(l)


def read_multiplex_node_layer_edges_from_csv(file_path, weighted=False, directed=False):
    """
    Reads multilayer edges from a CSV in node,layer,node,layer format and returns a pymnet network.

    Parameters
    ----------
    file_path : str
        Path to the CSV file.
    weighted : bool
        If True, assumes the last column is edge weight.
    directed : bool
        If True, creates a directed network.
    couplings : str
        Coupling type for pymnet network (only applies to Multiplex).

    Returns
    -------
    MultilayerNetwork
    """
    # Load data
    df = pd.read_csv(file_path)

    # Handle weights if present
    if weighted:
        weights = df.iloc[:, -1]
        edge_data = df.iloc[:, :-1]
    else:
        weights = pd.Series([1] * len(df))
        edge_data = df

    if edge_data.shape[1] != 4:
        raise ValueError("Expected columns: node1, layer1, node2, layer2 (plus optional weight).")

    # Initialize general multilayer network
    net = MultilayerNetwork(directed=directed, aspects=1)

    # Add edges
    for (_, row), weight in zip(edge_data.iterrows(), weights):
        u = (row[0], row[1])  # (node1, layer1)
        v = (row[2], row[3])  # (node2, layer2)
        net[u][v] = weight

    return net

def read_multiplex_edges_from_csv(file_path, weighted=False, directed=False):
    """
    Reads multilayer edges from a CSV and returns a pymnet network.

    Parameters
    ----------
    file_path : str
        Path to the CSV file.
    weighted : bool
        If True, assumes the last column is edge weight.
    directed : bool
        If True, creates a directed network.

    Returns
    -------
    MultiplexNetwork
    """
    # Load data
    df = pd.read_csv(file_path)

    # Handle weights if present
    if weighted:
        weights = df.iloc[:, -1]
        edge_data = df.iloc[:, :-1]
    else:
        weights = pd.Series([1] * len(df))
        edge_data = df.iloc[:, :]
    # Detect number of layers from column count
    n_cols = edge_data.shape[1]
    if n_cols % 2 != 0:
        raise ValueError("Number of columns must be even (pairs of node-layer).")

    # Initialize network
    net = MultilayerNetwork(directed=directed, aspects=1)
    #min wight > 0
    min_weight = weights[weights > 0].min() if weighted else 1


    # Add edges
    for (_, row), weight in zip(edge_data.iterrows(), weights):
        # net[tuple(row)] =1 

        net[tuple(row)] = min_weight/weight if weight > 0 else 0

    return net

In [109]:
nodes_path = '../Data/Final_data/madrid_transport_nodes.csv'
shp_nodes_path = '../Data/Final_data/final_nodes.csv'
edges_path = '../Data/Final_data/madrid_transport_edges.csv'
edges_no_weight_path = '../Data/Final_data/madrid_transport_edges_no_weight.csv'
madrid_transport_nodes_shp = pd.read_csv(shp_nodes_path)

node_coords = {
    row['stop']: (row['stop_lon'], row['stop_lat'])  # X = lon, Y = lat
    for _, row in madrid_transport_nodes_shp.iterrows()
}

nodes = pd.read_csv(nodes_path)

In [110]:
mnet_weighted = read_multiplex_edges_from_csv(edges_path, weighted=True)
mnet_no_weighted = read_multiplex_edges_from_csv(edges_no_weight_path, weighted=False)

In [142]:
# if they have 3 or less neighbors, set the size to 0, otherwise, 1
nodesize_neighbors = {
    (node, layer): 1 if len(mnet_no_weighted[(node, layer)]) > 3 else 0
    for (node, layer) in mnet_no_weighted.iter_node_layers()
}

#### Madrid louvain

In [232]:
# madrid_louv_states= []
# while len(madrid_louv_states) != 15:
#     madrid_louv_states = louvain_algorithm(mnet_no_weighted, max_iter=1000, force_merge=True)

for i, state in enumerate(madrid_louv_states[:4]):
    print(f"    Iteration {i}: Objective function value: {state['objective_function_value']}")
    louv_com_net, louv_com_sizes, louv_com_2_name = generate_louvain_communities_net(mnet_no_weighted, madrid_louv_states[i])




    nodeSizeDict_louv = {
        (louv_com_2_name[com], 1) : louv_com_sizes[louv_com_2_name[com]] /1000
        for com, nodes in state['com2nodes'].items()
    }


    colors = plt.get_cmap('tab20', len(louv_com_net.slices[0])+2)
    # map each community to a color
    nodeColorDict_louv = {
        node: colors(i) if node in louv_com_sizes else 'black'
        for i, node in enumerate(louv_com_net.slices[0])
        if node in louv_com_sizes
    }

    nodeColorDict_louv = {
        (louv_com_2_name[com], 1) : nodeColorDict_louv[louv_com_2_name[com]]
        for com, nodes in state['com2nodes'].items()
    }

    custom_node_labels = {
        (louv_com_2_name[com], 1): louv_com_2_name[com] if nodeSizeDict_louv[(louv_com_2_name[com], 1)] > 0.003 else ''  # Custom labels for communities
        for com, nodes in state['com2nodes'].items()
    }


    
    fig2 = plt.figure(figsize=(36, 24))
    ax2 = fig2.add_subplot(111, projection='3d')

    draw(
        louv_com_net,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict=nodeSizeDict_louv,  # Use global node sizes
        nodeColorDict=nodeColorDict_louv,
        nodeLabelRule={},                  # No automatic labels
        nodeLabelDict=custom_node_labels,  # Use custom labels for communities
        defaultNodeLabelSize=10,
        defaultLayerLabelSize=0,
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.3,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )

    fig2.savefig(f'imgs/madrid/louvain_step_{i}.png', dpi=360, bbox_inches='tight')
    plt.close(fig2)
    

    Iteration 0: Objective function value: 0.24441960613538444
    Iteration 1: Objective function value: 0.6636638118740004
    Iteration 2: Objective function value: 0.7686763116723911
    Iteration 3: Objective function value: 0.7700072579474512


In [234]:
best_state = madrid_louv_states[4]
louv_com_net, louv_com_sizes, louv_com_2_name = generate_louvain_communities_net(mnet_no_weighted, best_state)
colors = plt.get_cmap('tab20', len(best_state['com2nodes']))
# map each community to a color


nodeSizeDict = {
    (n,l ) : 0.01
    for n,l in mnet_no_weighted.iter_node_layers()
}




nodeColorDict_louv_base = {
    (node,layer) :nodeColorDict_louv[(louv_com_2_name[best_state['node2com'][(node,layer)]],1)]
    for node, layer in mnet_no_weighted.iter_node_layers()
    
}

fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')

draw(
        mnet_no_weighted,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict={
            # if the node has more than 3 neighbors, set the size to 0.01, else 0
            nl: 0.01 if len(mnet_no_weighted[nl]) > 2 else 0
            for nl in nodeSizeDict.keys()
        },  # Use global node sizes
        nodeColorDict=nodeColorDict_louv_base,
        nodeLabelRule={},                  # No automatic labels
        nodeLabelDict=custom_node_labels,  # Use custom labels for communities
        defaultNodeLabelSize=40,
        defaultLayerLabelSize=30,
        defaultLayerLabelLoc=(-0.01,0),
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.2,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )
fig2.savefig(f'imgs/madrid/base_louv.png', bbox_inches='tight')
plt.close(fig2)

fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')




nodeSizeDict_louv = {
    (louv_com_2_name[com], 1) : louv_com_sizes[louv_com_2_name[com]] /1000
    for com, nodes in best_state['com2nodes'].items()
}
draw(
        louv_com_net,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict=nodeSizeDict_louv,  # Use global node sizes
        nodeColorDict=nodeColorDict_louv,
        nodeLabelRule={},                  # No automatic labels
        nodeLabelDict=custom_node_labels,  # Use custom labels for communities
        defaultNodeLabelSize=10,
        defaultLayerLabelSize=0,
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.3,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )
fig2.savefig(f'imgs/madrid/louvain_step_{3}.png', bbox_inches='tight')

In [ ]:
nodeSizeDict_louv = {
    (louv_com_2_name[com], 1) : louv_com_sizes[louv_com_2_name[com]] /1000
    for com, nodes in state['com2nodes'].items()
}

{'CONGOSTO': 'VICALVARO',
 'LA RAMBLA': 'CANILLEJAS',
 'NUEVOS MINISTERIOS': 'NUEVOS MINISTERIOS',
 'DELICIAS': 'PLAZA ELIPTICA',
 'CAMPAMENTO': 'ALUCHE',
 'QUINTANA': 'ALSACIA',
 'BEGOÑA': 'BEGOÑA',
 'HOSPITAL SEVERO OCHOA': 'VILLAVERDE BAJO CRUCE',
 'JOAQUIN VILUMBRALES': 'CUATRO VIENTOS',
 'PINAR DEL REY': 'COLOMBIA',
 'GOYA': 'AVENIDA DE AMERICA'}

In [238]:
#dendrogram
from sklearn.metrics import normalized_mutual_info_score
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

# Suppose:
# node2com_list = [dict_node2com_level_0, dict_node2com_level_1, ..., dict_node2com_level_n]
# modularity = [q0, q1, ..., qn]

nodes = sorted(set().union(*[set(state['node2com'].keys()) for state in madrid_louv_states]))

def get_labels(node2com):
    return [node2com.get(node, -1) for node in nodes]

labels_matrix = [get_labels(state['node2com']) for state in madrid_louv_states]

n_levels = len(madrid_louv_states)
distances = np.zeros((n_levels, n_levels))

for i in range(n_levels):
    for j in range(i + 1, n_levels):
        distances[i, j] = 1 - normalized_mutual_info_score(labels_matrix[i], labels_matrix[j])
        distances[j, i] = distances[i, j]

linkage_matrix = linkage(squareform(distances), method="average")

fig = plt.figure(figsize=(10, 6))
dendrogram(
    linkage_matrix,
    labels=[f"L{i} (Q={madrid_louv_states[i]['objective_function_value']:.3f})" for i in range(n_levels)],
    orientation='top'
)
plt.title("Louvain Partition Dendrogram")
plt.ylabel("1 - NMI Distance")
#modify x-axis labels to show modularity values
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.xlabel("Louvain Levels")
plt.tick_params(axis='x', which='major', labelsize=10)
plt.tick_params(axis='y', which='major', labelsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)


plt.tight_layout()
fig.savefig('imgs/madrid/louvain_dendrogram.png', dpi=300, bbox_inches='tight')
plt.close(fig)

In [299]:
# louvain modularity scatterplot and number of communities barplot figure
import matplotlib.pyplot as plt

modularities = [state['objective_function_value'] for state in madrid_louv_states]
num_communities = [len(state['com2nodes']) for state in madrid_louv_states]


fig, ax1 = plt.subplots(figsize=(10, 6))
ax2 = ax1.twinx()  # Create a second y-axis
ax1.plot(modularities, marker='o', color='b', label='Modularity (Q)', linestyle='-')
ax2.bar(range(len(num_communities)), num_communities, alpha=0.3, color='r', label='Number of Communities')
ax1.set_xticks(range(len(modularities)))
ax2.set_xticklabels([f'L{i}\n{num_communities[i]} ' for i in range(len(modularities))], rotation=45, ha='right', fontsize=10)
# under y axis write  partition \n number of communities 
ax1.set_xlabel('Louvain Iteration')
ax1.set_ylabel('Modularity (Q)', color='b')

ax2.set_ylabel('Number of Communities', color='r')
ax2.grid(axis='y', linestyle='--', alpha=0.5)
ax1.tick_params(axis='y', labelcolor='b')
ax2.tick_params(axis='y', labelcolor='r')
ax1.set_title('Louvain Modularity and Number of Communities')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.tight_layout()
fig.savefig('imgs/madrid/louvain_modularity_communities.png', dpi=300, bbox_inches='tight')
plt.close(fig)

In [172]:
important_nodes = [
    'AVENIDA DE AMERICA',
    'USERA',
    'JUNTA MUNICIPAL USERA',
    
    'BARRIO DEL PILAR',

    'PUERTA DEL SOL',
    
    'LA MORALEJA',
    'GETAFE CENTRAL',
    'PUERTA DEL SUR',
    'PUEBLO NUEVO',
    'MONCLOA',
    'EL CAPRICHO',
    'COLONIA JARDIN']


# Custom labels for selected stops
custom_node_labels = {  node:node for node in important_nodes }
# Assign a unique color to each node (using a color palette or named colors)
custom_node_colors = { node: 'red' for node in important_nodes}


nodeAlphaDict = {
    (node, layer): 0.6 if node in custom_node_labels else 0.4
    for (node, layer) in mnet_no_weighted.iter_node_layers()
}
nodeSizeDict = {
    (node, layer): 0.01 if node in custom_node_labels else 0.005
    for (node, layer) in mnet_no_weighted.iter_node_layers()
}
nodeColorDict = {
    (node, layer): 'red' if node in custom_node_labels else 'lightgray'
    for (node, layer) in mnet_no_weighted.iter_node_layers()
}
#for each (node,layer) if layer is metro but node is not in metro_nodes_dict, set the size to 0
for (node, layer) in mnet_no_weighted.iter_node_layers():
    if layer == 'metro' and node not in nodes[nodes['transporte'] == 'metro']['stop'].values:
        nodeSizeDict[(node, layer)] = 0.0
for (node, layer) in mnet_no_weighted.iter_node_layers():
    if (layer == 'urban') and node not in nodes[nodes['transporte'] == 'urban']['stop'].values:
        nodeSizeDict[(node, layer)] = 0.0
for (node, layer) in mnet_no_weighted.iter_node_layers():
    if (layer == 'interurban') and node not in nodes[nodes['transporte'] == 'interurban']['stop'].values:
        nodeSizeDict[(node, layer)] = 0.0

# nodeSizeRuleDict={"rule": "scaled", "scalecoeff": 0.15},
# Build nodeLabelDict and nodeColorDict for all matching node-layer pairs
nodeLabelDict = {
    (node, layer): custom_node_labels[node]
    for (node, layer) in mnet_no_weighted.iter_node_layers()
    if node in custom_node_labels
}


# Draw the multilayer network
fig = plt.figure(figsize=(36, 24))
ax_n = fig.add_subplot(111, projection='3d')
draw(
    mnet_no_weighted,
    nodeLabelDict=nodeLabelDict,       # Show only selected labels
    nodeLabelRule={},                  # No automatic labels
    nodeCoords=node_coords,            # Use shapefile-coordinates
    defaultNodeLabel="",               # Hide others
    defaultLayerLabelLoc=(-0.1, 0.1),  # Position layer names
    defaultEdgeAlpha=0.4,              # Edge transparency
    defaultNodeLabelAlpha=1,
    defaultNodeLabelColor='red',
    defaultNodeColor='lightgray',  # Default node color
    
    nodeColorDict=nodeColorDict,       # Custom node colors
    nodeSizeDict=nodeSizeDict,         # Custom node sizes
    layergap=1,                        # Layer vertical separation
    ax=ax_n
)

fig.savefig('partial_madrid_transport_network.png', dpi=300, bbox_inches='tight')

In [ ]:
# --- First: Plot leiden_net with community size scaling ---
# Scale node sizes using leiden_com_sizes (adjust scale factor as needed)
GAMMA = 0.0005

leiden_states = leiden_algorithm(mnet_weighted, gamma=GAMMA, max_iterations=100)
print(len(leiden_states))
print('communities found by leiden algorithm:', len(leiden_states[-1]['com2nodes']))
leiden_com_net, leiden_com_sizes, leiden_com_2_name = generate_leiden_communities_net(mnet_weighted, leiden_states[-1])


nodeSizeDict_leiden = {
    (node, layer): leiden_com_sizes[node] / 1300 # Tune 0.5 as needed
    for (node, layer) in leiden_com_net.iter_node_layers()
    if node in leiden_com_sizes
}
nodeSizeDict_global = {
    (node, layer): 0.01 for (node, layer) in mnet_weighted.iter_node_layers()
}


node_coords = {
    row['stop']: (row['stop_lon'], row['stop_lat'])  # X = lon, Y = lat
    for _, row in madrid_transport_nodes_shp.iterrows()
}
colors = plt.get_cmap('tab20', len(leiden_com_net.slices[0]))
# map each community to a color
nodeColorDict_leiden = {
    node: colors(i) if node in leiden_com_sizes else 'black'
    for i, node in enumerate(leiden_com_net.slices[0])
    if node in leiden_com_sizes
}

# for each node in the community, assign the color of the community
nodeColorDict_leiden = {
    (node, layer): nodeColorDict_leiden[node]
    for (node, layer) in leiden_com_net.iter_node_layers()
    if node in nodeColorDict_leiden
}
# change GETAFE CENTRAL color to COLOMBIA color and vice versa
nodeColorDict_global = {
    node:  nodeColorDict_leiden[( leiden_com_2_name[com],1)]
    for com, nodes in leiden_states[-1]['com2nodes'].items()
    for node in nodes
}



#### PLOTS ####
fig1 = plt.figure(figsize=(36, 24))
ax1 = fig1.add_subplot(111, projection='3d')

draw(
    leiden_com_net,
    nodeCoords=node_coords,
    nodeColorDict=nodeColorDict_leiden,  # Use custom colors for communities
    defaultNodeLabel="",               # Hide labels
    defaultEdgeAlpha=0.4,
    nodeSizeDict=nodeSizeDict_leiden, # Scaled by community size
    layergap=1,
    ax=ax1
)

custom_node_labels = {  node:node for node in leiden_com_net.slices[0] }


fig1.savefig(f'leiden_communities_scaled_gamma_{GAMMA}.png', dpi=300, bbox_inches='tight')


# --- Second: Plot original net with community nodes in red ---
# Identify nodes in leiden_net
leiden_nodes = set(node for node, _ in leiden_com_net.iter_node_layers())

base_nodecoords = {
    node: node_coords[node] for node in leiden_nodes if node in node_coords
}
fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')

draw(
    mnet_weighted,
    nodeCoords=node_coords,
    nodeSizeDict={
    # if the node has more than 3 neighbors, set the size to 0.01, else 0
            nl: 0.01 if len(mnet_no_weighted[nl]) > 2 else 0
            for nl in nodeSizeDict.keys()
        },  # Use global node sizes
    nodeColorDict=nodeColorDict_global,
    nodeLabelRule={},                  # No automatic labels
    defaultNodeLabel="",
    defaultEdgeAlpha=0.3,
    layergap=1,
    ax=ax2
)

fig2.savefig(f'original_network_highlighted_leiden_nodes_gamma_{GAMMA}.png', dpi=300, bbox_inches='tight')

Convergence reached.
1
communities found by leiden algorithm: 14


KeyboardInterrupt: 

In [509]:
# --- First: Plot louvain_net with community size scaling ---
# Scale node sizes using louvain_com_sizes (adjust scale factor as needed)

louvain_states = louvain_algorithm(mnet_no_weighted)
print(len(louvain_states))
print('communities found by louvain algorithm:', len(louvain_states[-1]['com2nodes']))
louvain_com_net, louvain_com_sizes, louvain_com_2_name = generate_louvain_communities_net(mnet_no_weighted, louvain_states[-1])


nodeSizeDict_louvain = {
    (node, layer): louvain_com_sizes[node] / 1000 # Tune 0.5 as needed
    for (node, layer) in louvain_com_net.iter_node_layers()
    if node in louvain_com_sizes
}
nodeSizeDict_global = {
    (node, layer): 0.01 for (node, layer) in mnet_no_weighted.iter_node_layers()
}


node_coords = {
    row['stop']: (row['stop_lon'], row['stop_lat'])  # X = lon, Y = lat
    for _, row in madrid_transport_nodes_shp.iterrows()
}
colors = plt.get_cmap('tab20', len(louvain_com_net.slices[0]))
# map each community to a color
nodeColorDict_louvain = {
    node: colors(i) if node in louvain_com_sizes else 'black'
    for i, node in enumerate(louvain_com_net.slices[0])
    if node in louvain_com_sizes
}

# for each node in the community, assign the color of the community
nodeColorDict_louvain = {
    (node, layer): nodeColorDict_louvain[node]
    for (node, layer) in louvain_com_net.iter_node_layers()
    if node in nodeColorDict_louvain
}
# change GETAFE CENTRAL color to COLOMBIA color and vice versa
nodeColorDict_global = {
    node:  nodeColorDict_louvain[( louvain_com_2_name[com],1)]
    for com, nodes in louvain_states[-1]['com2nodes'].items()
    for node in nodes
}  

#show only labeles of communities with more than 6 nodes 
nodeLabelDict_louvain = {
    (node, layer): node
    for (node, layer) in louvain_com_net.iter_node_layers()
    if node in louvain_com_sizes and louvain_com_sizes[node] > 9
}


#### PLOTS ####
fig1 = plt.figure(figsize=(36, 24))
ax1 = fig1.add_subplot(111, projection='3d')

draw(
    louvain_com_net,
    nodeCoords=node_coords,
    nodeColorDict=nodeColorDict_louvain,  # Use custom colors for communities
    nodeLabelDict=nodeLabelDict_louvain,  # Show only selected labels
    nodeLabelRule={},                  # No automatic labels
    defaultNodeLabel="",               # Hide labels
    defaultEdgeAlpha=0.4,
    nodeSizeDict=nodeSizeDict_louvain, # Scaled by community size
    layergap=1,
    ax=ax1
)

custom_node_labels = {  node:node for node in louvain_com_net.slices[0] }


fig1.savefig(f'imgs/louvain_communities_scaled.png', dpi=300, bbox_inches='tight')


# --- Second: Plot original net with community nodes in red ---
# Identify nodes in louvain_net
louvain_nodes = set(node for node, _ in louvain_com_net.iter_node_layers())


fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')

draw(
    mnet_no_weighted,
    nodeCoords=node_coords,
    nodeSizeDict=nodeSizeDict_global,  # Use global node sizes
    nodeColorDict=nodeColorDict_global,
    nodeLabelRule={},                  # No automatic labels
    defaultNodeLabel="",
    defaultEdgeAlpha=0.3,
    layergap=1,
    ax=ax2
)

fig2.savefig(f'imgs/original_network_highlighted_louvain_nodes.png', dpi=300, bbox_inches='tight')

101
communities found by louvain algorithm: 142


#### Madrid Leiden

##### unweighted

In [255]:
gammas = [0.001]
leiden_by_gamma = {}
for gamma in gammas:
    leiden_states = leiden_algorithm(mnet_weighted, gamma=gamma, max_iterations=100)
    print(f'Leiden algorithm with gamma={gamma} found {len(leiden_states)} states.')
    print('communities found by leiden algorithm:', len(leiden_states[-1]['com2nodes']))
    leiden_by_gamma[gamma] = leiden_states[-1]

Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Convergence reached.
Leiden algorithm with gamma=0.001 found 5 states.
communities found by leiden algorithm: 14


In [305]:

for gamma, state in leiden_by_gamma.items():

    print(f"Gamma: {gamma}, Communities: {len(state['com2nodes'])}, Objective Function Value: {state['objective_function_value']:.4f}")
    print('Number of communities:', len(state['com2nodes']))
    leid_com_net, leid_com_sizes, leid_com_2_name = generate_leiden_communities_net(mnet_no_weighted, state)


    nodeSizeDict_leid = {
        (leid_com_2_name[com], 1) : np.sqrt(leid_com_sizes[leid_com_2_name[com]]) /150
        for com, nodes in state['com2nodes'].items()
    }


    colors = plt.get_cmap('tab20', len(leid_com_net.slices[0])+3)
    # map each community to a color
    nodeColorDict_leid = {
        node: colors(i) if node in leid_com_sizes else 'black'
        for i, node in enumerate(leid_com_net.slices[0])
        if node in leid_com_sizes
    }

    nodeColorDict_leid = {
        (leid_com_2_name[com], 1) : nodeColorDict_leid[leid_com_2_name[com]]
        for com, nodes in state['com2nodes'].items()
    }

    custom_node_labels = {
        (leid_com_2_name[com], 1): leid_com_2_name[com] if nodeSizeDict_leid[(leid_com_2_name[com], 1)] > 0.0075 else ''  # Custom labels for communities
        for com, nodes in state['com2nodes'].items()
    }



    
    fig2 = plt.figure(figsize=(36, 24))
    ax2 = fig2.add_subplot(111, projection='3d')

    draw(
        leid_com_net,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict=nodeSizeDict_leid,  # Use global node sizes
        nodeColorDict=nodeColorDict_leid,
        nodeLabelRule={},                  # No automatic labels
        nodeLabelDict=custom_node_labels,  # Use custom labels for communities
        defaultNodeLabelSize=10,
        defaultLayerLabelSize=0,
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.3,
        defaultEdgeAlpha=0.4,
        ax=ax2
    )

    fig2.savefig(f'imgs/madrid/leiden/leiden_gamma_{gamma}.png', bbox_inches='tight')
    plt.close(fig2)

    fig2 = plt.figure(figsize=(36, 24))
    ax2 = fig2.add_subplot(111, projection='3d')

    # print( state['node2com'][('PACIFICO', 'interurban')])
    nodeColorDict_leid_base = {
    (node,layer) :nodeColorDict_leid[(leid_com_2_name[state['node2com'][(node,layer)]],1)] 
            if len(mnet_no_weighted[(node, layer)]) > 2 
            else 'black'
            for node, layer in mnet_no_weighted.iter_node_layers() 
    }
    nodeColorDict_leid_base = {
        (node,layer) :nodeColorDict_leid[(leid_com_2_name[state['node2com'][(node,layer)]],1)]
                if len(mnet_no_weighted[(node, layer)]) > 2 
                else 'black'
        for node, layer in mnet_no_weighted.iter_node_layers()
        
    }  
    nodeColorDict_leid_base  
    draw(
        mnet_no_weighted,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict={
            # if the node has more than 3 neighbors, set the size to 0.01, else 0
            nl: 0.01 if len(mnet_no_weighted[nl]) > 2 else 0
            for nl in nodeSizeDict.keys()
        },  # Use global node sizes,  # Use global node sizes
        nodeColorDict=nodeColorDict_leid_base,
        nodeLabelRule={},                  # No automatic labels
        nodeLabelDict=custom_node_labels,  # Use custom labels for communities
        defaultLayerLabelLoc=(-0.01,0),
        defaultNodeLabelSize=40,
        defaultLayerLabelSize=30,
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.2,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )
    
    fig2.savefig(f'imgs/madrid/leiden/base_leiden_gamma_{gamma}.png', bbox_inches='tight')
    plt.close(fig2)

Gamma: 0.001, Communities: 14, Objective Function Value: 330.8068
Number of communities: 14


In [304]:
state = leiden_by_gamma[gammas[0]]
# print( state['node2com'][('PACIFICO', 'interurban')])
nodeColorDict_leid_base = {
(node,layer) :nodeColorDict_leid[(leid_com_2_name[state['node2com'][(node,layer)]],1)] 
        if len(mnet_no_weighted[(node, layer)]) > 3 
        else 'lightgray'
        for node, layer in mnet_no_weighted.iter_node_layers() 
}
nodeColorDict_leid_base = {
    (node,layer) :nodeColorDict_leid[(leid_com_2_name[state['node2com'][(node,layer)]],1)]
            if len(mnet_no_weighted[(node, layer)]) > 3 
            else 'lightgray'
    for node, layer in mnet_no_weighted.iter_node_layers()
    
}  
nodeColorDict_leid_base 


{('PACIFICO', 'urban'): (0.8392156862745098,
  0.15294117647058825,
  0.1568627450980392,
  1.0),
 ('PACIFICO', 'metro'): (0.8392156862745098,
  0.15294117647058825,
  0.1568627450980392,
  1.0),
 ('PACIFICO', 'interurban'): 'lightgray',
 ('EMBAJADORES', 'urban'): (0.8392156862745098,
  0.15294117647058825,
  0.1568627450980392,
  1.0),
 ('EMBAJADORES', 'metro'): 'lightgray',
 ('EMBAJADORES', 'interurban'): 'lightgray',
 ('JULIAN BESTEIRO', 'urban'): 'lightgray',
 ('JULIAN BESTEIRO', 'metro'): 'lightgray',
 ('JULIAN BESTEIRO', 'interurban'): (0.9686274509803922,
  0.7137254901960784,
  0.8235294117647058,
  1.0),
 ('CONDE DE CASAL', 'urban'): (0.8392156862745098,
  0.15294117647058825,
  0.1568627450980392,
  1.0),
 ('CONDE DE CASAL', 'metro'): (0.8392156862745098,
  0.15294117647058825,
  0.1568627450980392,
  1.0),
 ('CONDE DE CASAL', 'interurban'): 'lightgray',
 ('PARQUE DE LOS ESTADOS', 'urban'): 'lightgray',
 ('PARQUE DE LOS ESTADOS', 'metro'): 'lightgray',
 ('PARQUE DE LOS ESTADO

In [ ]:
best_state = madrid_louv_states[4]
louv_com_net, louv_com_sizes, louv_com_2_name = generate_louvain_communities_net(mnet_no_weighted, best_state)
colors = plt.get_cmap('tab20', len(best_state['com2nodes']))
# map each community to a color


nodeSizeDict = {
    (n,l ) : 0.01
    for n,l in mnet_no_weighted.iter_node_layers()
}




nodeColorDict_louv_base = {
    (node,layer) :nodeColorDict_louv[(louv_com_2_name[best_state['node2com'][(node,layer)]],1)]
    for node, layer in mnet_no_weighted.iter_node_layers()
    
}

fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')

draw(
        mnet_no_weighted,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict=nodeSizeDict,  # Use global node sizes
        nodeColorDict=nodeColorDict_louv_base,
        nodeLabelRule={},                  # No automatic labels
        nodeLabelDict=custom_node_labels,  # Use custom labels for communities
        defaultNodeLabelSize=40,
        defaultLayerLabelSize=30,
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.2,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )
fig2.savefig(f'imgs/madrid/base_louv.png', bbox_inches='tight')
plt.close(fig2)

fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')

nodeColorDict_louv_base = {
    (node,layer) :nodeColorDict_louv[(louv_com_2_name[best_state['node2com'][(node,layer)]],1)]
    for node, layer in mnet_no_weighted.iter_node_layers()
    
}

nodeSizeDict_louv = {
    (louv_com_2_name[com], 1) : louv_com_sizes[louv_com_2_name[com]] /1000
    for com, nodes in state['com2nodes'].items()
}
draw(
        louv_com_net,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict={
            nl: 0.01 if len(mnet_no_weighted[nl]) > 2 else 0
            for nl in nodeSizeDict.keys()
        },  # Use global node sizes
        nodeColorDict=nodeColorDict_louv,
        nodeLabelRule={},                  # No automatic labels
        nodeLabelDict=custom_node_labels,  # Use custom labels for communities
        defaultNodeLabelSize=10,
        defaultLayerLabelSize=0,
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.3,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )
fig2.savefig(f'imgs/madrid/louvain_step_{3}.png', bbox_inches='tight')

NameError: name 'madrid_louv_states' is not defined

In [143]:
# remove    leid_com_net, leid_com_sizes, leid_com_2_name = generate_leiden_communities_net(mnet_no_weighted, state)
leid_com_net, leid_com_sizes, leid_com_2_name = generate_leiden_communities_net(mnet_no_weighted, leiden_by_gamma[0.001])
leid_com_2_name

{'JUAN DE LA CIERVA': 'GETAFE CENTRAL',
 'SAN CRISTOBAL': 'VILLAVERDE BAJO CRUCE',
 'PALOS DE LA FRONTERA': 'PLAZA ELIPTICA',
 'PUEBLO NUEVO': 'ALSACIA',
 'AEROPUERTO T1-T2-T3': 'CANILLEJAS',
 'LA PESETA': 'PAN BENDITO',
 'ACACIAS': 'PIRAMIDES',
 'MARQUES DE LA VALDAVIA': 'MARQUES DE LA VALDAVIA',
 'SAN FERNANDO': 'SAN FERNANDO',
 'VICALVARO': 'VICALVARO',
 'ARGÜELLES': 'PRINCIPE PIO',
 'ARROYO CULEBRO': 'CONSERVATORIO',
 'PARQUE DE SANTA MARIA': 'COLOMBIA',
 'UNIVERSIDAD REY JUAN CARLOS': 'PRADILLO',
 'AVENIDA DE AMERICA': 'AVENIDA DE AMERICA',
 'RIVAS VACIAMADRID': 'RIVAS VACIAMADRID',
 'VILLA DE VALLECAS': 'SIERRA DE GUADALUPE',
 'SANTIAGO BERNABEU': 'BEGOÑA',
 'BATAN': 'CAMPAMENTO'}

In [122]:
# for each gamma compute 100 times the resolution 
for gamma, state in leiden_by_gamma.items():
    for i in range(200):
        best_leiden_com2nodes = leiden_algorithm(mnet_weighted, gamma=gamma)[-1]['com2nodes']
        sizes = [len(nodes) for nodes in best_leiden_com2nodes.values()]
        gamma_loglog[gamma].extend(sizes)  # Flattened list of all community sizes


Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Convergence reac

In [123]:
#remove keys in gamma_loglog that are not in gammas = [0.0001, 0.0005, 0.001, 0.005, 0.01]
gammas = [0.0001, 0.0005, 0.001, 0.005, 0.01]
gamma_loglog = {gamma: gamma_loglog[gamma] for gamma in gammas if gamma in gamma_loglog}

gammas = [0.0001, 0.0005]

# for each gamma compute 100 times the resolution 
for gamma, state in leiden_by_gamma.items():
    for i in range(200):
        best_leiden_com2nodes = leiden_algorithm(mnet_weighted, gamma=gamma)[-1]['com2nodes']
        sizes = [len(nodes) for nodes in best_leiden_com2nodes.values()]
        gamma_loglog[gamma].extend(sizes)  # Flattened list of all community sizes


Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Convergence reached.
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Convergence reac

In [125]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(10, 6))

for gamma, sizes in gamma_loglog.items():
    values, counts = np.unique(sizes, return_counts=True)
    plt.plot(values, counts, marker='o', linestyle='-', label=f'γ={gamma}')

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Community Size (log scale)')
plt.ylabel('Frequency (log scale)')
plt.title('Community Size Distribution across γ values')
plt.legend()
plt.grid(True, which="both", ls="--", linewidth=0.5)
plt.tight_layout()
plt.savefig('imgs/madrid/leiden/community_size_distribution.png', dpi=300, bbox_inches='tight')
plt.close(fig)

##### Weighted

In [92]:
gammas = [0.0001, 0.0003, 0.0005, 0.001, 0.003, 0.005, 0.01, ]
leiden_by_gamma = {}
for gamma in gammas:
    leiden_states = leiden_algorithm(mnet_weighted, gamma=gamma, max_iterations=100)
    print(f'Leiden algorithm with gamma={gamma} found {len(leiden_states)} states.')
    print('Number of communities:', len(leiden_states[-1]['com2nodes']))
    leiden_by_gamma[gamma] = leiden_states[-1]
    

Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Convergence reached.
Leiden algorithm with gamma=0.0001 found 6 states.
Number of communities: 3
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Convergence reached.
Leiden algorithm with gamma=0.0003 found 5 states.
Number of communities: 9
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Convergence reached.
Leiden algorithm with gamma=0.0005 found 5 states.
Number of communities: 13
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Convergence reached.
Leiden algorithm with gamma=0.001 found 5 states.
Number of communities: 23
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Convergence reached.
Leiden algorithm with gamma=0.003 found 5 states.
Number of communities: 42
Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Convergence reache

In [ ]:

for gamma, state in leiden_by_gamma.items():
    print(f"Gamma: {gamma}, Communities: {len(state['com2nodes'])}, Objective Function Value: {state['objective_function_value']:.4f}")
    print('Number of communities:', len(state['com2nodes']))
    leid_com_net, leid_com_sizes, leid_com_2_name = generate_leiden_communities_net(mnet_no_weighted, state)


    nodeSizeDict_leid = {
        (leid_com_2_name[com], 1) : np.sqrt(leid_com_sizes[leid_com_2_name[com]]) /200
        for com, nodes in state['com2nodes'].items()
    }


    colors = plt.get_cmap('tab20', len(leid_com_net.slices[0])+2)
    # map each community to a color
    nodeColorDict_leid = {
        node: colors(i) if node in leid_com_sizes else 'black'
        for i, node in enumerate(leid_com_net.slices[0])
        if node in leid_com_sizes
    }

    nodeColorDict_leid = {
        (leid_com_2_name[com], 1) : nodeColorDict_leid[leid_com_2_name[com]]
        for com, nodes in state['com2nodes'].items()
    }

    custom_node_labels = {
        (leid_com_2_name[com], 1): leid_com_2_name[com] if nodeSizeDict_leid[(leid_com_2_name[com], 1)] > 0.025 else ''  # Custom labels for communities
        for com, nodes in state['com2nodes'].items()
    }


    
    fig2 = plt.figure(figsize=(36, 24))
    ax2 = fig2.add_subplot(111, projection='3d')

    draw(
        leid_com_net,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict=nodeSizeDict_leid,  # Use global node sizes
        nodeColorDict=nodeColorDict_leid,
        nodeLabelRule={},                  # No automatic labels
        nodeLabelDict=custom_node_labels,  # Use custom labels for communities
        defaultNodeLabelSize=10,
        defaultLayerLabelSize=0,
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.3,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )

    fig2.savefig(f'imgs/madrid/leiden/leiden_gamma_{gamma}.png', bbox_inches='tight')
    plt.close(fig2)
    

Gamma: 0.0001, Communities: 3, Objective Function Value: 359.7948
Number of communities: 3
Gamma: 0.0003, Communities: 9, Objective Function Value: 339.5362
Number of communities: 9
Gamma: 0.0005, Communities: 13, Objective Function Value: 334.0583
Number of communities: 13
Gamma: 0.001, Communities: 23, Objective Function Value: 317.2680
Number of communities: 23
Gamma: 0.003, Communities: 42, Objective Function Value: 286.4946
Number of communities: 42
Gamma: 0.005, Communities: 52, Objective Function Value: 268.4527
Number of communities: 52
Gamma: 0.01, Communities: 75, Objective Function Value: 232.5679
Number of communities: 75


#### Madrid PCI

In [ ]:
color_map = plt.get_cmap('viridis', 9)  # 9 colors for PCI values from 0 to 8
color_map(0)
#select a color map 

(0.267004, 0.004874, 0.329415, 1.0)

In [73]:

max(list(mlPCI_1_mad.values()))

6

In [75]:



mlPCI_1_mad = {
    nl:mlPCI_n(mnet_no_weighted, nl,1) for nl in mnet_no_weighted.iter_node_layers()
}

mlPCI_2_mad = {
    nl:mlPCI_n(mnet_no_weighted, nl,2) for nl in mnet_no_weighted.iter_node_layers()
}

mlPCI_3_mad = {
    nl:mlPCI_n(mnet_no_weighted, nl,3) for nl in mnet_no_weighted.iter_node_layers()
}

mlPCI_4_mad = {
    nl:mlPCI_n(mnet_no_weighted, nl,4) for nl in mnet_no_weighted.iter_node_layers()
}
mlPCI_5_mad = {
    nl:mlPCI_n(mnet_no_weighted, nl,5) for nl in mnet_no_weighted.iter_node_layers()
}


allPCI_mad = {
    nl:allPCI(mnet_no_weighted, nl) for nl in mnet_no_weighted.iter_node_layers()
}

lsPCI_mad = {
    nl:lsPCI(mnet_no_weighted, nl) for nl in mnet_no_weighted.iter_node_layers()
}
#print for eachg dictionary the high


color_map = plt.get_cmap('viridis', max(list(mlPCI_1_mad.values())))
mlPCI_1_color_map = {
    nl: color_map(mlPCI_n(mnet_no_weighted, nl, 1))
    for nl in mnet_no_weighted.iter_node_layers()
}

color_map = plt.get_cmap('viridis', max(list(mlPCI_2_mad.values())))
mlPCI_2_color_map = {
    nl: color_map(mlPCI_n(mnet_no_weighted, nl, 2))
    for nl in mnet_no_weighted.iter_node_layers()
}

color_map = plt.get_cmap('viridis', max(list(mlPCI_3_mad.values())))
allPCI_color_map = {
    nl: color_map(allPCI(mnet_no_weighted, nl)) for nl in mnet_no_weighted.iter_node_layers()
}

color_map = plt.get_cmap('viridis', max(list(mlPCI_4_mad.values())))
lsPCI_color_map = {
    nl: color_map(lsPCI(mnet_no_weighted, nl)) for nl in mnet_no_weighted.iter_node_layers()
}


In [76]:
# for mlPCI_1_mad, mlPCI_2_mad, allPCI_mad, lsPCI_mad print the highest value and all nodes with that value
for pci_dict, name in zip(
    [mlPCI_1_mad, mlPCI_2_mad, mlPCI_3_mad, mlPCI_4_mad, mlPCI_5_mad, allPCI_mad, lsPCI_mad],
    ['mlPCI_1', 'mlPCI_2', 'mlPCI_3', 'mlPCI_4', 'mlPCI_5', 'allPCI', 'lsPCI']):
    max_value = max(pci_dict.values())
    nodes_with_max_value = [node for node, value in pci_dict.items() if value == max_value]
    print(f"{name} highest value: {max_value}, nodes: {len(nodes_with_max_value)}")

mlPCI_1 highest value: 6, nodes: 7
mlPCI_2 highest value: 1, nodes: 552
mlPCI_3 highest value: 1, nodes: 248
mlPCI_4 highest value: 0, nodes: 726
mlPCI_5 highest value: 0, nodes: 726
allPCI highest value: 3, nodes: 4
lsPCI highest value: 2, nodes: 230


In [70]:
# important noides are the ones with mlPCI_1 > 4 or allPCI > 2
important_nodes = [
    node for node, value in mlPCI_1_mad.items() if value > 5 or allPCI_mad[node] > 2
]
important_nodes = set(important_nodes)  # Convert to set for faster lookup
important_nodes

# for each important node print the mlPCI_1, mlPCI_2, allPCI and lsPCI values
print("mlPCI_1_mad max value:", max(mlPCI_1_mad.values()))
print("mlPCI_2_mad max value:", max(mlPCI_2_mad.values()))
print("allPCI_mad max value:", max(allPCI_mad.values()))
print("lsPCI_mad max value:", max(lsPCI_mad.values()))
for node in important_nodes:
    print(f"{node}: mlPCI_1={mlPCI_1_mad[node]}, mlPCI_2={mlPCI_2_mad[node]}, allPCI={allPCI_mad[node]}, lsPCI={lsPCI_mad[node]}")


mlPCI_1_mad max value: 6
mlPCI_2_mad max value: 1
allPCI_mad max value: 3
lsPCI_mad max value: 2
('CASA DE CAMPO', 'metro'): mlPCI_1=2, mlPCI_2=1, allPCI=3, lsPCI=2
('ESTACION DEL ARTE', 'urban'): mlPCI_1=6, mlPCI_2=1, allPCI=1, lsPCI=2
('AVENIDA DE AMERICA', 'urban'): mlPCI_1=6, mlPCI_2=1, allPCI=1, lsPCI=2
('LEGAZPI', 'metro'): mlPCI_1=2, mlPCI_2=1, allPCI=3, lsPCI=2
('MARQUES DE VADILLO', 'urban'): mlPCI_1=6, mlPCI_2=1, allPCI=0, lsPCI=0
('PLAZA ELIPTICA', 'metro'): mlPCI_1=2, mlPCI_2=1, allPCI=3, lsPCI=2
('ATOCHA', 'urban'): mlPCI_1=6, mlPCI_2=1, allPCI=1, lsPCI=2
('USERA', 'urban'): mlPCI_1=6, mlPCI_2=1, allPCI=1, lsPCI=2
('RUBEN DARIO', 'urban'): mlPCI_1=6, mlPCI_2=1, allPCI=0, lsPCI=0
('PLAZA ELIPTICA', 'urban'): mlPCI_1=6, mlPCI_2=1, allPCI=1, lsPCI=2
('CHAMARTIN', 'metro'): mlPCI_1=3, mlPCI_2=1, allPCI=3, lsPCI=2


In [78]:

colors_draw = {
    'mlPCI_1': (mlPCI_1_color_map, mlPCI_1_mad),
    'mlPCI_2': (mlPCI_2_color_map, mlPCI_2_mad),
    'allPCI': (allPCI_color_map, allPCI_mad),
    'lsPCI': (lsPCI_color_map, lsPCI_mad)
}

for name, (PCI_color_map, PCI_VALS) in colors_draw.items():
    fig = plt.figure(figsize=(34, 38))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title(name, fontsize=30)

    max_PCI = max(PCI_VALS.values())
    min_PCI = min(PCI_VALS.values())

    #cbar of colormap with as many colors as the number of unique PCI values
    unique_PCI_values = sorted(set(PCI_VALS.values()))
    norm = plt.Normalize(vmin=min_PCI, vmax=max_PCI)
    sm = plt.cm.ScalarMappable(cmap=plt.get_cmap('viridis'), norm=norm)
    sm.set_array([])  # Only needed for older versions of matplotlib
    # cbar must be discrete
    cbar = plt.colorbar(sm, ticks=unique_PCI_values, ax=ax)
    cbar.set_label('PCI Value', fontsize=20)
    cbar.ax.tick_params(labelsize=25)  # Set colorbar tick label size


    nodeColorDict = {
        (node, layer): PCI_color_map[(node, layer)]
        for (node, layer) in mnet_weighted.iter_node_layers()
    }

    #nodesizeDict = 0 if no neighbors, else 0.01

    
    draw(
        mnet_weighted,
        nodeCoords=node_coords,
        nodeColorDict=nodeColorDict,
        nodeSizeDict={
            (node, layer): 0.01 if len(mnet_weighted[(node, layer)]) > 0 else 0
            for (node, layer) in mnet_weighted.iter_node_layers()
        },
        nodeLabelRule={},
        #  nodeLabelDict = {(node, layer): node for (node, layer) in mnet_weighted.iter_node_layers() if
        #                PCI_VALS[(node, layer)] == max_PCI},  # Show only labels for nodes with max PCI
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.1,
        defaultLayerLabelLoc=(-0.01,0), 
        defaultEdgeAlpha=0.5,
        defaultNodeLabelSize=20,
        defaultLayerLabelSize=20,
        layergap=1.2,
        ax=ax
    )

    plt.savefig(f"imgs/madrid/pci/{name}-min.png",dpi=50, bbox_inches='tight')
    plt.close(fig)

#### Madrid EigenVector

In [80]:
weights = tf.constant([[1,0.1, 0.8], [0.1, 3, 0.1], [0.8, 0.1, 1]], dtype=tf.float32)


IL_eig , IL_n_2_layer = independent_layer_eigenvector_centrality(mnet_no_weighted) 
U_eig = uniform_eigenvector_centrality(mnet_no_weighted)
LH_eig, LH_nodes_to_int, LH_layers_to_int, LH_int_to_nodes, LH_int_to_layers = local_heterogeneus_eigenvector_centrality(mnet_no_weighted, weights=weights)
GH_eig, GH_nodes_to_int, GH_layers_to_int, GH_int_to_nodes, GH_int_to_layers = global_heterogeneus_eigenvector_centrality(mnet_no_weighted, weights=weights)

# functions = [independent_layer_eigenvector_centrality, uniform_eigenvector_centrality, local_heterogeneus_eigenvector_centrality, global_heterogeneus_eigenvector_centrality]

In [82]:
for n,layer in IL_n_2_layer.items():
    IL_eig_np = IL_eig[n].numpy()
    #transform to absolute values and normalize
    IL_eig_np = np.abs(IL_eig_np)
    min_val = IL_eig_np.min()
    max_val = IL_eig_np.max()
    IL_eig_grad = (IL_eig_np - min_val) / (max_val - min_val + 1e-8)  # Avoid divide-by-zero
    print()
    # Use a matplotlib colormap
    colormap = plt.get_cmap('viridis')  # or 'plasma', 'inferno', etc.

    # Convert to RGB (exclude alpha channel)
    IL_eig_colors_rgb = np.array(
        [colormap(IL_eig_grad[i])[:3] 
        for i in range(IL_eig_grad.shape[0])
    ])

    # Construct the dictionary {(node, layer): normalized_value}
    color_map = {}
    for node_idx in range(IL_eig_grad.shape[0]):

        node_name = LH_int_to_nodes[node_idx]
        color_map[(node_name, 1)] = IL_eig_colors_rgb[node_idx]


    fig = plt.figure(figsize=(34, 38))
    ax = fig.add_subplot(111, projection='3d')
    #print only one layer
    ax.set_title(f"IL_eig Gradient for Layer {layer}", fontsize=30)
    draw(
        mnet_no_weighted,
        nodeCoords=node_coords,
        nodeColorDict=color_map,
        nodeLabelRule={},                  # No automatic labels
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.1,
        defaultLayerLabelLoc=(-0.01,0), 
        defaultEdgeAlpha=0.5,
        defaultNodeLabelSize=20,
        defaultLayerLabelSize=30,
        layergap=1.2,
        ax=ax
    )

    # Create a mappable object for the colorbar
    from matplotlib.cm import ScalarMappable
    from matplotlib.colors import Normalize

    norm = Normalize(vmin=min_val, vmax=max_val)
    sm = ScalarMappable(norm=norm, cmap=colormap)
    sm.set_array([])

    # Add colorbar to the figure (adjust location as needed)
    cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.1, aspect=30)
    cbar.set_label("LH_eig Gradient", fontsize=30)
    cbar.ax.tick_params(labelsize=24)                       # Tick label font size


    plt.savefig(f"imgs/madrid/eigen/IL_{layer}-min.png", bbox_inches='tight')
    plt.close(fig)

In [83]:
LH_eig_np = LH_eig.numpy()
#transform to absolute values and normalize
LH_eig_np = np.abs(LH_eig_np)
min_val = LH_eig_np.min()
max_val = LH_eig_np.max()
LH_eig_grad = (LH_eig_np - min_val) / (max_val - min_val + 1e-8)  # Avoid divide-by-zero

# Use a matplotlib colormap
colormap = plt.get_cmap('viridis')  # or 'plasma', 'inferno', etc.

# Convert to RGB (exclude alpha channel)
LH_eig_colors_rgb = np.array([
    [colormap(LH_eig_grad[i, j])[:3] for j in range(LH_eig_grad.shape[1])]
    for i in range(LH_eig_grad.shape[0])
])

# Construct the dictionary {(node, layer): normalized_value}
color_map = {}
for layer_idx in range(LH_eig_grad.shape[0]):
    for node_idx in range(LH_eig_grad.shape[1]):
        layer_name = LH_int_to_layers[0][layer_idx]
        node_name = LH_int_to_nodes[node_idx]
        color_map[(node_name, layer_name)] = LH_eig_colors_rgb[layer_idx, node_idx]

fig = plt.figure(figsize=(34, 38))
ax = fig.add_subplot(111, projection='3d')
#add color gradient legend


draw(
    mnet_no_weighted,
    nodeCoords=node_coords,
    nodeSizeDict={
            (node, layer): 0.012 if len(mnet_no_weighted[(node, layer)]) > 0 else 0
            for (node, layer) in mnet_no_weighted.iter_node_layers()
        },
    nodeColorDict=color_map,
    nodeLabelRule={},                  # No automatic labels
    defaultNodeLabel="",               # Hide labels
    defaultLayerAlpha=0.1,
    defaultLayerLabelLoc=(-0.01,0), 
    defaultEdgeAlpha=0.3,
    defaultNodeLabelSize=20,
    defaultLayerLabelSize=30,
    layergap=1.2,
    ax=ax
)

# Create a mappable object for the colorbar
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

norm = Normalize(vmin=min_val, vmax=max_val)
sm = ScalarMappable(norm=norm, cmap=colormap)
sm.set_array([])

# Add colorbar to the figure (adjust location as needed)
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.1, aspect=30)
cbar.set_label("LH_eig Gradient", fontsize=30)
cbar.ax.tick_params(labelsize=24)                       # Tick label font size


plt.savefig(f"imgs/madrid/eigen/LH-min.png", bbox_inches='tight')
plt.close(fig)


In [84]:
GH_eig_np = GH_eig.numpy()
#transform to absolute values and normalize
GH_eig_np = np.abs(GH_eig_np)
min_val = GH_eig_np.min()
max_val = GH_eig_np.max()
GH_eig_grad = (GH_eig_np - min_val) / (max_val - min_val + 1e-8)  # Avoid divide-by-zero

# Use a matplotlib colormap
colormap = plt.get_cmap('viridis')  # or 'plasma', 'inferno', etc.

# Convert to RGB (exclude alpha channel)
GH_eig_colors_rgb = np.array([
    [colormap(GH_eig_grad[i, j])[:3] for j in range(GH_eig_grad.shape[1])]
    for i in range(GH_eig_grad.shape[0])
])

# Construct the dictionary {(node, layer): normalized_value}
color_map = {}
for layer_idx in range(GH_eig_grad.shape[0]):
    for node_idx in range(GH_eig_grad.shape[1]):
        layer_name = GH_int_to_layers[0][layer_idx]
        node_name = GH_int_to_nodes[node_idx]
        color_map[(node_name, layer_name)] = GH_eig_colors_rgb[layer_idx, node_idx]

fig = plt.figure(figsize=(34, 38))
ax = fig.add_subplot(111, projection='3d')
#add color gradient legend


draw(
    mnet_no_weighted,
    nodeCoords=node_coords,
    nodeSizeDict={
            (node, layer): 0.012 if len(mnet_no_weighted[(node, layer)]) > 0 else 0
            for (node, layer) in mnet_no_weighted.iter_node_layers()
        },
    nodeColorDict=color_map,
    nodeLabelRule={},                  # No automatic labels
    defaultNodeLabel="",               # Hide labels
    defaultLayerAlpha=0.1,
    defaultLayerLabelLoc=(-0.01,0), 
    defaultEdgeAlpha=0.3,
    defaultNodeLabelSize=20,
    defaultLayerLabelSize=30,
    layergap=1.2,
    ax=ax
)

# Create a mappable object for the colorbar
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

norm = Normalize(vmin=min_val, vmax=max_val)
sm = ScalarMappable(norm=norm, cmap=colormap)
sm.set_array([])

# Add colorbar to the figure (adjust location as needed)
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.1, aspect=30)
cbar.set_label("GH_eig Gradient", fontsize=30)
cbar.ax.tick_params(labelsize=24)                       # Tick label font size


plt.savefig(f"imgs/madrid/eigen/GH-min.png", bbox_inches='tight')
plt.close(fig)


In [ ]:
IL_eig_np = IL_eig.numpy()
#transform to absolute values and normalize
IL_eig_np = np.abs(IL_eig_np)
min_val = IL_eig_np.min()
max_val = IL_eig_np.max()
IL_eig_grad = (IL_eig_np - min_val) / (max_val - min_val + 1e-8)  # Avoid divide-by-zero

# Use a matplotlib colormap
colormap = plt.get_cmap('viridis')  # or 'plasma', 'inferno', etc.

# Convert to RGB (exclude alpha channel)
IL_eig_colors_rgb = np.array([
    [colormap(IL_eig_grad[i, j])[:3] for j in range(IL_eig_grad.shape[1])]
    for i in range(IL_eig_grad.shape[0])
])

# Construct the dictionary {(node, layer): normalized_value}
color_map = {}
for layer_idx in range(IL_eig_grad.shape[0]):
    for node_idx in range(IL_eig_grad.shape[1]):
        layer_name = GH_int_to_layers[0][layer_idx]
        node_name = GH_int_to_nodes[node_idx]
        color_map[(node_name, layer_name)] = IL_eig_colors_rgb[layer_idx, node_idx]

fig = plt.figure(figsize=(34, 38))
ax = fig.add_subplot(111, projection='3d')
#add color gradient legend


draw(
    mnet_no_weighted,
    nodeCoords=node_coords,
    nodeSizeDict={
            (node, layer): 0.012 if len(mnet_no_weighted[(node, layer)]) > 0 else 0
            for (node, layer) in mnet_no_weighted.iter_node_layers()
        },
    nodeColorDict=color_map,
    nodeLabelRule={},                  # No automatic labels
    defaultNodeLabel="",               # Hide labels
    defaultLayerAlpha=0.1,
    defaultLayerLabelLoc=(-0.01,0), 
    defaultEdgeAlpha=0.3,
    defaultNodeLabelSize=20,
    defaultLayerLabelSize=30,
    layergap=1.2,
    ax=ax
)

# Create a mappable object for the colorbar
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

norm = Normalize(vmin=min_val, vmax=max_val)
sm = ScalarMappable(norm=norm, cmap=colormap)
sm.set_array([])

# Add colorbar to the figure (adjust location as needed)
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.1, aspect=30)
cbar.set_label("IL_eig Gradient", fontsize=30)
cbar.ax.tick_params(labelsize=24)                       # Tick label font size


plt.savefig(f"imgs/madrid/eigen/IL.png", bbox_inches='tight')
plt.close(fig)


In [92]:
LAYER = 'All'
new_net = MultiplexNetwork()

new_net.add_layer(LAYER)

for node, layer in mnet_no_weighted.iter_node_layers():
    new_net.add_node(node, layer=LAYER)
    for neighbor in mnet_no_weighted[(node, layer)]:
        if node != neighbor[0]:  # Avoid self-loops
            new_net[new_net._nodes_to_link((node, LAYER),(neighbor[0],LAYER))] = 1


#transform to absolute values and normalize
U_eig_np = np.abs(U_eig.numpy())
min_val = U_eig_np.min()
max_val = U_eig_np.max()
U_eig_grad = (U_eig_np - min_val) / (max_val - min_val + 1e-8)  # Avoid divide-by-zero

# Use a matplotlib colormap
colormap = plt.get_cmap('viridis')  # or 'plasma', 'inferno', etc.

# Use a matplotlib colormap
colormap = plt.get_cmap('viridis')  # or 'plasma', 'inferno', etc.

# Convert to RGB (exclude alpha channel)
U_eig_colors_rgb = np.array(
    [colormap(U_eig_grad[i])[:3] for i in range(U_eig_grad.shape[0])
])

# Construct the dictionary {(node, layer): normalized_value}
color_map = {}
for node_idx in range(U_eig_grad.shape[0]):
    layer_name = LAYER
    node_name = LH_int_to_nodes[node_idx]
    color_map[(node_name, layer_name)] = U_eig_colors_rgb[ node_idx]




nsd = {
        (node, layer): 0.013 if len(new_net[(node, layer)]) > 0 else 0
          for (node, layer) in new_net.iter_node_layers()
        
    }

fig = plt.figure(figsize=(22, 30))
ax = fig.add_subplot(111, projection='3d')

draw(
    new_net,
    nodeCoords=node_coords,
    nodeSizeDict=nsd,
    nodeColorDict=color_map,
    nodeLabelRule={},                  # No automatic labels
    defaultNodeLabel="",               # Hide labels
    defaultLayerAlpha=0.1,
    defaultLayerLabelLoc=(-0.01, 0),
    defaultEdgeAlpha=0.5,
    defaultNodeLabelSize=20,
    defaultLayerLabelSize=30,
    layergap=1.2,
    ax=ax
)
     
# Create a mappable object for the colorbar
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

norm = Normalize(vmin=min_val, vmax=max_val)
sm = ScalarMappable(norm=norm, cmap=colormap)
sm.set_array([])

# Add colorbar to the figure (adjust location as needed)
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.1, aspect=30)
cbar.set_label("U_eig Gradient", fontsize=30)
cbar.ax.tick_params(labelsize=24)                       # Tick label font size


plt.savefig(f"imgs/madrid/eigen/U_{LAYER}.png", bbox_inches='tight')
plt.close(fig)


In [370]:
# get 10 best IL_eig nodes for each layer
IL_eig_np_0 = np.abs(IL_eig[0].numpy())
IL_eig_np_1 = np.abs(IL_eig[1].numpy())
IL_eig_np_2 = np.abs(IL_eig[2].numpy())

# for each layer get the 10 best nodes
IL_eig_best_nodes = {   
    'urban': np.argsort(IL_eig_np_0)[-5:][::-1],
    'metro': np.argsort(IL_eig_np_1)[-5:][::-1],
    'interurban': np.argsort(IL_eig_np_2)[-5:][::-1]
}

IL_eig_best_nodes['interurban'] = [LH_int_to_nodes[node] for node in IL_eig_best_nodes['interurban']]
IL_eig_best_nodes['metro'] = [LH_int_to_nodes[node] for node in IL_eig_best_nodes['metro']]
IL_eig_best_nodes['urban'] = [LH_int_to_nodes[node] for node in IL_eig_best_nodes['urban']]

IL_eig_best_nodes

{'urban': ['USERA',
  'ESTACION DEL ARTE',
  'LEGAZPI',
  'MARQUES DE VADILLO',
  'PUERTA DEL ANGEL'],
 'metro': ['SOL', 'CALLAO', 'ALONSO MARTINEZ', 'GRAN VIA', 'TRIBUNAL'],
 'interurban': ['LA MORALEJA',
  'MARQUES DE LA VALDAVIA',
  'BEGOÑA',
  'BAUNATAL',
  'REYES CATOLICOS']}

In [371]:
U_eig_np = np.abs(U_eig.numpy())

U_eig_best_nodes =  np.argsort(U_eig_np)[-5:][::-1]

U_eig_best_nodes = [LH_int_to_nodes[node] for node in U_eig_best_nodes]
U_eig_best_nodes

['PLAZA ELIPTICA', 'USERA', 'DELICIAS', 'PAN BENDITO', 'LEGAZPI']

In [373]:
LH_eig_np_0 = np.abs(LH_eig[0].numpy())
LH_eig_np_1 = np.abs(LH_eig[1].numpy())
LH_eig_np_2 =  np.abs(LH_eig[2].numpy())

LH_eig_best_nodes = {   
    'urban': np.argsort(LH_eig_np_0)[-5:][::-1],
    'metro': np.argsort(LH_eig_np_1)[-5:][::-1],
    'interurban': np.argsort(LH_eig_np_2)[-5:][::-1]
}

LH_eig_best_nodes['interurban'] = [LH_int_to_nodes[node] for node in LH_eig_best_nodes['interurban']]
LH_eig_best_nodes['metro'] = [LH_int_to_nodes[node] for node in LH_eig_best_nodes['metro']]
LH_eig_best_nodes['urban'] = [LH_int_to_nodes[node] for node in LH_eig_best_nodes['urban']]
LH_eig_best_nodes


{'urban': ['PLAZA ELIPTICA',
  'USERA',
  'DELICIAS',
  'ESTACION DEL ARTE',
  'PALOS DE LA FRONTERA'],
 'metro': ['SOL', 'CALLAO', 'ALONSO MARTINEZ', 'GRAN VIA', 'TRIBUNAL'],
 'interurban': ['LA MORALEJA',
  'MARQUES DE LA VALDAVIA',
  'BEGOÑA',
  'BAUNATAL',
  'REYES CATOLICOS']}

In [362]:
GH_eig_np_0 = np.abs(GH_eig[0].numpy())
GH_eig_np_1 = np.abs(GH_eig[1].numpy())
GH_eig_np_2 = np.abs(GH_eig[2].numpy())

GH_eig_best_nodes = {
    'urban': np.argsort(GH_eig_np_0)[-5:][::-1],
    'metro': np.argsort(GH_eig_np_1)[-5:][::-1],
    'interurban': np.argsort(GH_eig_np_2)[-5:][::-1]
}

GH_eig_best_nodes['interurban'] = [GH_int_to_nodes[node] for node in GH_eig_best_nodes['interurban']]
GH_eig_best_nodes['metro'] = [GH_int_to_nodes[node] for node in GH_eig_best_nodes['metro']]
GH_eig_best_nodes['urban'] = [GH_int_to_nodes[node] for node in GH_eig_best_nodes['urban']]
GH_eig_best_nodes

{'urban': ['AVENIDA DE AMERICA',
  'CALLAO',
  'ALONSO MARTINEZ',
  'TRIBUNAL',
  'GRAN VIA'],
 'metro': ['SOL', 'CALLAO', 'ALONSO MARTINEZ', 'GRAN VIA', 'TRIBUNAL'],
 'interurban': ['AVENIDA DE AMERICA',
  'CALLAO',
  'ALONSO MARTINEZ',
  'TRIBUNAL',
  'GRAN VIA']}

In [361]:
important_nodes

{('ATOCHA', 'urban'),
 ('AVENIDA DE AMERICA', 'urban'),
 ('CASA DE CAMPO', 'metro'),
 ('CHAMARTIN', 'metro'),
 ('ESTACION DEL ARTE', 'urban'),
 ('LEGAZPI', 'metro'),
 ('MARQUES DE VADILLO', 'urban'),
 ('PLAZA ELIPTICA', 'metro'),
 ('PLAZA ELIPTICA', 'urban'),
 ('RUBEN DARIO', 'urban'),
 ('USERA', 'urban')}

In [379]:
for (node,layer)in important_nodes:
    if node in IL_eig_best_nodes['urban'] or node in IL_eig_best_nodes['metro'] or node in IL_eig_best_nodes['interurban']:
        print(f"IL_eig best node: {node} in layer {layer}")
    if node in U_eig_best_nodes:
        print(f"U_eig best node: {node} in layer {layer}")
    if node in LH_eig_best_nodes['urban'] or node in LH_eig_best_nodes['metro'] or node in LH_eig_best_nodes['interurban']:
        print(f"LH_eig best node: {node} in layer {layer}")
    if node in GH_eig_best_nodes['urban'] or node in GH_eig_best_nodes['metro'] or node in GH_eig_best_nodes['interurban']:
        print(f"GH_eig best node: {node} in layer {layer}")

IL_eig best node: USERA in layer urban
U_eig best node: USERA in layer urban
LH_eig best node: USERA in layer urban
IL_eig best node: LEGAZPI in layer metro
U_eig best node: LEGAZPI in layer metro
GH_eig best node: AVENIDA DE AMERICA in layer urban
IL_eig best node: MARQUES DE VADILLO in layer urban
U_eig best node: PLAZA ELIPTICA in layer metro
LH_eig best node: PLAZA ELIPTICA in layer metro
U_eig best node: PLAZA ELIPTICA in layer urban
LH_eig best node: PLAZA ELIPTICA in layer urban
IL_eig best node: ESTACION DEL ARTE in layer urban
LH_eig best node: ESTACION DEL ARTE in layer urban


#### Madrid SIR

In [303]:


nodel_layers = [('LORANCA', 'metro'), ('AVENIDA DE AMERICA', 'metro'),('USERA', 'metro')]


time_infections2 = { nl : {i:[] for i in range(200)} for nl in nodel_layers }


interlayers_beta = {
    (('metro',),('metro',)): 0.3,
    (('metro',), ('urban',)): 0.3,
    (('metro',), ('interurban',)): 0.3,

    (('urban',), ('urban',)): 0.3,
    (('urban',), ('interurban',)): 0.3,
    (('urban',), ('metro',)):0.3,

    (('interurban',), ('interurban',)): 0.3,
    (('interurban',), ('metro',)): 0.3,

    (('interurban',), ('urban',)): 0.3
}

intra_gamma = {
    ('metro',): 0.1,
    ('urban',): 0.1,
    ('interurban',): 0.1
}

for (node, layer) in nodel_layers:
    initial_state = {
        node:  0
        for node in mnet_no_weighted.iter_node_layers()
    }

    infections2 = {
    (n,l) : 0 for (n,l) in mnet_no_weighted.iter_node_layers()
    }
    initial_state[(node,layer)] = 1  # Infected node


    for i in range(800):
        print(f"Iteration {i+1}/2000")
        states = SIR_net_diffusion(
            mnet_no_weighted,
            initial_state=initial_state,
            interlayers_beta=interlayers_beta,
            intra_gamma=intra_gamma,
            iterations=1000
        )
        previous_state = {}
        for i in range(200):
            if i >= len(states):
                state = previous_state
            else:
                state = states[i]
            previous_state = state
            infected = sum(1 for value in state.values() if value == 1)
            healed = sum(1 for value in state.values() if value == 2)
            susceptible = sum(1 for value in state.values() if value == 0)
            time_infections2[(node,layer)][i].append((susceptible, infected, healed))
            for (n,l), value in state.items():
                if value == 1:
                    infections2[(n, l)] += 1

    nodeLabelDict = {
        (node, layer): ''
        for (node, layer) in mnet_weighted.iter_node_layers()
    }
    nodeLabelDict[(node,layer)] = node

    # use infections2 to set node colors, red the more infected, grey the less infected
    nodeColorDict = {
        (node, layer): plt.cm.Reds(infections2[(node, layer)] / max(infections2.values())) if infections2[(node, layer)] > 0 else 'grey'
        for (node, layer) in mnet_weighted.iter_node_layers()
    }

    nodeColorDict[(node,layer)] = 'black'



    fig = plt.figure(figsize=(34, 38))
    ax = fig.add_subplot(111, projection='3d')

    draw(
        mnet_no_weighted,
        nodeCoords=node_coords,
        nodeColorDict=nodeColorDict,
        nodeLabelDict=nodeLabelDict,       # Show only selected labels
        nodeLabelRule={},                  # No automatic labels
        defaultNodeLabelSize=40,
        defaultNodeLabel="",               # Hide labels
        defaultEdgeAlpha=0.4,
        nodeSizeDict={
                (node, layer): 0.012 if len(mnet_no_weighted[(node, layer)]) > 0 else 0
                for (node, layer) in mnet_no_weighted.iter_node_layers()
            }, # Scaled by community size
        layergap=1,
        ax=ax
    )
    print(f"Node {node} in layer {layer} infected {infections2[(node, layer)]} times.")
    plt.title(f'SIR Diffusion with {(node,layer)} Infected', fontsize=30)
    plt.savefig(f'imgs/madrid/SIR/sir_diffusion_direct_{node}_0.2.png', bbox_inches='tight')
    plt.close(fig)



mean_infections = {(node, layer): [] for (node, layer) in time_infections2.keys()}

mean_healed = {(node, layer): [] for (node, layer) in time_infections2.keys()}
mean_infections = {(node, layer): [] for (node, layer) in time_infections2.keys()}

for (node, layer), state_dict in time_infections2.items():
# Calculate the mean of susceptible, infected, and healed over time
    print(f"Processing {node} in layer {layer}")
    print(f"Number of time steps: {len(state_dict)}")
    for i, states in state_dict.items():
        infected_states = [state[1] for state in states]
        healed_states = [state[2] for state in states]
        susceptible_states = [state[0] for state in states]

    # mean_susceptible.append(np.mean(susceptible_states))
        mean_infections[(node, layer)].append(np.mean(infected_states))
        mean_healed[(node, layer)].append(np.mean(healed_states))

# Create the plot with all nodes and layers
fig, ax = plt.subplots(figsize=(14, 8))
for (node, layer), infections in mean_infections.items():
    ax.plot(infections, label=f'{node} ({layer})')
ax.set_xlabel('Time Steps')
ax.set_ylabel('Mean Infected Nodes')
ax.set_title('Mean Infected Nodes Over Time for Each Node and Layer')
ax.legend()
plt.grid(True, which="both", ls="--", linewidth=0.5)
plt.tight_layout()
plt.savefig('imgs/madrid/SIR/mean_infected_over_time_0.2.png', bbox_inches='tight')
plt.close(fig)

Iteration 1/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 2/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 3/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 4/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 5/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 6/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 7/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 8/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 9/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 10/2000
is_multiplex:  False


KeyboardInterrupt: 

In [302]:

mean_infections = {(node, layer): [] for (node, layer) in time_infections2.keys()}

mean_healed = {(node, layer): [] for (node, layer) in time_infections2.keys()}
mean_infections = {(node, layer): [] for (node, layer) in time_infections2.keys()}

for (node, layer), state_dict in time_infections2.items():
# Calculate the mean of susceptible, infected, and healed over time
    print(f"Processing {node} in layer {layer}")
    print(f"Number of time steps: {len(state_dict)}")
    for i, states in state_dict.items():
        infected_states = [state[1] for state in states]
        healed_states = [state[2] for state in states]
        susceptible_states = [state[0] for state in states]

    # mean_susceptible.append(np.mean(susceptible_states))
        mean_infections[(node, layer)].append(np.mean(infected_states))
        mean_healed[(node, layer)].append(np.mean(healed_states))

# Create the plot with all nodes and layers
fig, ax = plt.subplots(figsize=(14, 8))
for (node, layer), infections in mean_infections.items():
    ax.plot(infections[:81], label=f'{node} ({layer})')
ax.set_xlabel('Time Steps')
ax.set_ylabel('Mean Infected Nodes')
ax.set_title('Mean Infected Nodes Over Time for Each Node and Layer')
ax.legend()
plt.grid(True, which="both", ls="--", linewidth=0.5)
plt.tight_layout()
plt.savefig('imgs/madrid/SIR/mean_infected_over_time_direct_0.2.png', bbox_inches='tight')
plt.close(fig)

Processing LORANCA in layer metro
Number of time steps: 200
Processing AVENIDA DE AMERICA in layer metro
Number of time steps: 200
Processing USERA in layer metro
Number of time steps: 200


In [698]:
# Create the plot with all nodes and layers
fig, ax = plt.subplots(figsize=(14, 8))
for (node, layer), infections in mean_infections.items():
    ax.plot(infections[:80], label=f'{node} ({layer})')
ax.set_xlabel('Time Steps')
ax.set_ylabel('Mean Infected Nodes')
ax.set_title('Mean Infected Nodes Over Time for Each Node and Layer')
ax.legend()
plt.grid(True, which="both", ls="--", linewidth=0.5)
plt.tight_layout()
plt.savefig('imgs/madrid/SIR/mean_infected_over_time.png', bbox_inches='tight')
plt.close(fig)

In [100]:
node = 'USERA'
layer = 'metro'


interlayers_beta = {
    (('metro',),('metro',)): 0.3,
    (('metro',), ('urban',)): 0.2,
    (('metro',), ('interurban',)): 0.2,

    (('urban',), ('urban',)): 0.2,
    (('urban',), ('interurban',)): 1,
    (('urban',), ('metro',)):0.3,

    (('interurban',), ('interurban',)): 0.2,
    (('interurban',), ('metro',)): 0.3,

    (('interurban',), ('urban',)): 1
}

intra_gamma = {
    ('metro',): 0.1,
    ('urban',): 0.15,
    ('interurban',): 0.15
}


initial_state = {
    node:  0
    for node in mnet_no_weighted.iter_node_layers()
}

infections2 = {
(n,l) : 0 for (n,l) in mnet_no_weighted.iter_node_layers()
}
initial_state[(node,layer)] = 1  # Infected node


for i in range(1000):
    print(f"Iteration {i+1}/2000")
    states = SIR_net_diffusion(
        mnet_no_weighted,
        initial_state=initial_state,
        interlayers_beta=interlayers_beta,
        intra_gamma=intra_gamma,
        iterations=1000
    )
    for i, state in enumerate(states):
        infected = sum(1 for value in state.values() if value == 1)
        healed = sum(1 for value in state.values() if value == 2)
        susceptible = sum(1 for value in state.values() if value == 0)
        for (n,l), value in state.items():
            if value == 1:
                infections2[(n, l)] += 1

Iteration 1/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 2/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 3/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 4/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 5/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 6/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 7/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 8/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 9/2000
is_multiplex:  False
Epidemic has ended, all nodes are either susceptible or recovered.
Iteration 10/2000
is_multiplex:  False
Epidemic has end

In [ ]:

        
nodeLabelDict = {
    (node, layer): ''
    for (node, layer) in mnet_weighted.iter_node_layers()
}
nodeLabelDict[(node,layer)] = node

# use infections2 to set node colors, red the more infected, grey the less infected
nodeColorDict = {
    (node, layer): plt.cm.Reds(infections2[(node, layer)] / max(infections2.values())) if infections2[(node, layer)] > 0 else 'grey'
    for (node, layer) in mnet_weighted.iter_node_layers()
}

nodeColorDict[(node,layer)] = 'black'



fig = plt.figure(figsize=(34, 38))
ax = fig.add_subplot(111, projection='3d')

draw(
    mnet_no_weighted,
    nodeCoords=node_coords,
    nodeColorDict=nodeColorDict,
    nodeLabelDict=nodeLabelDict,       # Show only selected labels
    nodeLabelRule={},                  # No automatic labels
    defaultNodeLabelSize=40,
    defaultNodeLabel="",               # Hide labels
    defaultEdgeAlpha=0.4,
    nodeSizeDict={
            (node, layer): 0.012 if len(mnet_no_weighted[(node, layer)]) > 0 else 0
            for (node, layer) in mnet_no_weighted.iter_node_layers()
        }, # Scaled by community size
    layergap=1,
    ax=ax
)
print(f"Node {node} in layer {layer} infected {infections2[(node, layer)]} times.")

plt.title(f'SIR Diffusion with {(node,layer)} Infected', fontsize=30)
plt.savefig(f'imgs/madrid/SIR/sir_diffusion_{node}.png', bbox_inches='tight')
plt.close(fig)


Node USERA in layer metro infected 2896 times.


In [654]:
print(louv_com_2_name[madrid_louv_states[6]['node2com'][('LORANCA', 'metro')]])
print(louv_com_2_name[madrid_louv_states[6]['node2com'][('AVENIDA DE AMERICA', 'metro')]])
print(louv_com_2_name[madrid_louv_states[6]['node2com'][('USERA', 'metro')]])
print()
print(leid_com_2_name[leiden_states[-1]['node2com'][('LORANCA', 'metro')]])
print(leid_com_2_name[leiden_states[-1]['node2com'][('AVENIDA DE AMERICA', 'metro')]])
print(leid_com_2_name[leiden_states[-1]['node2com'][('USERA', 'metro')]])

CUATRO VIENTOS
AVENIDA DE AMERICA
PLAZA ELIPTICA

CONSERVATORIO
AVENIDA DE AMERICA
PLAZA ELIPTICA


In [645]:
louv_com_net, louv_com_sizes, louv_com_2_name = generate_louvain_communities_net(mnet_no_weighted, madrid_louv_states[6])


In [652]:
leid_com_2_name

{'JUAN DE LA CIERVA': 'GETAFE CENTRAL',
 'SAN CRISTOBAL': 'VILLAVERDE BAJO CRUCE',
 'PALOS DE LA FRONTERA': 'PLAZA ELIPTICA',
 'PUEBLO NUEVO': 'ALSACIA',
 'AEROPUERTO T1-T2-T3': 'CANILLEJAS',
 'LA PESETA': 'PAN BENDITO',
 'ACACIAS': 'PIRAMIDES',
 'MARQUES DE LA VALDAVIA': 'MARQUES DE LA VALDAVIA',
 'SAN FERNANDO': 'SAN FERNANDO',
 'VICALVARO': 'VICALVARO',
 'ARGÜELLES': 'PRINCIPE PIO',
 'ARROYO CULEBRO': 'CONSERVATORIO',
 'PARQUE DE SANTA MARIA': 'COLOMBIA',
 'UNIVERSIDAD REY JUAN CARLOS': 'PRADILLO',
 'AVENIDA DE AMERICA': 'AVENIDA DE AMERICA',
 'RIVAS VACIAMADRID': 'RIVAS VACIAMADRID',
 'VILLA DE VALLECAS': 'SIERRA DE GUADALUPE',
 'SANTIAGO BERNABEU': 'BEGOÑA',
 'BATAN': 'CAMPAMENTO'}

In [651]:
leid_com_net, leid_com_sizes, leid_com_2_name = generate_leiden_communities_net(mnet_no_weighted, leiden_states[-1])

## 2.2. ---Tensorflow functions to Madrid data---

In [ ]:
print(len(mnet_no_weighted[('PAN BENDITO','urban')]))
print(len(mnet_no_weighted[('PAN BENDITO','metro')]))
print(len(mnet_no_weighted[('PAN BENDITO','interurban')]))

for neighbor in mnet_no_weighted[('PAN BENDITO','urban')]:
    print(neighbor)

7
4
7
('ABRANTES', 'urban')
('HOSPITAL 12 DE OCTUBRE', 'urban')
('LA PESETA', 'urban')
('PLAZA ELIPTICA', 'urban')
('SAN FERMIN-ORCASUR', 'urban')
('SAN FRANCISCO', 'urban')
('PAN BENDITO', 'metro')


In [575]:
sparse, nodes, layers, inv_nodes, inv_layers = __get_sparse_tensor__(mnet_weighted, return_mappings=True)
sparse_inter = __get_sparse_tensor_between_layers__(mnet_weighted, ['metro'], ['urban'])
print(sparse)

SparseTensor(indices=tf.Tensor(
[[  0   0   0   2]
 [  0   0   2   0]
 [  0  17   0   0]
 ...
 [241 226   0   0]
 [241 241   0   2]
 [241 241   2   0]], shape=(2488, 4), dtype=int64), values=tf.Tensor([0.12346772 0.12346772 0.02445602 ... 0.32881767 0.09640628 0.09640628], shape=(2488,), dtype=float32), dense_shape=tf.Tensor([242 242   3   3], shape=(4,), dtype=int64))


## 2.3. ---Centrality measures to Madrid data---

In [550]:
print(mu_PCI(mnet_no_weighted, ('AVENIDA DE AMERICA', 'metro'), mu = 1))
print(mu_PCI(mnet_no_weighted, ('AVENIDA DE AMERICA', 'metro'), mu = 2))

# for each node in the mnet_no_weightedwork store the mu_PCI, mlPCI_n, allPCI and lsPCI in a dictionary
nl_PCIs = {}
for node in mnet_no_weighted.iter_node_layers():
    nl_PCIs[node] = {
        'mu_PCI': mu_PCI(mnet_no_weighted, node, mu=1),
        'mlPCI_n': mlPCI_n(mnet_no_weighted, node),
        'allPCI': allPCI(mnet_no_weighted, node),
        'lsPCI': lsPCI(mnet_no_weighted, node)
    }

4
7


In [556]:
# show nodes with lsPCI > 1
high_lsPCI_nodes = {node: pci['lsPCI'] for node, pci in nl_PCIs.items() if pci['lsPCI'] > 1}
print(f"{len(high_lsPCI_nodes)} Nodes with lsPCI > 1:")
# for node, lsPCI in high_lsPCI_nodes.items():
#     print(f"{node}: {lsPCI}")

230 Nodes with lsPCI > 1:


In [557]:
# show nodes with lsPCI > 1
high_allPCI_nodes = {node: pci['allPCI'] for node, pci in nl_PCIs.items() if pci['allPCI'] > 1}
print(f"{len(high_allPCI_nodes)} Nodes with allPCI > 1:")
# for node, lsPCI in high_lsPCI_nodes.items():
#     print(f"{node}: {lsPCI}")

0 Nodes with allPCI > 1:


In [595]:
tensor, nodes2int, layers2int, int2nodes, int2layers = __get_sparse_tensor__(mnet_no_weighted, return_mappings=True)
inv_layers

[{0: 'interurban', 1: 'urban', 2: 'metro'}]

In [651]:
ILEC_madrid, ILEC_int2layers = independent_layer_eigenvector_centrality(mnet_no_weighted)
UEC_madrid = uniform_eigenvector_centrality(mnet_no_weighted)
LHEC_madrid, LHEC_nodes2int, LHEC_layers2int, LHEC_int2nodes, LHEC_int2layers  = local_heterogeneus_eigenvector_centrality(mnet_no_weighted)
GHEC_madrid, GHEC_nodes2int, GHEC_layers2int, GHEC_int2nodes, GHEC_int2layers = global_heterogeneus_eigenvector_centrality(mnet_no_weighted)

print("ILEC_madrid:", ILEC_madrid.shape)
print("UEC_madrid:", UEC_madrid.shape)
print("LHEC_madrid:", LHEC_madrid.shape)
print("GHEC_madrid:", GHEC_madrid.shape)

ILEC_madrid: (3, 242)
UEC_madrid: (242,)
LHEC_madrid: (3, 242)
GHEC_madrid: (3, 242)


In [655]:
NODE = 'AVENIDA DE AMERICA'
NODE = 'PLAZA ELIPTICA'
LAYER = 'urban'
print(GHEC_madrid[GHEC_layers2int[0][LAYER]][GHEC_nodes2int[NODE]])

print(LHEC_madrid[LHEC_layers2int[0][LAYER]][LHEC_nodes2int[NODE]])


tf.Tensor(1.2144022e-21, shape=(), dtype=float32)
tf.Tensor(0.19444656, shape=(), dtype=float32)


## 2.4. --- SIR ---

In [ ]:
interlayers_beta = {
    (('metro',),('metro',)): 65000,
    (('metro',), ('urban',)): 4,
    (('metro',), ('interurban',)): 0.9,

    (('urban',), ('urban',)): 50000,
    (('urban',), ('interurban',)): 10000000,
    (('urban',), ('metro',)):4,

    (('interurban',), ('interurban',)): 50000,
    (('interurban',), ('metro',)): 4,

    (('interurban',), ('urban',)): 10000000
}
intra_gamma = {
    ('metro',): 0.1,
    ('urban',): 0.2,
    ('interurban',): 0.2
}

initial_state = {
    node:  0
    for node in mnet_weighted.iter_node_layers()
}

initial_state[('AVENIDA DE AMERICA', 'metro')] = 1  # Infected node
initial_state[('PLAZA ELIPTICA', 'urban')] = 1  # Infected node

nodeSizeDict = {
    (node, layer): 0.01 
    for (node, layer) in mnet_no_weighted.iter_node_layers()
}

final_states = SIR_net_diffusion( mnet_weighted,
    interlayers_beta=interlayers_beta,
    intra_gamma=intra_gamma,
    iterations= 10,
    initial_state=initial_state
)

for i, state in enumerate(final_states):
# plot the network labeling only the starting infected nodes, all infected nodes are in red
    fig = plt.figure(figsize=(36, 24))
    ax = fig.add_subplot(111, projection='3d')

    nodeColorDict = {
        (node, layer): 'red' if state[(node, layer)] == 1 else 'blue' if state[(node, layer)] == 2 else 'gray'
        for (node, layer) in mnet_weighted.iter_node_layers()
    }
    #initial_state eq 1
    nodeLabelDict = {
        (node, layer): node if initial_state[(node, layer)] == 1 else ''
        for (node, layer) in mnet_weighted.iter_node_layers()
    }
    draw(
        mnet_weighted,
        nodeCoords=node_coords,
        nodeColorDict=nodeColorDict,
        nodeLabelDict=nodeLabelDict,       # Show only selected labels
        nodeLabelRule={},                  # No automatic labels
        defaultNodeLabel="",               # Hide labels
        defaultEdgeAlpha=0.4,
        nodeSizeDict=nodeSizeDict, # Scaled by community size
        layergap=1,
        ax=ax
    )
    # print('infected nodes:', [node for node, state in state.items() if state == 1])
    # print('recovered nodes:', [node for node, state in state.items() if state == 2])

    plt.title(f"State after {i} iterations")
    plt.savefig(f'imgs/SIR/sir_state_{i}.png', dpi=300, bbox_inches='tight')


## other approach

### louvain direct

In [47]:
def init_multiplex_communities_louvain_direct(self):
    # all representation of a node is within the same community


    communities = {
        'com2nodes' : {nl: [nl] for nl in self.iter_node_layers() if len(self[nl]) > 0},
        'node2com' : {nl: nl for nl in self.iter_node_layers() if len(self[nl]) > 0},
        'com_inner_weight' : {nl : 0 for nl in self.iter_node_layers() if len(self[nl]) > 0},
        'com_total_weight' : {nl : 0 for nl in self.iter_node_layers() if len(self[nl]) > 0},
        'neigh_coms' : {nl : set() for nl in self.iter_node_layers() if len(self[nl]) > 0},
        'graph_size' : len(self.edges),
        'objective_function_value' : 0,
    }

    for com, nodes in communities['com2nodes'].items():
        inner_w = 0
        total_w = 0
        for node1 in nodes:
            for neighbor in self._iter_neighbors(node1):
                total_w += self[node1][neighbor]
        communities['com_inner_weight'][com] = inner_w
        communities['com_total_weight'][com] = total_w
        communities['neigh_coms'][com].update([x for node in nodes for x in self._iter_neighbors(node)])
    
    
    # communities['node2com'] = {node: com for com, nodes in communities['com2nodes'].items() for node in nodes}
    communities['objective_function_value'] = modularity(communities)
    return communities    



def louvain_algorithm_direct(self, max_iter=1000, force_merge=False):
    """
    Apply the Louvain algorithm to find communities in a multiplex network.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    max_iter : int, optional
        The maximum number of iterations to run the algorithm (default is 1000).
    force_merge : bool, optional
        If True, forces the merging of communities with the smallest modularity loss when no improvement is found.
    
    Returns
    -------
    dict
        A dictionary containing the communities and their properties.
    """
    
    communities = init_multiplex_communities_louvain_direct(self)
    print
    states = [communities]
    for _ in range(max_iter):
        print(f"Iteration {_+1}/{max_iter}")
        new_communities = move_nodes_step(self, communities, force_merge=force_merge)
        if len(new_communities['com2nodes']) == len(communities['com2nodes']):
            # if there is no improvement, we stop
            print("No movements found, stopping.")
            states.append(new_communities)
            break
        if len(new_communities['com2nodes']) == 1:
            # if there is only one community, we stop
            states.append(new_communities)
            break

        states.append(new_communities)
        communities = new_communities
    
    return states  

louv_direc_states = louvain_algorithm_direct(mnet_no_weighted, max_iter=1000, force_merge=True)


Iteration 1/1000
Iteration 2/1000
Iteration 3/1000
Iteration 4/1000
Iteration 5/1000
Iteration 6/1000
Iteration 7/1000
Iteration 8/1000
Iteration 9/1000
Iteration 10/1000
Iteration 11/1000
Iteration 12/1000
Iteration 13/1000
Iteration 14/1000
Iteration 15/1000


In [72]:
best_state = louv_direc_states[4]
best_state['node2com']
nodes_coms = {n[0]:set()for n in mnet_no_weighted.iter_node_layers() if len(mnet_no_weighted[n]) > 0}
for node, com in best_state['node2com'].items():
    nodes_coms[node[0]].add(com)

nodes_coms
    

{'PACIFICO': {('HOSPITAL 12 DE OCTUBRE', 'metro')},
 'EMBAJADORES': {('HOSPITAL 12 DE OCTUBRE', 'metro')},
 'JULIAN BESTEIRO': {('LA PESETA', 'metro')},
 'CONDE DE CASAL': {('PORTAZGO', 'metro')},
 'PARQUE DE LOS ESTADOS': {('LA PESETA', 'metro')},
 'SIMANCAS': {('AVENIDA DE GUADALAJARA', 'urban')},
 'MIRASIERRA': {('TRES OLIVOS', 'urban')},
 'ALTO DEL ARENAL': {('PORTAZGO', 'metro')},
 'LAS ROSAS': {('AVENIDA DE GUADALAJARA', 'urban')},
 'BUENOS AIRES': {('PORTAZGO', 'metro')},
 'ARGANDA DEL REY': {('PORTAZGO', 'metro')},
 'ESTRELLA': {('PORTAZGO', 'metro')},
 'LA ELIPA': {('AVENIDA DE GUADALAJARA', 'urban')},
 'PARQUE DE SANTA MARIA': {('ESPERANZA', 'urban')},
 'HOSPITAL DE FUENLABRADA': {('MANUELA MALASAÑA', 'interurban')},
 'LA MORALEJA': {('PINAR DE CHAMARTIN', 'interurban')},
 'EL CAPRICHO': {('SAN FERNANDO', 'interurban')},
 'ISLAS FILIPINAS': {('CANAL', 'metro')},
 'PIRAMIDES': {('HOSPITAL 12 DE OCTUBRE', 'metro')},
 'VALDEBERNARDO': {('PORTAZGO', 'metro')},
 'QUEVEDO': {('CANA

In [121]:
##### new net generation functions #####

def communities_are_neighbors(net, com1, com2, communities):
    """
    Check if two communities are neighbors in the network.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    com1 : str
        The first community.
    com2 : str
        The second community.
    communities : dict
        A dictionary containing the communities of the network.

    Returns
    -------
    bool
        True if the communities are neighbors, False otherwise.
    """
    
    for node1 in communities['com2nodes'][com1]:
        for node2 in net._iter_neighbors(node1):
            if node2 in communities['com2nodes'][com2]:
                return True
    return False

def get_highest_degree_node_direct(net, communities, com):
    """
    Get the node with the highest degree in a given community.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    communities : dict
        A dictionary containing the communities of the network.
    com : str
        The community to analyze.

    Returns
    -------
    tuple
        The node with the highest degree in the community.
    """

    nodes_no_layers_degree = { n_l : 0 for n_l in communities['com2nodes'][com]}

    for node1 in communities['com2nodes'][com]:
        nodes_no_layers_degree[node1] += net._get_degree(node1)
    # get the node with the highest degree
    highest_degree_node = max(nodes_no_layers_degree, key=nodes_no_layers_degree.get)
    return highest_degree_node
        
    
def generate_louvain_communities_net_direct(self,communities):
    """
    Generate a new network from the communities of a multilayer network.
    
    Parameters
    ----------
    self : MultilayerNetwork
        The multilayer network to analyze.
    communities : dict
        A dictionary containing the communities of the network.

    Returns
    -------
    MultilayerNetwork
        A new multilayer network with the communities as nodes and edges between them.
    """
    
    com_net = MultilayerNetwork(aspects=1, directed=False)
    for layer in self.slices[1]:
        com_net.add_layer(layer)
        
    com_2_name = {com:"" for com in communities['com2nodes'].keys()}
    for (com_n,com_l), nodes in communities['com2nodes'].items():
        com_2_name[(com_n,com_l)] = get_highest_degree_node_direct(self, communities, (com_n,com_l))
    for com in communities['com2nodes'].keys():
        com_net.add_node(com_2_name[com][0])

    neighbors = defaultdict(set)
    for com in communities['com2nodes'].keys():
        for com2 in communities['com2nodes'].keys():
            if com != com2 and communities_are_neighbors(self, com, com2, communities):
                com_name = com_2_name[com]
                com2_name = com_2_name[com2]
                neighbors[com_name].add(com2_name)
            if com2 != com and communities_are_neighbors(self, com2, com, communities):
                com_name = com_2_name[com]
                com2_name = com_2_name[com2]
                neighbors[com2_name].add(com_name)

    for com, neighbors_set in neighbors.items():
        for neighbor in neighbors_set:
            com_net[com][neighbor] = 1  # or any weight you want, here we use 1

    com_sizes = {com_2_name[com]: len(nodes) for com, nodes in communities['com2nodes'].items()}

    

    return com_net, com_sizes, com_2_name


In [182]:
best_state = louv_direc_states[4]
louv_com_net, louv_com_sizes, louv_com_2_name = generate_louvain_communities_net_direct(mnet_no_weighted, best_state)
colors = plt.get_cmap('tab20', len(best_state['com2nodes']))
# map each community to a color

nodeSizeDict = {
    # if the node has more than 3 neighbors, set the size to 0.01, else 0
    nl: 0.01 if len(mnet_no_weighted[nl]) > 2 else 0
    for nl in nodeSizeDict.keys()
}

nodeColorDict_louv_base = {
    (node,layer) :nodeColorDict_louv[louv_com_2_name[best_state['node2com'][(node,layer)]]]
    for node, layer in mnet_no_weighted.iter_node_layers() if len(mnet_no_weighted[(node, layer)]) > 0    
}

node_com = {
    node: set()   for node, layer in mnet_no_weighted.iter_node_layers() 
}
for node, com in best_state['node2com'].items():
    node_com[node[0]].add(com)

node_com = {node: com for node, com in node_com.items() if len(com) > 1}
node_com

nodeLabelDict = {
    ('AVENIDA DE AMERICA', 'metro'): louv_com_2_name[best_state['node2com'][('AVENIDA DE AMERICA', 'metro')]],
    ('AVENIDA DE AMERICA', 'urban'): louv_com_2_name[best_state['node2com'][('AVENIDA DE AMERICA', 'urban')]],
    ('AVENIDA DE AMERICA', 'interurban'): louv_com_2_name[best_state['node2com'][('AVENIDA DE AMERICA', 'interurban')]],
    ('PLAZA DE CASTILLA', 'metro'): louv_com_2_name[best_state['node2com'][('PLAZA DE CASTILLA', 'metro')]],
    ('PLAZA DE CASTILLA', 'urban'): louv_com_2_name[best_state['node2com'][('PLAZA DE CASTILLA', 'urban')]],
    ('PLAZA DE CASTILLA', 'interurban'): louv_com_2_name[best_state['node2com'][('PLAZA DE CASTILLA', 'interurban')]],
    ('VICALVARO', 'metro'): louv_com_2_name[best_state['node2com'][('VICALVARO', 'metro')]],
    ('VICALVARO', 'urban'): louv_com_2_name[best_state['node2com'][('VICALVARO', 'urban')]],
    ('VICALVARO', 'interurban'): louv_com_2_name[best_state['node2com'][('VICALVARO', 'interurban')]]
}
nodeLabelColorDict_louv = {
    ('AVENIDA DE AMERICA', 'metro'): 'black',
    ('AVENIDA DE AMERICA', 'urban'): 'black',
    ('AVENIDA DE AMERICA', 'interurban'): 'black',
    ('PLAZA DE CASTILLA', 'metro'): 'blue',
    ('PLAZA DE CASTILLA', 'urban'): 'blue',
    ('PLAZA DE CASTILLA', 'interurban'): 'blue',
    ('VICALVARO', 'metro'): 'red',
    ('VICALVARO', 'urban'): 'red',
    ('VICALVARO', 'interurban'): 'red'
}

fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')
# add legend black for AVENIDA DE AMERICA, blue for PLAZA DE CASTILLA, red for VICALVARO
legend_handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=10, label='AVENIDA DE AMERICA'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='PLAZA DE CASTILLA'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='VICALVARO')
]
ax2.legend(handles=legend_handles, loc='upper left', fontsize=30)


draw(
        mnet_no_weighted,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict=nodeSizeDict,  # Use global node sizes
        nodeColorDict=nodeColorDict_louv_base,
        nodeLabelDict=nodeLabelDict,       # Show only selected labels
        nodeLabelColorDict=nodeLabelColorDict_louv,  # Use specific colors for labels
        nodeLabelRule={},                  # No automatic labels
        defaultNodeLabelSize=20,
        defaultLayerLabelSize=30,
        defaultLayerLabelLoc=(-0.01, 0),
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.2,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )
fig2.savefig(f'imgs/madrid/base_louv_direct.png', bbox_inches='tight')
plt.close(fig2)

fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')



### Leiden direct

In [171]:


def leiden_algorithm(self, gamma=1.0, max_iterations=100):
    """
    Perform the Leiden algorithm for community detection in a multilayer network.

    This algorithm identifies communities through iterative improvement based on the Constant Potts Model (CPM).

    Args:
        net (MultilayerNetwork): The multilayer network to analyze.
        gamma (float, optional): Resolution parameter. Higher values lead to smaller communities.
        max_iterations (int, optional): Maximum number of iterations to run. Default is 100.

    Returns:
        list of dict: List of community states for each iteration until convergence.
    """

    
    communities = init_multiplex_communities_leiden_direct(self,gamma)
    states = [deepcopy(communities)]
    for _ in range(max_iterations):
        print(f"Iteration {_+1}/{max_iterations}")
        old_communities, communities, merge_history = leiden_FastNodeMove(self, deepcopy(communities))

        communities = refine_partition(self, deepcopy(communities),deepcopy(old_communities), merge_history)

        # check if the com2nodes of old_communities and communities are the same
        if old_communities['com2nodes'] == communities['com2nodes']:
            print("Convergence reached.")
            break
        else:
            states.append(communities)


    return states


def init_multiplex_communities_leiden_direct(self,gamma):
    """
    Initialize each node-layer pair in its own community for the Leiden algorithm.

    Args:
        net (MultilayerNetwork): The network to initialize communities on.

    Returns:
        dict: A dictionary with initial community structures including:
            - com2nodes: Community to node-layer mapping.
            - node2com: Node-layer to community mapping.
            - com_inner_weight: Intra-community weights.
            - com_total_weight: Total weights per community.
            - neigh_coms: Neighboring community relationships.
            - graph_size: Total number of edges.
    """


    communities = {
        'com2nodes' : {nl: [nl] for nl in self.iter_node_layers()if len(self[nl]) > 0},
        'node2com' : {nl: nl for nl in self.iter_node_layers() if len(self[nl]) > 0},
        'com_inner_weight' : {nl : 0 for nl in self.iter_node_layers() if len(self[nl]) > 0},
        'com_total_weight' : {nl : 0 for nl in self.iter_node_layers() if len(self[nl]) > 0},
        'neigh_coms' : {nl : set() for nl in self.iter_node_layers() if len(self[nl]) > 0},
        'graph_size' : len(self.edges),
        'gamma': gamma,

        'objective_function_value' : 0,
    }

    for com, nodes in communities['com2nodes'].items():
        inner_w = 0
        total_w = 0
        for node1 in nodes:
            for neighbor in self._iter_neighbors(node1):
                total_w += self[node1][neighbor]
        communities['com_total_weight'][com] = total_w
        communities['neigh_coms'][com].update([x for node in nodes for x in self._iter_neighbors(node)])
    
    # communities['node2com'] = {node: com for com, nodes in communities['com2nodes'].items() for node in nodes}
    communities['objective_function_value'] = CPM(communities)
    return communities

leid_states = leiden_algorithm(mnet_no_weighted, gamma=0.01, max_iterations=100)

Iteration 1/100
Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100
Iteration 7/100
Convergence reached.


In [ ]:
def generate_leiden_communities_net_direct(net, communities):
    """
    Generate a new network from the communities of a multiplex network using the Leiden algorithm.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    communities : dict
        A dictionary containing the communities of the network.

    Returns
    -------
    MultilayerNetwork
        A new multilayer network with the communities as nodes and edges between them.
    """
    
    com_net = MultilayerNetwork(aspects=1, directed=False)
    com_net.add_layer(1)  # Add a single layer for the communities


    
    com_2_name = {com:"" for com in communities['com2nodes'].keys()}
    for com, nodes in communities['com2nodes'].items():
        com_2_name[com] = get_highest_degree_node(net, communities, com)


    for com in communities['com2nodes'].keys():
        com_net.add_node(com_2_name[com])

    neighbors = defaultdict(set)
    for com in communities['com2nodes'].keys():
        for com2 in communities['neigh_coms'][com]:
            if com != com2:
                com_name = com_2_name[com]
                com2_name = com_2_name[com2]
                neighbors[com_name].add(com2_name)
    for com, neighbors_set in neighbors.items():
        for neighbor in neighbors_set:
            com_net[(com,1)][(neighbor,1)] = 1  # or any weight you want, here we use 1

    com_sizes = {com_2_name[com]: len(nodes) for com, nodes in communities['com2nodes'].items()}

    return com_net, com_sizes, com_2_name

In [192]:
best_state = leid_states[-1]
leid_com_net, leid_com_sizes, leid_com_2_name = generate_louvain_communities_net_direct(mnet_no_weighted, best_state)
colors = plt.get_cmap('tab20', len(best_state['com2nodes']))
# map each community to a color

nodeSizeDict = {
    # if the node has more than 3 neighbors, set the size to 0.01, else 0
    nl: 0.01 if len(mnet_no_weighted[nl]) > 2 else 0
    for nl in nodeSizeDict.keys()
}
nodeColorDict_leid = {
    node: colors(i) 
    for i, node in enumerate(leid_com_sizes.keys())
}
nodeColorDict_leid_base = {
    (node,layer) :nodeColorDict_leid[leid_com_2_name[best_state['node2com'][(node,layer)]]]
    for node, layer in mnet_no_weighted.iter_node_layers() if len(mnet_no_weighted[(node, layer)]) > 0    
}

node_com = {
    node: set()   for node, layer in mnet_no_weighted.iter_node_layers() 
}
for node, com in best_state['node2com'].items():
    node_com[node[0]].add(com)

node_com = {node: com for node, com in node_com.items() if len(com) > 1}
node_com

nodeLabelDict = {
    ('AVENIDA DE AMERICA', 'metro'): leid_com_2_name[best_state['node2com'][('AVENIDA DE AMERICA', 'metro')]],
    ('AVENIDA DE AMERICA', 'urban'): leid_com_2_name[best_state['node2com'][('AVENIDA DE AMERICA', 'urban')]],
    ('AVENIDA DE AMERICA', 'interurban'): leid_com_2_name[best_state['node2com'][('AVENIDA DE AMERICA', 'interurban')]],
    ('PLAZA ELIPTICA', 'metro'): leid_com_2_name[best_state['node2com'][('PLAZA ELIPTICA', 'metro')]],
    ('PLAZA ELIPTICA', 'urban'): leid_com_2_name[best_state['node2com'][('PLAZA ELIPTICA', 'urban')]],
    ('PLAZA ELIPTICA', 'interurban'): leid_com_2_name[best_state['node2com'][('PLAZA ELIPTICA', 'interurban')]],
    ('VICALVARO', 'metro'): leid_com_2_name[best_state['node2com'][('VICALVARO', 'metro')]],
    ('VICALVARO', 'urban'): leid_com_2_name[best_state['node2com'][('VICALVARO', 'urban')]],
    ('VICALVARO', 'interurban'): leid_com_2_name[best_state['node2com'][('VICALVARO', 'interurban')]]
}
nodeLabelColorDict_leid = {
    ('AVENIDA DE AMERICA', 'metro'): 'black',
    ('AVENIDA DE AMERICA', 'urban'): 'black',
    ('AVENIDA DE AMERICA', 'interurban'): 'black',
    ('PLAZA ELIPTICA', 'metro'): 'blue',
    ('PLAZA ELIPTICA', 'urban'): 'blue',
    ('PLAZA ELIPTICA', 'interurban'): 'blue',
    ('VICALVARO', 'metro'): 'red',
    ('VICALVARO', 'urban'): 'red',
    ('VICALVARO', 'interurban'): 'red'
}

fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')
# add legend black for AVENIDA DE AMERICA, blue for PLAZA ELIPTICA, red for VICALVARO
legend_handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=10, label='AVENIDA DE AMERICA'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='PLAZA ELIPTICA'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='VICALVARO')
]
ax2.legend(handles=legend_handles, loc='upper left', fontsize=30)


draw(
        mnet_no_weighted,
        nodeCoords=node_coords,  # Use the same coordinates as the original network
        nodeSizeDict=nodeSizeDict,  # Use global node sizes
        nodeColorDict=nodeColorDict_leid_base,
        nodeLabelDict=nodeLabelDict,       # Show only selected labels
        nodeLabelColorDict=nodeLabelColorDict_leid,  # Use specific colors for labels
        nodeLabelRule={},                  # No automatic labels
        defaultNodeLabelSize=20,
        defaultLayerLabelSize=30,
        defaultLayerLabelLoc=(-0.01, 0),
        defaultNodeLabel="",               # Hide labels
        defaultLayerAlpha=0.2,
        defaultEdgeAlpha=0.5,
        ax=ax2
    )
fig2.savefig(f'imgs/madrid/leiden/base_leid_direct.png', bbox_inches='tight')
plt.close(fig2)

fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')


In [191]:
nodeComs = {node[0]:set() for node in mnet_no_weighted.iter_node_layers() }
for node, com in best_state['node2com'].items():
    nodeComs[node[0]].add(com)
nodeComs = {node: com for node, com in nodeComs.items() if len(com) > 1}
nodeComs

{'CANILLEJAS': {('AEROPUERTO T1-T2-T3', 'metro'), ('MANUEL BECERRA', 'metro')},
 'SOL': {('CHUECA', 'urban'), ('LA LATINA', 'urban')},
 'AEROPUERTO T-4': {('AEROPUERTO T1-T2-T3', 'metro'), ('FUENCARRAL', 'urban')},
 'PUENTE DE VALLECAS': {('LA LATINA', 'urban'), ('NUEVA NUMANCIA', 'metro')},
 'PINAR DE CHAMARTIN': {('ESPERANZA', 'urban'), ('FUENCARRAL', 'urban')},
 'AVIACION ESPAÑOLA': {('HOSPITAL DE MOSTOLES', 'metro'),
  ('PAN BENDITO', 'urban')},
 'HORTALEZA': {('ESPERANZA', 'urban'), ('FUENCARRAL', 'urban')},
 'PARQUE DE LAS AVENIDAS': {('AEROPUERTO T1-T2-T3', 'metro'),
  ('ESPERANZA', 'urban'),
  ('MANUEL BECERRA', 'metro')},
 'VICALVARO': {('NUEVA NUMANCIA', 'metro'), ('SAN BLAS', 'urban')},
 'FERIA DE MADRID': {('ESPERANZA', 'urban'), ('FUENCARRAL', 'urban')},
 'COLON': {('ALVARADO', 'metro'), ('CHUECA', 'urban')},
 'ALFONSO XIII': {('ESPERANZA', 'urban'), ('MANUEL BECERRA', 'metro')},
 'PROSPERIDAD': {('ESPERANZA', 'urban'), ('MANUEL BECERRA', 'metro')},
 'TORRE ARIAS': {('AERO

### SIR